### Initial imports

In [1]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
import pandas as pd
import numpy as np
import os
import re
from dateutil.relativedelta import relativedelta
from statsmodels.tsa import seasonal
from matplotlib import pyplot as plt
from functools import reduce  # Operação de reduce para cálculo de média de uma lista
from datetime import datetime
import math
%matplotlib inline

#Configurações para exibição de tableas
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)



/Users/Tiago/Library/Python/3.6/lib/python/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


height has been deprecated.



### Auxiliary functions

In [2]:
# Nomes das colunas adicionadas ao dataframe
COLUNA_ANOMALIA_ACUMULADA = "anomalia_acumulada"
COLUNA_ANOMALIA_DO_MES = "anomalia_mensal"
COLUNA_MEDIA_MENSAL = "media_mensal"

# Todo mês possui 25 linhas por 38 colunas que dá 950
BLOCO_DE_DADOS_DE_UM_MES = 950
QUANTIDADE_DE_VALORES_DO_ARQUIVO = 573800  # (950 blocos x 604 meses,01/1964 até 04/2014)

class LatitudeColumns:

    def __init__(self):
        self.values = range(29, -21, -2)
        self.positive = 'N'
        self.negative = 'S'

    def get_all_columns(self):
        colum_names = []
        for value in self.values:
            if value > 0:
                colum_name = str(value) + self.positive
            else:
                colum_name = str(value) + self.negative
            colum_names.append(colum_name)
        return colum_names

    def get_single_column(self, desired_value, human_readable=True):
        sub_values = []
        for value in self.values:
            if value != desired_value:
                continue
            sub_values.append(value)
        if human_readable:
            return self.parse_human_readable(sub_values)
        return sub_values

    def get_range(self, init, final, human_readable=True):
        final, init = self.switch(final, init)

        sub_values = []
        if init == final:
            return self.get_single_column(init, human_readable)

        for value in self.values:
            if (value > init) or (value < final):
                continue
            sub_values.append(value)

        if human_readable:
            return self.parse_human_readable(sub_values)

        return sub_values

    def switch(self, final, init):
        if final > init:
            aux = init
            init = final
            final = aux
        return final, init

    def parse_human_readable(self, list):
        colum_names = []
        for value in list:
            if value > 0:
                colum_name = str(value) + self.positive
            else:
                colum_name = str(-value) + self.negative
            colum_names.append(colum_name)
        return colum_names

class LongitudeColumns:

    def __init__(self):
        #From map 60W to 15E(-17 because 'force' go to -15)
        self.values = range(59, -17, -2)
        self.positive = 'W'
        self.negative = 'E'

    def get_all_columns(self):
        colum_names = []
        for value in self.values:
            if value > 0:
                colum_name = str(value) + self.positive
            else:
                colum_name = str(value) + self.negative
            colum_names.append(colum_name)
        return colum_names

    def get_single_column(self, desired_value, human_readable=True):
        sub_values = []
        for value in self.values:
            if value != desired_value:
                continue
            sub_values.append(value)
        if human_readable:
            return self.parse_human_readable(sub_values)
        return sub_values

    def get_range(self, init, final, human_readable=True):
        final, init = self.switch(final, init)

        sub_values = []
        if init == final:
            return self.get_single_column(init, human_readable)

        for value in self.values:
            if (value > init) or (value < final):
                continue
            sub_values.append(value)

        if human_readable:
            return self.parse_human_readable(sub_values)

        return sub_values

    def switch(self, final, init):
        if final > init:
            aux = init
            init = final
            final = aux
        return final, init

    def parse_human_readable(self, list):
        colum_names = []
        for value in list:
            if value > 0:
                colum_name = str(value) + self.positive
            else:
                colum_name = str(-value) + self.negative
            colum_names.append(colum_name)
        return colum_names


def constroi_colunas_latitude_longitude(init_lat=29, end_lat=-21,
                                        init_long=59, end_long=-17):
    lat = LatitudeColumns().get_range(init_lat, end_lat)
    long = LongitudeColumns().get_range(init_long, end_long)
    colunas_do_data_frame = []
    for linha in lat:
        for coluna in long:
            lat_long = linha + "-" + coluna
            colunas_do_data_frame.append(lat_long)
    return colunas_do_data_frame


def carrega_array_com_valores_do_arquivo_geral(
        arquivo_com_decadas_de_anomalia="funceme_db/anomalia_tsm/geral/_Dados_TSMvento_2014_04_anomt6414b04"):
    global QUANTIDADE_DE_VALORES_DO_ARQUIVO

    conteudo_do_arquivo = open(arquivo_com_decadas_de_anomalia).read()
    conteudo_do_arquivo = conteudo_do_arquivo.replace("\n", "")

    # Carrega todos os dados de anomalia em um único array
    qtd_char_no_arquivo = 5
    # Todos os valores do arquivo em um único array. Não há separação de mês. Tudo está de forma sequencial
    valores_do_arquivo = []
    for rows_index in range(QUANTIDADE_DE_VALORES_DO_ARQUIVO):
        # slice data like (n:n+5)
        value = float(conteudo_do_arquivo[
                      rows_index * qtd_char_no_arquivo: rows_index * qtd_char_no_arquivo + qtd_char_no_arquivo])
        value = float("%.3f" % value)
        value /= 10
        valores_do_arquivo.append(value)
    return valores_do_arquivo


def carrega_array_com_valores_do_arquivo_mensal(file_name):
    file_content = open(file_name).read()

    # Remove header de um único arquivo
    file_content = file_content[25:]
    file_content = file_content.replace("\n", "")

    block_size = 5
    dados_do_arquivo = []
    for rows_index in range(BLOCO_DE_DADOS_DE_UM_MES):
        # slice data like (n:n+5)
        value = float(file_content[rows_index * block_size: rows_index * block_size + block_size])
        value = float("%.3f" % value)
        value /= 10
        dados_do_arquivo.append(value)

    return dados_do_arquivo


def merge_dados_do_diretorio(diretorio_arquivo_geral,diretorio_arquivo_individual):
    global QUANTIDADE_DE_VALORES_DO_ARQUIVO
    qtde = QUANTIDADE_DE_VALORES_DO_ARQUIVO
    lista_de_arquivos_individuais = []
    arquivos_do_diretorio = os.listdir(diretorio_arquivo_individual)

    quantidade_de_arquivos = 0
    # Adiciona apenas arquivos com extensão .22
    for arquivo in arquivos_do_diretorio:
        if arquivo.endswith(".22"):
            lista_de_arquivos_individuais.append(arquivo)
            quantidade_de_arquivos += 1

    valores_dos_arquivos = carrega_array_com_valores_do_arquivo_geral(diretorio_arquivo_geral)
    # Para cada arquivo na lista é feito append na lista full_data
    for arquivo in lista_de_arquivos_individuais:
        dados_mensais = carrega_array_com_valores_do_arquivo_mensal(diretorio_arquivo_individual + arquivo)
        for item in dados_mensais:
            valores_dos_arquivos.append(item)

    ##44 meses de 05/2014 até 12/2017
    qtde += quantidade_de_arquivos * BLOCO_DE_DADOS_DE_UM_MES  # Que dá 41800

    array_de_anomalias_por_mes = []
    for i in range(0, qtde, 950):
        anomalias_do_mes = valores_dos_arquivos[i:i + BLOCO_DE_DADOS_DE_UM_MES]
        array_de_anomalias_por_mes.append(anomalias_do_mes)
    return array_de_anomalias_por_mes


def inicia_funceme_data_frame(array_de_anomalias_por_mes):
    funceme_rainy_df = pd.DataFrame()
    for anomalias_do_mes in array_de_anomalias_por_mes:
        data = np.array(anomalias_do_mes)
        row_df = pd.DataFrame(data.reshape(-1, len(data)), columns=constroi_colunas_latitude_longitude())
        funceme_rainy_df = funceme_rainy_df.append(row_df)
    funceme_rainy_df.index = range(0, len(array_de_anomalias_por_mes), 1)
    # ### Setando indices baseados na data
    FORMAT = "%Y-%m"
    some_date_time1 = "1964-01"
    data_inicial = datetime.strptime(some_date_time1, FORMAT)
    indexes_data = []
    for i in range(len(array_de_anomalias_por_mes)):
        indexes_data.append(data_inicial + relativedelta(months=i))
    funceme_rainy_df = funceme_rainy_df.set_index(pd.DatetimeIndex(data=indexes_data))
    
    return funceme_rainy_df

def plota_coluna_do_dataframe(dataframe,titulo, nome_da_coluna, save_figure=False):
    fig, axarr = plt.subplots(1)
    fig.set_size_inches(8, 5)
    ax=dataframe[nome_da_coluna].plot( color='b', linestyle='-', grid=True)
    ax.set(xlabel="Year", ylabel="Celsius/10")

    plt.title(titulo)
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=2.0)
    plt.axhline(0, color='black')
    if save_figure:
        plt.savefig("Imagens/tsm/"+titulo)
    else:        
        plt.show()
    
    plt.close()

def adiciona_media_mensal(dataframe):
    # Cria média para cada mês adicionando a coluna "media_mensal"
    media_da_figura_no_mes = []

    for date, row in dataframe.iterrows():
        media = (reduce(lambda x, y: x + y, row) / len(row))
        media_da_figura_no_mes.append(media)

    # Cria nova coluna chamada media_mensal e adiciona ao dataframe da funceme(funceme_rainy_df)
    dataframe.loc[:, "%s" % COLUNA_MEDIA_MENSAL] = pd.Series(media_da_figura_no_mes, index=dataframe.index)
    return dataframe

def calcula_climatologia_para_dataframe(dataframe):
    # Cria dicionário de médias anuais
    medias_anuais = {}
    for mes in range(1, 13, 1):
        medias_anuais[mes] = []

    for date, row in dataframe.iterrows():
        data = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S')
        medias_anuais[data.month].append(row[('%s' % COLUNA_MEDIA_MENSAL)])

    # Calcula climatologia para cada mês
    climatologias_mensais = {}
    for mes in range(1, 13, 1):
        climatologias_mensais[mes] = reduce(lambda x, y: x + y, medias_anuais[mes]) / len(medias_anuais[mes])
    return climatologias_mensais

def adiciona_anomalia(dataframe):
    climatologias_mensais = calcula_climatologia_para_dataframe(dataframe)

    ### Calculando anomalias
    anomalia = []
    for date, row in dataframe.iterrows():
        data = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S')
        anomalia_do_mes = row[COLUNA_MEDIA_MENSAL] - climatologias_mensais[data.month]
        anomalia.append(anomalia_do_mes)
    dataframe.loc[:, COLUNA_ANOMALIA_DO_MES] = pd.Series(anomalia, index=dataframe.index)
    dataframe.head(2)
    return dataframe

def adiciona_anomalia_acumulada(dataframe):
    anomalia_acumulada = []
    # Calcula anomalia acumulada
    for index in range(len(dataframe.index)):
        if index == 0:
            anomalia_acumulada.append(dataframe.iloc[index][COLUNA_ANOMALIA_DO_MES])
            continue
        anterior = anomalia_acumulada[index-1]
        atual = dataframe.iloc[index][COLUNA_ANOMALIA_DO_MES]
        anomalia_acumulada.append(float("%.3f" % (atual+anterior)))

    dataframe.loc[:, COLUNA_ANOMALIA_ACUMULADA] = pd.Series(anomalia_acumulada, index=dataframe.index)
    return dataframe

# Analisys starts from here

## Rainfall data from FUNCEME - Scraped since 1973

In [3]:
funceme_rainy_df = pd.read_csv("scraping/funceme_media_macrorregiao.csv", index_col=0 ,parse_dates=['datahora'])
media_observado_a_substituir = funceme_rainy_df.loc['1973-08-01 12:00:00']['Observado(mm)'].mean()
media_desvio_a_substituir = funceme_rainy_df.loc['1973-08-01 12:00:00']['Desvio(%)'].mean()

#Trata missing number
funceme_rainy_df['Normal(mm)'].fillna(media_observado_a_substituir, inplace=True)
funceme_rainy_df['Observado(mm)'].fillna(media_observado_a_substituir, inplace=True)
funceme_rainy_df['Desvio(%)'].fillna(media_desvio_a_substituir, inplace=True)

#Obtém média para cada coluna dos dados pluviométricos
medias = []
observado = []
desvio = []
for indices_unicos in funceme_rainy_df.index.unique():
    medias.append(funceme_rainy_df.loc[indices_unicos]['Normal(mm)'].mean())
    observado.append(funceme_rainy_df.loc[indices_unicos]['Observado(mm)'].mean())
    desvio.append(funceme_rainy_df.loc[indices_unicos]['Desvio(%)'].mean())

    #Cria novo Pandas Dataframe
funceme_rainy_montlhy_df = pd.DataFrame(index=funceme_rainy_df.index.unique().tolist())

#Adiciona dados mensais do estado ao Dataframe criado
funceme_rainy_montlhy_df = pd.concat(
    [funceme_rainy_montlhy_df, 
        pd.DataFrame(data=medias, index=funceme_rainy_montlhy_df.index, columns= ['Normal(mm)']),
        pd.DataFrame(data=observado, index=funceme_rainy_montlhy_df.index, columns= ['Observado(mm)']),
        pd.DataFrame(data=desvio, index=funceme_rainy_montlhy_df.index, columns= ['Desvio(%)'])],
    axis=1, join_axes=[funceme_rainy_montlhy_df.index])
funceme_rainy_montlhy_df.head()

##Set values to class
rainy_seasonal_months = [2,3,4] #February, March, April

indexes = []
rows = []

for index,row in funceme_rainy_montlhy_df.iterrows():
    if index.month not in rainy_seasonal_months:
        continue
    indexes.append(index)
    rows.append(row)

rainy_classified_df = pd.DataFrame(index=indexes, columns=funceme_rainy_montlhy_df.columns,data=rows)

rainy_classified_df = rainy_classified_df.groupby(rainy_classified_df.index.year).mean()

strong = 'strong'
normal = 'normal'
weak = 'weak'

classes = []

for index, row in rainy_classified_df.iterrows():
#     print(index)
    if row['Observado(mm)'] < 200:
        classes.append(weak)
        continue
    
    if row['Observado(mm)'] < 300:
        classes.append(normal)
        continue
        
    classes.append(strong)
rainy_classified_df['classes'] = classes
rainy_classified_df = rainy_classified_df['classes']
rainy_classified_df.head(5)

1973    normal
1974    strong
1975    normal
1976      weak
1977      weak
Name: classes, dtype: object

## Filter SST anomaly from 1973

In [4]:
array_de_anomalias_por_mes = merge_dados_do_diretorio("funceme_db/tsm/geral/_Dados_TSMvento_2014_04_sst6414b04","funceme_db/anomalia_tsm/individual/")
geral_df = inicia_funceme_data_frame(array_de_anomalias_por_mes)
funceme_sst_df = geral_df.loc['1973-01-01':'2017-12-01']
funceme_sst_df = funceme_sst_df.replace(9999.8, np.nan)
funceme_sst_df = funceme_sst_df.dropna(axis=1, how='any')

# Probably won`t use it
# funceme_sst_df = adiciona_media_mensal(funceme_sst_df)
# funceme_sst_df = adiciona_anomalia(funceme_sst_df)
# funceme_sst_df = adiciona_anomalia_acumulada(funceme_sst_df)

## Remove first semester from SST dataframe

In [5]:
second_semester_months=[7,8,9,10,11,12] # From August to December


In [6]:
indexes = []
rows = []

for index,row in funceme_sst_df.iterrows():
    if index.month not in second_semester_months:
        continue
    indexes.append(index)
    rows.append(row)

sst_full_area_jul_dez_df = pd.DataFrame(index=indexes, columns=funceme_sst_df.columns,data=rows)
sst_full_area_jul_dez_df.head(3)

,29N-59W,29N-57W,29N-55W,29N-53W,29N-51W,29N-49W,29N-47W,29N-45W,29N-43W,29N-41W,29N-39W,29N-37W,29N-35W,29N-33W,29N-31W,29N-29W,29N-27W,29N-25W,29N-23W,29N-21W,29N-19W,29N-17W,29N-15W,29N-13W,27N-59W,27N-57W,27N-55W,27N-53W,27N-51W,27N-49W,27N-47W,27N-45W,27N-43W,27N-41W,27N-39W,27N-37W,27N-35W,27N-33W,27N-31W,27N-29W,27N-27W,27N-25W,27N-23W,27N-21W,27N-19W,27N-17W,27N-15W,25N-59W,25N-57W,25N-55W,25N-53W,25N-51W,25N-49W,25N-47W,25N-45W,25N-43W,25N-41W,25N-39W,25N-37W,25N-35W,25N-33W,25N-31W,25N-29W,25N-27W,25N-25W,25N-23W,25N-21W,25N-19W,25N-17W,25N-15W,23N-59W,23N-57W,23N-55W,23N-53W,23N-51W,23N-49W,23N-47W,23N-45W,23N-43W,23N-41W,23N-39W,23N-37W,23N-35W,23N-33W,23N-31W,23N-29W,23N-27W,23N-25W,23N-23W,23N-21W,23N-19W,23N-17W,21N-59W,21N-57W,21N-55W,21N-53W,21N-51W,21N-49W,21N-47W,21N-45W,21N-43W,21N-41W,21N-39W,21N-37W,21N-35W,21N-33W,21N-31W,21N-29W,21N-27W,21N-25W,21N-23W,21N-21W,21N-19W,21N-17W,19N-59W,19N-57W,19N-55W,19N-53W,19N-51W,19N-49W,19N-47W,19N-45W,19N-43W,19N-41W,19N-39W,19N-37W,19N-35W,19N-33W,19N-31W,19N-29W,19N-27W,19N-25W,19N-23W,19N-21W,19N-19W,19N-17W,17N-59W,17N-57W,17N-55W,17N-53W,17N-51W,17N-49W,17N-47W,17N-45W,17N-43W,17N-41W,17N-39W,17N-37W,17N-35W,17N-33W,17N-31W,17N-29W,17N-27W,17N-25W,17N-23W,17N-21W,17N-19W,17N-17W,15N-59W,15N-57W,15N-55W,15N-53W,15N-51W,15N-49W,15N-47W,15N-45W,15N-43W,15N-41W,15N-39W,15N-37W,15N-35W,15N-33W,15N-31W,15N-29W,15N-27W,15N-25W,15N-23W,15N-21W,15N-19W,15N-17W,13N-59W,13N-57W,13N-55W,13N-53W,13N-51W,13N-49W,13N-47W,13N-45W,13N-43W,13N-41W,13N-39W,13N-37W,13N-35W,13N-33W,13N-31W,13N-29W,13N-27W,13N-25W,13N-23W,13N-21W,13N-19W,13N-17W,11N-59W,11N-57W,11N-55W,11N-53W,11N-51W,11N-49W,11N-47W,11N-45W,11N-43W,11N-41W,11N-39W,11N-37W,11N-35W,11N-33W,11N-31W,11N-29W,11N-27W,11N-25W,11N-23W,11N-21W,11N-19W,11N-17W,11N-15W,9N-59W,9N-57W,9N-55W,9N-53W,9N-51W,9N-49W,9N-47W,9N-45W,9N-43W,9N-41W,9N-39W,9N-37W,9N-35W,9N-33W,9N-31W,9N-29W,9N-27W,9N-25W,9N-23W,9N-21W,9N-19W,9N-17W,9N-15W,9N-13W,7N-57W,...,1S-19W,1S-17W,1S-15W,1S-13W,1S-11W,1S-9W,1S-7W,1S-5W,1S-3W,1S-1W,1S-1E,1S-3E,1S-5E,1S-7E,1S-9E,3S-43W,3S-41W,3S-39W,3S-37W,3S-35W,3S-33W,3S-31W,3S-29W,3S-27W,3S-25W,3S-23W,3S-21W,3S-19W,3S-17W,3S-15W,3S-13W,3S-11W,3S-9W,3S-7W,3S-5W,3S-3W,3S-1W,3S-1E,3S-3E,3S-5E,3S-7E,3S-9E,5S-37W,5S-35W,5S-33W,5S-31W,5S-29W,5S-27W,5S-25W,5S-23W,5S-21W,5S-19W,5S-17W,5S-15W,5S-13W,5S-11W,5S-9W,5S-7W,5S-5W,5S-3W,5S-1W,5S-1E,5S-3E,5S-5E,5S-7E,5S-9E,5S-11E,7S-35W,7S-33W,7S-31W,7S-29W,7S-27W,7S-25W,7S-23W,7S-21W,7S-19W,7S-17W,7S-15W,7S-13W,7S-11W,7S-9W,7S-7W,7S-5W,7S-3W,7S-1W,7S-1E,7S-3E,7S-5E,7S-7E,7S-9E,7S-11E,7S-13E,9S-35W,9S-33W,9S-31W,9S-29W,9S-27W,9S-25W,9S-23W,9S-21W,9S-19W,9S-17W,9S-15W,9S-13W,9S-11W,9S-9W,9S-7W,9S-5W,9S-3W,9S-1W,9S-1E,9S-3E,9S-5E,9S-7E,9S-9E,9S-11E,9S-13E,11S-37W,11S-35W,11S-33W,11S-31W,11S-29W,11S-27W,11S-25W,11S-23W,11S-21W,11S-19W,11S-17W,11S-15W,11S-13W,11S-11W,11S-9W,11S-7W,11S-5W,11S-3W,11S-1W,11S-1E,11S-3E,11S-5E,11S-7E,11S-9E,11S-11E,11S-13E,13S-39W,13S-37W,13S-35W,13S-33W,13S-31W,13S-29W,13S-27W,13S-25W,13S-23W,13S-21W,13S-19W,13S-17W,13S-15W,13S-13W,13S-11W,13S-9W,13S-7W,13S-5W,13S-3W,13S-1W,13S-1E,13S-3E,13S-5E,13S-7E,13S-9E,13S-11E,13S-13E,15S-39W,15S-37W,15S-35W,15S-33W,15S-31W,15S-29W,15S-27W,15S-25W,15S-23W,15S-21W,15S-19W,15S-17W,15S-15W,15S-13W,15S-11W,15S-9W,15S-7W,15S-5W,15S-3W,15S-1W,15S-1E,15S-3E,15S-5E,15S-7E,15S-9E,15S-11E,15S-13E,17S-39W,17S-37W,17S-35W,17S-33W,17S-31W,17S-29W,17S-27W,17S-25W,17S-23W,17S-21W,17S-19W,17S-17W,17S-15W,17S-13W,17S-11W,17S-9W,17S-7W,17S-5W,17S-3W,17S-1W,17S-1E,17S-3E,17S-5E,17S-7E,17S-9E,17S-11E,19S-39W,19S-37W,19S-35W,19S-33W,19S-31W,19S-29W,19S-27W,19S-25W,19S-23W,19S-21W,19S-19W,19S-17W,19S-15W,19S-13W,19S-11W,19S-9W,19S-7W,19S-5W,19S-3W,19S-1W,19S-1E,19S-3E,19S-5E,19S-7E,19S-9E,19S-11E,19S-13E
1973-07-01,26.3,26.2,26.0,25.7,25.6,25.5,25.4,25.3,25.2,25.1,24.7,24.3,24.0,23.9,23.4,23.2,22.9,22.6,22.3,22.0,21.7,21.3,21.1,20.8,26.9,26.7,26.4,26.1,25.9,25.7,25.5,25.4,25.2,24.9,24.6,24.2,23.9,23.6,23.4,23.2,23.0,22.7,22.3,22.0,21.6,2

## Creating transformed df from SST full area

In [7]:
def create_transformed_df(dataframe, column_name_prefix, initial_year = 1973, end_year=2018):
    columns = dataframe.columns 
    months = second_semester_months
    merged_colum_names=[]
    

    for col in columns:
        for month in months:
            column_name = column_name_prefix + col + '_' +str(month)
            merged_colum_names.append(column_name)
            
    dataframe_transformed_df = pd.DataFrame(index=range(1973,2018,1), columns=merged_colum_names)
    
    for year in range(initial_year, end_year, 1):
    #     print("Year " , year) # FOR DEBUG ONLY
        for month in range(7,13,1):
            for column_name in dataframe.columns:
                value = dataframe.loc[str(year)+'-'+str(month)+'-01'][column_name]
                column_name = column_name_prefix +column_name+'_'+str(month)
                dataframe_transformed_df.set_value(year, column_name, value)
    return dataframe_transformed_df

In [8]:
# columns = sst_full_area_jul_dez_df.columns 
# months = second_semester_months
# merged_colum_names=[]

# for col in columns:
#     for month in months:
#         column_name = 'sst_' + col + '_' +str(month)
#         merged_colum_names.append(column_name)

In [9]:
#For debugy only
# for column_name in merged_colum_names:
#     print(column_name)

In [10]:
# sst_transformed_df = pd.DataFrame(index=range(1973,2018,1), columns=merged_colum_names)

In [11]:
# for year in range(1973,2018,1):
#     #     print("Year " , year) # FOR DEBUG ONLY
#     for month in range(7,13,1):
#         for sst_column in sst_full_area_jul_dez_df.columns:
#             value = sst_full_area_jul_dez_df.loc[str(year)+'-'+str(month)+'-01'][sst_column]
#             column_name = 'sst_' +sst_column+'_'+str(month)
#             sst_transformed_df.set_value(year, column_name, value)
            

In [12]:
sst_transformed_df = create_transformed_df(sst_full_area_jul_dez_df, "sst_")

In [13]:
sst_transformed_df.head(3)

,sst_29N-59W_7,sst_29N-59W_8,sst_29N-59W_9,sst_29N-59W_10,sst_29N-59W_11,sst_29N-59W_12,sst_29N-57W_7,sst_29N-57W_8,sst_29N-57W_9,sst_29N-57W_10,sst_29N-57W_11,sst_29N-57W_12,sst_29N-55W_7,sst_29N-55W_8,sst_29N-55W_9,sst_29N-55W_10,sst_29N-55W_11,sst_29N-55W_12,sst_29N-53W_7,sst_29N-53W_8,sst_29N-53W_9,sst_29N-53W_10,sst_29N-53W_11,sst_29N-53W_12,sst_29N-51W_7,sst_29N-51W_8,sst_29N-51W_9,sst_29N-51W_10,sst_29N-51W_11,sst_29N-51W_12,sst_29N-49W_7,sst_29N-49W_8,sst_29N-49W_9,sst_29N-49W_10,sst_29N-49W_11,sst_29N-49W_12,sst_29N-47W_7,sst_29N-47W_8,sst_29N-47W_9,sst_29N-47W_10,sst_29N-47W_11,sst_29N-47W_12,sst_29N-45W_7,sst_29N-45W_8,sst_29N-45W_9,sst_29N-45W_10,sst_29N-45W_11,sst_29N-45W_12,sst_29N-43W_7,sst_29N-43W_8,sst_29N-43W_9,sst_29N-43W_10,sst_29N-43W_11,sst_29N-43W_12,sst_29N-41W_7,sst_29N-41W_8,sst_29N-41W_9,sst_29N-41W_10,sst_29N-41W_11,sst_29N-41W_12,sst_29N-39W_7,sst_29N-39W_8,sst_29N-39W_9,sst_29N-39W_10,sst_29N-39W_11,sst_29N-39W_12,sst_29N-37W_7,sst_29N-37W_8,sst_29N-37W_9,sst_29N-37W_10,sst_29N-37W_11,sst_29N-37W_12,sst_29N-35W_7,sst_29N-35W_8,sst_29N-35W_9,sst_29N-35W_10,sst_29N-35W_11,sst_29N-35W_12,sst_29N-33W_7,sst_29N-33W_8,sst_29N-33W_9,sst_29N-33W_10,sst_29N-33W_11,sst_29N-33W_12,sst_29N-31W_7,sst_29N-31W_8,sst_29N-31W_9,sst_29N-31W_10,sst_29N-31W_11,sst_29N-31W_12,sst_29N-29W_7,sst_29N-29W_8,sst_29N-29W_9,sst_29N-29W_10,sst_29N-29W_11,sst_29N-29W_12,sst_29N-27W_7,sst_29N-27W_8,sst_29N-27W_9,sst_29N-27W_10,sst_29N-27W_11,sst_29N-27W_12,sst_29N-25W_7,sst_29N-25W_8,sst_29N-25W_9,sst_29N-25W_10,sst_29N-25W_11,sst_29N-25W_12,sst_29N-23W_7,sst_29N-23W_8,sst_29N-23W_9,sst_29N-23W_10,sst_29N-23W_11,sst_29N-23W_12,sst_29N-21W_7,sst_29N-21W_8,sst_29N-21W_9,sst_29N-21W_10,sst_29N-21W_11,sst_29N-21W_12,sst_29N-19W_7,sst_29N-19W_8,sst_29N-19W_9,sst_29N-19W_10,sst_29N-19W_11,sst_29N-19W_12,sst_29N-17W_7,sst_29N-17W_8,sst_29N-17W_9,sst_29N-17W_10,sst_29N-17W_11,sst_29N-17W_12,sst_29N-15W_7,sst_29N-15W_8,sst_29N-15W_9,sst_29N-15W_10,sst_29N-15W_11,sst_29N-15W_12,sst_29N-13W_7,sst_29N-13W_8,sst_29N-13W_9,sst_29N-13W_10,sst_29N-13W_11,sst_29N-13W_12,sst_27N-59W_7,sst_27N-59W_8,sst_27N-59W_9,sst_27N-59W_10,sst_27N-59W_11,sst_27N-59W_12,sst_27N-57W_7,sst_27N-57W_8,sst_27N-57W_9,sst_27N-57W_10,sst_27N-57W_11,sst_27N-57W_12,sst_27N-55W_7,sst_27N-55W_8,sst_27N-55W_9,sst_27N-55W_10,sst_27N-55W_11,sst_27N-55W_12,sst_27N-53W_7,sst_27N-53W_8,sst_27N-53W_9,sst_27N-53W_10,sst_27N-53W_11,sst_27N-53W_12,sst_27N-51W_7,sst_27N-51W_8,sst_27N-51W_9,sst_27N-51W_10,sst_27N-51W_11,sst_27N-51W_12,sst_27N-49W_7,sst_27N-49W_8,sst_27N-49W_9,sst_27N-49W_10,sst_27N-49W_11,sst_27N-49W_12,sst_27N-47W_7,sst_27N-47W_8,sst_27N-47W_9,sst_27N-47W_10,sst_27N-47W_11,sst_27N-47W_12,sst_27N-45W_7,sst_27N-45W_8,sst_27N-45W_9,sst_27N-45W_10,sst_27N-45W_11,sst_27N-45W_12,sst_27N-43W_7,sst_27N-43W_8,sst_27N-43W_9,sst_27N-43W_10,sst_27N-43W_11,sst_27N-43W_12,sst_27N-41W_7,sst_27N-41W_8,sst_27N-41W_9,sst_27N-41W_10,sst_27N-41W_11,sst_27N-41W_12,sst_27N-39W_7,sst_27N-39W_8,sst_27N-39W_9,sst_27N-39W_10,sst_27N-39W_11,sst_27N-39W_12,sst_27N-37W_7,sst_27N-37W_8,sst_27N-37W_9,sst_27N-37W_10,sst_27N-37W_11,sst_27N-37W_12,sst_27N-35W_7,sst_27N-35W_8,sst_27N-35W_9,sst_27N-35W_10,sst_27N-35W_11,sst_27N-35W_12,sst_27N-33W_7,sst_27N-33W_8,sst_27N-33W_9,sst_27N-33W_10,sst_27N-33W_11,sst_27N-33W_12,sst_27N-31W_7,sst_27N-31W_8,sst_27N-31W_9,sst_27N-31W_10,sst_27N-31W_11,sst_27N-31W_12,sst_27N-29W_7,sst_27N-29W_8,sst_27N-29W_9,sst_27N-29W_10,sst_27N-29W_11,sst_27N-29W_12,sst_27N-27W_7,sst_27N-27W_8,sst_27N-27W_9,sst_27N-27W_10,sst_27N-27W_11,sst_27N-27W_12,sst_27N-25W_7,sst_27N-25W_8,sst_27N-25W_9,sst_27N-25W_10,...,sst_17S-17W_9,sst_17S-17W_10,sst_17S-17W_11,sst_17S-17W_12,sst_17S-15W_7,sst_17S-15W_8,sst_17S-15W_9,sst_17S-15W_10,sst_17S-15W_11,sst_17S-15W_12,sst_17S-13W_7,sst_17S-13W_8,sst_17S-13W_9,sst_17S-13W_10,sst_17S-13W_11,sst_17S-13W_12,sst_17S-11W_7,sst_17S-11W_8,sst_17S-11W_9,sst_17S-11W_10,sst_17S-11W_11,sst_17S-11W_12,sst_17S-9W_7,sst_17S-9W_8,sst_17S-9W_9,sst_17S-9W_1

In [14]:
sst_transformed_df.head(3)

,sst_29N-59W_7,sst_29N-59W_8,sst_29N-59W_9,sst_29N-59W_10,sst_29N-59W_11,sst_29N-59W_12,sst_29N-57W_7,sst_29N-57W_8,sst_29N-57W_9,sst_29N-57W_10,sst_29N-57W_11,sst_29N-57W_12,sst_29N-55W_7,sst_29N-55W_8,sst_29N-55W_9,sst_29N-55W_10,sst_29N-55W_11,sst_29N-55W_12,sst_29N-53W_7,sst_29N-53W_8,sst_29N-53W_9,sst_29N-53W_10,sst_29N-53W_11,sst_29N-53W_12,sst_29N-51W_7,sst_29N-51W_8,sst_29N-51W_9,sst_29N-51W_10,sst_29N-51W_11,sst_29N-51W_12,sst_29N-49W_7,sst_29N-49W_8,sst_29N-49W_9,sst_29N-49W_10,sst_29N-49W_11,sst_29N-49W_12,sst_29N-47W_7,sst_29N-47W_8,sst_29N-47W_9,sst_29N-47W_10,sst_29N-47W_11,sst_29N-47W_12,sst_29N-45W_7,sst_29N-45W_8,sst_29N-45W_9,sst_29N-45W_10,sst_29N-45W_11,sst_29N-45W_12,sst_29N-43W_7,sst_29N-43W_8,sst_29N-43W_9,sst_29N-43W_10,sst_29N-43W_11,sst_29N-43W_12,sst_29N-41W_7,sst_29N-41W_8,sst_29N-41W_9,sst_29N-41W_10,sst_29N-41W_11,sst_29N-41W_12,sst_29N-39W_7,sst_29N-39W_8,sst_29N-39W_9,sst_29N-39W_10,sst_29N-39W_11,sst_29N-39W_12,sst_29N-37W_7,sst_29N-37W_8,sst_29N-37W_9,sst_29N-37W_10,sst_29N-37W_11,sst_29N-37W_12,sst_29N-35W_7,sst_29N-35W_8,sst_29N-35W_9,sst_29N-35W_10,sst_29N-35W_11,sst_29N-35W_12,sst_29N-33W_7,sst_29N-33W_8,sst_29N-33W_9,sst_29N-33W_10,sst_29N-33W_11,sst_29N-33W_12,sst_29N-31W_7,sst_29N-31W_8,sst_29N-31W_9,sst_29N-31W_10,sst_29N-31W_11,sst_29N-31W_12,sst_29N-29W_7,sst_29N-29W_8,sst_29N-29W_9,sst_29N-29W_10,sst_29N-29W_11,sst_29N-29W_12,sst_29N-27W_7,sst_29N-27W_8,sst_29N-27W_9,sst_29N-27W_10,sst_29N-27W_11,sst_29N-27W_12,sst_29N-25W_7,sst_29N-25W_8,sst_29N-25W_9,sst_29N-25W_10,sst_29N-25W_11,sst_29N-25W_12,sst_29N-23W_7,sst_29N-23W_8,sst_29N-23W_9,sst_29N-23W_10,sst_29N-23W_11,sst_29N-23W_12,sst_29N-21W_7,sst_29N-21W_8,sst_29N-21W_9,sst_29N-21W_10,sst_29N-21W_11,sst_29N-21W_12,sst_29N-19W_7,sst_29N-19W_8,sst_29N-19W_9,sst_29N-19W_10,sst_29N-19W_11,sst_29N-19W_12,sst_29N-17W_7,sst_29N-17W_8,sst_29N-17W_9,sst_29N-17W_10,sst_29N-17W_11,sst_29N-17W_12,sst_29N-15W_7,sst_29N-15W_8,sst_29N-15W_9,sst_29N-15W_10,sst_29N-15W_11,sst_29N-15W_12,sst_29N-13W_7,sst_29N-13W_8,sst_29N-13W_9,sst_29N-13W_10,sst_29N-13W_11,sst_29N-13W_12,sst_27N-59W_7,sst_27N-59W_8,sst_27N-59W_9,sst_27N-59W_10,sst_27N-59W_11,sst_27N-59W_12,sst_27N-57W_7,sst_27N-57W_8,sst_27N-57W_9,sst_27N-57W_10,sst_27N-57W_11,sst_27N-57W_12,sst_27N-55W_7,sst_27N-55W_8,sst_27N-55W_9,sst_27N-55W_10,sst_27N-55W_11,sst_27N-55W_12,sst_27N-53W_7,sst_27N-53W_8,sst_27N-53W_9,sst_27N-53W_10,sst_27N-53W_11,sst_27N-53W_12,sst_27N-51W_7,sst_27N-51W_8,sst_27N-51W_9,sst_27N-51W_10,sst_27N-51W_11,sst_27N-51W_12,sst_27N-49W_7,sst_27N-49W_8,sst_27N-49W_9,sst_27N-49W_10,sst_27N-49W_11,sst_27N-49W_12,sst_27N-47W_7,sst_27N-47W_8,sst_27N-47W_9,sst_27N-47W_10,sst_27N-47W_11,sst_27N-47W_12,sst_27N-45W_7,sst_27N-45W_8,sst_27N-45W_9,sst_27N-45W_10,sst_27N-45W_11,sst_27N-45W_12,sst_27N-43W_7,sst_27N-43W_8,sst_27N-43W_9,sst_27N-43W_10,sst_27N-43W_11,sst_27N-43W_12,sst_27N-41W_7,sst_27N-41W_8,sst_27N-41W_9,sst_27N-41W_10,sst_27N-41W_11,sst_27N-41W_12,sst_27N-39W_7,sst_27N-39W_8,sst_27N-39W_9,sst_27N-39W_10,sst_27N-39W_11,sst_27N-39W_12,sst_27N-37W_7,sst_27N-37W_8,sst_27N-37W_9,sst_27N-37W_10,sst_27N-37W_11,sst_27N-37W_12,sst_27N-35W_7,sst_27N-35W_8,sst_27N-35W_9,sst_27N-35W_10,sst_27N-35W_11,sst_27N-35W_12,sst_27N-33W_7,sst_27N-33W_8,sst_27N-33W_9,sst_27N-33W_10,sst_27N-33W_11,sst_27N-33W_12,sst_27N-31W_7,sst_27N-31W_8,sst_27N-31W_9,sst_27N-31W_10,sst_27N-31W_11,sst_27N-31W_12,sst_27N-29W_7,sst_27N-29W_8,sst_27N-29W_9,sst_27N-29W_10,sst_27N-29W_11,sst_27N-29W_12,sst_27N-27W_7,sst_27N-27W_8,sst_27N-27W_9,sst_27N-27W_10,sst_27N-27W_11,sst_27N-27W_12,sst_27N-25W_7,sst_27N-25W_8,sst_27N-25W_9,sst_27N-25W_10,...,sst_17S-17W_9,sst_17S-17W_10,sst_17S-17W_11,sst_17S-17W_12,sst_17S-15W_7,sst_17S-15W_8,sst_17S-15W_9,sst_17S-15W_10,sst_17S-15W_11,sst_17S-15W_12,sst_17S-13W_7,sst_17S-13W_8,sst_17S-13W_9,sst_17S-13W_10,sst_17S-13W_11,sst_17S-13W_12,sst_17S-11W_7,sst_17S-11W_8,sst_17S-11W_9,sst_17S-11W_10,sst_17S-11W_11,sst_17S-11W_12,sst_17S-9W_7,sst_17S-9W_8,sst_17S-9W_9,sst_17S-9W_1

## PSW

In [15]:
array_dados_brutos_x = merge_dados_do_diretorio("funceme_db/pseudo_tensao_x_dados_brutos/geral/_Dados_TSMvento_2014_04_pwsx6414b04","funceme_db/pseudo_tensao_x_dados_brutos/individual/")
array_dados_brutos_y = merge_dados_do_diretorio("funceme_db/pseudo_tensao_y_dados_brutos/geral/_Dados_TSMvento_2014_04_pwsy6414b04","funceme_db/pseudo_tensao_y_dados_brutos/individual/")

In [16]:
#For debug only
#array_dados_brutos_x[0]

In [17]:
#For debug only
# array_dados_brutos_y[0]

In [18]:
pws = []
for month_index in range(len(array_dados_brutos_x)):
    scalar_product_list = []
    for value_index in range(len(array_dados_brutos_x[month_index])):
        scalar_product = math.sqrt(pow(array_dados_brutos_x[month_index][value_index],2) + pow(array_dados_brutos_y[month_index][value_index],2))
        if scalar_product == 14141.852781018475:
            scalar_product_list.append(np.nan)
        else:
            scalar_product_list.append(scalar_product)
    pws.append(scalar_product_list)

In [19]:
#For debug only
# pws[0][0]

In [20]:
#For debug only
# pws[0][1]

In [21]:
pws_full_area_df = inicia_funceme_data_frame(pws)
pws_full_area_df.head(3)

,29N-59W,29N-57W,29N-55W,29N-53W,29N-51W,29N-49W,29N-47W,29N-45W,29N-43W,29N-41W,29N-39W,29N-37W,29N-35W,29N-33W,29N-31W,29N-29W,29N-27W,29N-25W,29N-23W,29N-21W,29N-19W,29N-17W,29N-15W,29N-13W,29N-11W,29N-9W,29N-7W,29N-5W,29N-3W,29N-1W,29N-1E,29N-3E,29N-5E,29N-7E,29N-9E,29N-11E,29N-13E,29N-15E,27N-59W,27N-57W,27N-55W,27N-53W,27N-51W,27N-49W,27N-47W,27N-45W,27N-43W,27N-41W,27N-39W,27N-37W,27N-35W,27N-33W,27N-31W,27N-29W,27N-27W,27N-25W,27N-23W,27N-21W,27N-19W,27N-17W,27N-15W,27N-13W,27N-11W,27N-9W,27N-7W,27N-5W,27N-3W,27N-1W,27N-1E,27N-3E,27N-5E,27N-7E,27N-9E,27N-11E,27N-13E,27N-15E,25N-59W,25N-57W,25N-55W,25N-53W,25N-51W,25N-49W,25N-47W,25N-45W,25N-43W,25N-41W,25N-39W,25N-37W,25N-35W,25N-33W,25N-31W,25N-29W,25N-27W,25N-25W,25N-23W,25N-21W,25N-19W,25N-17W,25N-15W,25N-13W,25N-11W,25N-9W,25N-7W,25N-5W,25N-3W,25N-1W,25N-1E,25N-3E,25N-5E,25N-7E,25N-9E,25N-11E,25N-13E,25N-15E,23N-59W,23N-57W,23N-55W,23N-53W,23N-51W,23N-49W,23N-47W,23N-45W,23N-43W,23N-41W,23N-39W,23N-37W,23N-35W,23N-33W,23N-31W,23N-29W,23N-27W,23N-25W,23N-23W,23N-21W,23N-19W,23N-17W,23N-15W,23N-13W,23N-11W,23N-9W,23N-7W,23N-5W,23N-3W,23N-1W,23N-1E,23N-3E,23N-5E,23N-7E,23N-9E,23N-11E,23N-13E,23N-15E,21N-59W,21N-57W,21N-55W,21N-53W,21N-51W,21N-49W,21N-47W,21N-45W,21N-43W,21N-41W,21N-39W,21N-37W,21N-35W,21N-33W,21N-31W,21N-29W,21N-27W,21N-25W,21N-23W,21N-21W,21N-19W,21N-17W,21N-15W,21N-13W,21N-11W,21N-9W,21N-7W,21N-5W,21N-3W,21N-1W,21N-1E,21N-3E,21N-5E,21N-7E,21N-9E,21N-11E,21N-13E,21N-15E,19N-59W,19N-57W,19N-55W,19N-53W,19N-51W,19N-49W,19N-47W,19N-45W,19N-43W,19N-41W,19N-39W,19N-37W,19N-35W,19N-33W,19N-31W,19N-29W,19N-27W,19N-25W,19N-23W,19N-21W,19N-19W,19N-17W,19N-15W,19N-13W,19N-11W,19N-9W,19N-7W,19N-5W,19N-3W,19N-1W,19N-1E,19N-3E,19N-5E,19N-7E,19N-9E,19N-11E,19N-13E,19N-15E,17N-59W,17N-57W,17N-55W,17N-53W,17N-51W,17N-49W,17N-47W,17N-45W,17N-43W,17N-41W,17N-39W,17N-37W,17N-35W,17N-33W,17N-31W,17N-29W,17N-27W,17N-25W,17N-23W,17N-21W,17N-19W,17N-17W,...,7S-27W,7S-25W,7S-23W,7S-21W,7S-19W,7S-17W,7S-15W,7S-13W,7S-11W,7S-9W,7S-7W,7S-5W,7S-3W,7S-1W,7S-1E,7S-3E,7S-5E,7S-7E,7S-9E,7S-11E,7S-13E,7S-15E,9S-59W,9S-57W,9S-55W,9S-53W,9S-51W,9S-49W,9S-47W,9S-45W,9S-43W,9S-41W,9S-39W,9S-37W,9S-35W,9S-33W,9S-31W,9S-29W,9S-27W,9S-25W,9S-23W,9S-21W,9S-19W,9S-17W,9S-15W,9S-13W,9S-11W,9S-9W,9S-7W,9S-5W,9S-3W,9S-1W,9S-1E,9S-3E,9S-5E,9S-7E,9S-9E,9S-11E,9S-13E,9S-15E,11S-59W,11S-57W,11S-55W,11S-53W,11S-51W,11S-49W,11S-47W,11S-45W,11S-43W,11S-41W,11S-39W,11S-37W,11S-35W,11S-33W,11S-31W,11S-29W,11S-27W,11S-25W,11S-23W,11S-21W,11S-19W,11S-17W,11S-15W,11S-13W,11S-11W,11S-9W,11S-7W,11S-5W,11S-3W,11S-1W,11S-1E,11S-3E,11S-5E,11S-7E,11S-9E,11S-11E,11S-13E,11S-15E,13S-59W,13S-57W,13S-55W,13S-53W,13S-51W,13S-49W,13S-47W,13S-45W,13S-43W,13S-41W,13S-39W,13S-37W,13S-35W,13S-33W,13S-31W,13S-29W,13S-27W,13S-25W,13S-23W,13S-21W,13S-19W,13S-17W,13S-15W,13S-13W,13S-11W,13S-9W,13S-7W,13S-5W,13S-3W,13S-1W,13S-1E,13S-3E,13S-5E,13S-7E,13S-9E,13S-11E,13S-13E,13S-15E,15S-59W,15S-57W,15S-55W,15S-53W,15S-51W,15S-49W,15S-47W,15S-45W,15S-43W,15S-41W,15S-39W,15S-37W,15S-35W,15S-33W,15S-31W,15S-29W,15S-27W,15S-25W,15S-23W,15S-21W,15S-19W,15S-17W,15S-15W,15S-13W,15S-11W,15S-9W,15S-7W,15S-5W,15S-3W,15S-1W,15S-1E,15S-3E,15S-5E,15S-7E,15S-9E,15S-11E,15S-13E,15S-15E,17S-59W,17S-57W,17S-55W,17S-53W,17S-51W,17S-49W,17S-47W,17S-45W,17S-43W,17S-41W,17S-39W,17S-37W,17S-35W,17S-33W,17S-31W,17S-29W,17S-27W,17S-25W,17S-23W,17S-21W,17S-19W,17S-17W,17S-15W,17S-13W,17S-11W,17S-9W,17S-7W,17S-5W,17S-3W,17S-1W,17S-1E,17S-3E,17S-5E,17S-7E,17S-9E,17S-11E,17S-13E,17S-15E,19S-59W,19S-57W,19S-55W,19S-53W,19S-51W,19S-49W,19S-47W,19S-45W,19S-43W,19S-41W,19S-39W,19S-37W,19S-35W,19S-33W,19S-31W,19S-29W,19S-27W,19S-25W,19S-23W,19S-21W,19S-19W,19S-17W,19S-15W,19S-13W,19S-11W,19S-9W,19S-7W,19S-5W,19S-3W,19S-1W,19S-1E,19S-3E,19S-5E,19S-7E,19S-9E,19S-11E,19S-13E,19S-15E
1964-01-01,0.632456,10.855874,16.995588,20.727035,22.036787,20.506097,15.060212,11.806778,13.165485,15.928905,18.720043,19.382982,19.275373,19.183847,18.293715,17.761194,18.120155,17.60

In [22]:
## Remove first semester from PWS dataframe

In [23]:
indexes = []
rows = []

for index,row in pws_full_area_df.iterrows():
    if index.month not in second_semester_months:
        continue
    indexes.append(index)
    rows.append(row)

pws_full_area_jul_dez_df = pd.DataFrame(index=indexes, columns=pws_full_area_df.columns,data=rows)


# TESTIN THIS CASE

In [24]:
pws_full_area_jul_dez_df.head(3)

,29N-59W,29N-57W,29N-55W,29N-53W,29N-51W,29N-49W,29N-47W,29N-45W,29N-43W,29N-41W,29N-39W,29N-37W,29N-35W,29N-33W,29N-31W,29N-29W,29N-27W,29N-25W,29N-23W,29N-21W,29N-19W,29N-17W,29N-15W,29N-13W,29N-11W,29N-9W,29N-7W,29N-5W,29N-3W,29N-1W,29N-1E,29N-3E,29N-5E,29N-7E,29N-9E,29N-11E,29N-13E,29N-15E,27N-59W,27N-57W,27N-55W,27N-53W,27N-51W,27N-49W,27N-47W,27N-45W,27N-43W,27N-41W,27N-39W,27N-37W,27N-35W,27N-33W,27N-31W,27N-29W,27N-27W,27N-25W,27N-23W,27N-21W,27N-19W,27N-17W,27N-15W,27N-13W,27N-11W,27N-9W,27N-7W,27N-5W,27N-3W,27N-1W,27N-1E,27N-3E,27N-5E,27N-7E,27N-9E,27N-11E,27N-13E,27N-15E,25N-59W,25N-57W,25N-55W,25N-53W,25N-51W,25N-49W,25N-47W,25N-45W,25N-43W,25N-41W,25N-39W,25N-37W,25N-35W,25N-33W,25N-31W,25N-29W,25N-27W,25N-25W,25N-23W,25N-21W,25N-19W,25N-17W,25N-15W,25N-13W,25N-11W,25N-9W,25N-7W,25N-5W,25N-3W,25N-1W,25N-1E,25N-3E,25N-5E,25N-7E,25N-9E,25N-11E,25N-13E,25N-15E,23N-59W,23N-57W,23N-55W,23N-53W,23N-51W,23N-49W,23N-47W,23N-45W,23N-43W,23N-41W,23N-39W,23N-37W,23N-35W,23N-33W,23N-31W,23N-29W,23N-27W,23N-25W,23N-23W,23N-21W,23N-19W,23N-17W,23N-15W,23N-13W,23N-11W,23N-9W,23N-7W,23N-5W,23N-3W,23N-1W,23N-1E,23N-3E,23N-5E,23N-7E,23N-9E,23N-11E,23N-13E,23N-15E,21N-59W,21N-57W,21N-55W,21N-53W,21N-51W,21N-49W,21N-47W,21N-45W,21N-43W,21N-41W,21N-39W,21N-37W,21N-35W,21N-33W,21N-31W,21N-29W,21N-27W,21N-25W,21N-23W,21N-21W,21N-19W,21N-17W,21N-15W,21N-13W,21N-11W,21N-9W,21N-7W,21N-5W,21N-3W,21N-1W,21N-1E,21N-3E,21N-5E,21N-7E,21N-9E,21N-11E,21N-13E,21N-15E,19N-59W,19N-57W,19N-55W,19N-53W,19N-51W,19N-49W,19N-47W,19N-45W,19N-43W,19N-41W,19N-39W,19N-37W,19N-35W,19N-33W,19N-31W,19N-29W,19N-27W,19N-25W,19N-23W,19N-21W,19N-19W,19N-17W,19N-15W,19N-13W,19N-11W,19N-9W,19N-7W,19N-5W,19N-3W,19N-1W,19N-1E,19N-3E,19N-5E,19N-7E,19N-9E,19N-11E,19N-13E,19N-15E,17N-59W,17N-57W,17N-55W,17N-53W,17N-51W,17N-49W,17N-47W,17N-45W,17N-43W,17N-41W,17N-39W,17N-37W,17N-35W,17N-33W,17N-31W,17N-29W,17N-27W,17N-25W,17N-23W,17N-21W,17N-19W,17N-17W,...,7S-27W,7S-25W,7S-23W,7S-21W,7S-19W,7S-17W,7S-15W,7S-13W,7S-11W,7S-9W,7S-7W,7S-5W,7S-3W,7S-1W,7S-1E,7S-3E,7S-5E,7S-7E,7S-9E,7S-11E,7S-13E,7S-15E,9S-59W,9S-57W,9S-55W,9S-53W,9S-51W,9S-49W,9S-47W,9S-45W,9S-43W,9S-41W,9S-39W,9S-37W,9S-35W,9S-33W,9S-31W,9S-29W,9S-27W,9S-25W,9S-23W,9S-21W,9S-19W,9S-17W,9S-15W,9S-13W,9S-11W,9S-9W,9S-7W,9S-5W,9S-3W,9S-1W,9S-1E,9S-3E,9S-5E,9S-7E,9S-9E,9S-11E,9S-13E,9S-15E,11S-59W,11S-57W,11S-55W,11S-53W,11S-51W,11S-49W,11S-47W,11S-45W,11S-43W,11S-41W,11S-39W,11S-37W,11S-35W,11S-33W,11S-31W,11S-29W,11S-27W,11S-25W,11S-23W,11S-21W,11S-19W,11S-17W,11S-15W,11S-13W,11S-11W,11S-9W,11S-7W,11S-5W,11S-3W,11S-1W,11S-1E,11S-3E,11S-5E,11S-7E,11S-9E,11S-11E,11S-13E,11S-15E,13S-59W,13S-57W,13S-55W,13S-53W,13S-51W,13S-49W,13S-47W,13S-45W,13S-43W,13S-41W,13S-39W,13S-37W,13S-35W,13S-33W,13S-31W,13S-29W,13S-27W,13S-25W,13S-23W,13S-21W,13S-19W,13S-17W,13S-15W,13S-13W,13S-11W,13S-9W,13S-7W,13S-5W,13S-3W,13S-1W,13S-1E,13S-3E,13S-5E,13S-7E,13S-9E,13S-11E,13S-13E,13S-15E,15S-59W,15S-57W,15S-55W,15S-53W,15S-51W,15S-49W,15S-47W,15S-45W,15S-43W,15S-41W,15S-39W,15S-37W,15S-35W,15S-33W,15S-31W,15S-29W,15S-27W,15S-25W,15S-23W,15S-21W,15S-19W,15S-17W,15S-15W,15S-13W,15S-11W,15S-9W,15S-7W,15S-5W,15S-3W,15S-1W,15S-1E,15S-3E,15S-5E,15S-7E,15S-9E,15S-11E,15S-13E,15S-15E,17S-59W,17S-57W,17S-55W,17S-53W,17S-51W,17S-49W,17S-47W,17S-45W,17S-43W,17S-41W,17S-39W,17S-37W,17S-35W,17S-33W,17S-31W,17S-29W,17S-27W,17S-25W,17S-23W,17S-21W,17S-19W,17S-17W,17S-15W,17S-13W,17S-11W,17S-9W,17S-7W,17S-5W,17S-3W,17S-1W,17S-1E,17S-3E,17S-5E,17S-7E,17S-9E,17S-11E,17S-13E,17S-15E,19S-59W,19S-57W,19S-55W,19S-53W,19S-51W,19S-49W,19S-47W,19S-45W,19S-43W,19S-41W,19S-39W,19S-37W,19S-35W,19S-33W,19S-31W,19S-29W,19S-27W,19S-25W,19S-23W,19S-21W,19S-19W,19S-17W,19S-15W,19S-13W,19S-11W,19S-9W,19S-7W,19S-5W,19S-3W,19S-1W,19S-1E,19S-3E,19S-5E,19S-7E,19S-9E,19S-11E,19S-13E,19S-15E
1964-07-01,18.346934,16.812198,16.357261,15.190787,14.748898,14.500000,12.800391,13.613229,14.334922,17.593465,22.378785,28.579188,35.243297,44.715210,51.315690,58.789965,57.622305,55.8

### Treating missing data

In [25]:
pws_full_area_jul_dez_df = pws_full_area_jul_dez_df.replace(14141.852781018475, np.nan)
pws_full_area_jul_dez_df.head(3)

,29N-59W,29N-57W,29N-55W,29N-53W,29N-51W,29N-49W,29N-47W,29N-45W,29N-43W,29N-41W,29N-39W,29N-37W,29N-35W,29N-33W,29N-31W,29N-29W,29N-27W,29N-25W,29N-23W,29N-21W,29N-19W,29N-17W,29N-15W,29N-13W,29N-11W,29N-9W,29N-7W,29N-5W,29N-3W,29N-1W,29N-1E,29N-3E,29N-5E,29N-7E,29N-9E,29N-11E,29N-13E,29N-15E,27N-59W,27N-57W,27N-55W,27N-53W,27N-51W,27N-49W,27N-47W,27N-45W,27N-43W,27N-41W,27N-39W,27N-37W,27N-35W,27N-33W,27N-31W,27N-29W,27N-27W,27N-25W,27N-23W,27N-21W,27N-19W,27N-17W,27N-15W,27N-13W,27N-11W,27N-9W,27N-7W,27N-5W,27N-3W,27N-1W,27N-1E,27N-3E,27N-5E,27N-7E,27N-9E,27N-11E,27N-13E,27N-15E,25N-59W,25N-57W,25N-55W,25N-53W,25N-51W,25N-49W,25N-47W,25N-45W,25N-43W,25N-41W,25N-39W,25N-37W,25N-35W,25N-33W,25N-31W,25N-29W,25N-27W,25N-25W,25N-23W,25N-21W,25N-19W,25N-17W,25N-15W,25N-13W,25N-11W,25N-9W,25N-7W,25N-5W,25N-3W,25N-1W,25N-1E,25N-3E,25N-5E,25N-7E,25N-9E,25N-11E,25N-13E,25N-15E,23N-59W,23N-57W,23N-55W,23N-53W,23N-51W,23N-49W,23N-47W,23N-45W,23N-43W,23N-41W,23N-39W,23N-37W,23N-35W,23N-33W,23N-31W,23N-29W,23N-27W,23N-25W,23N-23W,23N-21W,23N-19W,23N-17W,23N-15W,23N-13W,23N-11W,23N-9W,23N-7W,23N-5W,23N-3W,23N-1W,23N-1E,23N-3E,23N-5E,23N-7E,23N-9E,23N-11E,23N-13E,23N-15E,21N-59W,21N-57W,21N-55W,21N-53W,21N-51W,21N-49W,21N-47W,21N-45W,21N-43W,21N-41W,21N-39W,21N-37W,21N-35W,21N-33W,21N-31W,21N-29W,21N-27W,21N-25W,21N-23W,21N-21W,21N-19W,21N-17W,21N-15W,21N-13W,21N-11W,21N-9W,21N-7W,21N-5W,21N-3W,21N-1W,21N-1E,21N-3E,21N-5E,21N-7E,21N-9E,21N-11E,21N-13E,21N-15E,19N-59W,19N-57W,19N-55W,19N-53W,19N-51W,19N-49W,19N-47W,19N-45W,19N-43W,19N-41W,19N-39W,19N-37W,19N-35W,19N-33W,19N-31W,19N-29W,19N-27W,19N-25W,19N-23W,19N-21W,19N-19W,19N-17W,19N-15W,19N-13W,19N-11W,19N-9W,19N-7W,19N-5W,19N-3W,19N-1W,19N-1E,19N-3E,19N-5E,19N-7E,19N-9E,19N-11E,19N-13E,19N-15E,17N-59W,17N-57W,17N-55W,17N-53W,17N-51W,17N-49W,17N-47W,17N-45W,17N-43W,17N-41W,17N-39W,17N-37W,17N-35W,17N-33W,17N-31W,17N-29W,17N-27W,17N-25W,17N-23W,17N-21W,17N-19W,17N-17W,...,7S-27W,7S-25W,7S-23W,7S-21W,7S-19W,7S-17W,7S-15W,7S-13W,7S-11W,7S-9W,7S-7W,7S-5W,7S-3W,7S-1W,7S-1E,7S-3E,7S-5E,7S-7E,7S-9E,7S-11E,7S-13E,7S-15E,9S-59W,9S-57W,9S-55W,9S-53W,9S-51W,9S-49W,9S-47W,9S-45W,9S-43W,9S-41W,9S-39W,9S-37W,9S-35W,9S-33W,9S-31W,9S-29W,9S-27W,9S-25W,9S-23W,9S-21W,9S-19W,9S-17W,9S-15W,9S-13W,9S-11W,9S-9W,9S-7W,9S-5W,9S-3W,9S-1W,9S-1E,9S-3E,9S-5E,9S-7E,9S-9E,9S-11E,9S-13E,9S-15E,11S-59W,11S-57W,11S-55W,11S-53W,11S-51W,11S-49W,11S-47W,11S-45W,11S-43W,11S-41W,11S-39W,11S-37W,11S-35W,11S-33W,11S-31W,11S-29W,11S-27W,11S-25W,11S-23W,11S-21W,11S-19W,11S-17W,11S-15W,11S-13W,11S-11W,11S-9W,11S-7W,11S-5W,11S-3W,11S-1W,11S-1E,11S-3E,11S-5E,11S-7E,11S-9E,11S-11E,11S-13E,11S-15E,13S-59W,13S-57W,13S-55W,13S-53W,13S-51W,13S-49W,13S-47W,13S-45W,13S-43W,13S-41W,13S-39W,13S-37W,13S-35W,13S-33W,13S-31W,13S-29W,13S-27W,13S-25W,13S-23W,13S-21W,13S-19W,13S-17W,13S-15W,13S-13W,13S-11W,13S-9W,13S-7W,13S-5W,13S-3W,13S-1W,13S-1E,13S-3E,13S-5E,13S-7E,13S-9E,13S-11E,13S-13E,13S-15E,15S-59W,15S-57W,15S-55W,15S-53W,15S-51W,15S-49W,15S-47W,15S-45W,15S-43W,15S-41W,15S-39W,15S-37W,15S-35W,15S-33W,15S-31W,15S-29W,15S-27W,15S-25W,15S-23W,15S-21W,15S-19W,15S-17W,15S-15W,15S-13W,15S-11W,15S-9W,15S-7W,15S-5W,15S-3W,15S-1W,15S-1E,15S-3E,15S-5E,15S-7E,15S-9E,15S-11E,15S-13E,15S-15E,17S-59W,17S-57W,17S-55W,17S-53W,17S-51W,17S-49W,17S-47W,17S-45W,17S-43W,17S-41W,17S-39W,17S-37W,17S-35W,17S-33W,17S-31W,17S-29W,17S-27W,17S-25W,17S-23W,17S-21W,17S-19W,17S-17W,17S-15W,17S-13W,17S-11W,17S-9W,17S-7W,17S-5W,17S-3W,17S-1W,17S-1E,17S-3E,17S-5E,17S-7E,17S-9E,17S-11E,17S-13E,17S-15E,19S-59W,19S-57W,19S-55W,19S-53W,19S-51W,19S-49W,19S-47W,19S-45W,19S-43W,19S-41W,19S-39W,19S-37W,19S-35W,19S-33W,19S-31W,19S-29W,19S-27W,19S-25W,19S-23W,19S-21W,19S-19W,19S-17W,19S-15W,19S-13W,19S-11W,19S-9W,19S-7W,19S-5W,19S-3W,19S-1W,19S-1E,19S-3E,19S-5E,19S-7E,19S-9E,19S-11E,19S-13E,19S-15E
1964-07-01,18.346934,16.812198,16.357261,15.190787,14.748898,14.500000,12.800391,13.613229,14.334922,17.593465,22.378785,28.579188,35.243297,44.715210,51.315690,58.789965,57.622305,55.8

In [26]:
pws_full_area_jul_dez_df = pws_full_area_jul_dez_df.dropna(axis=1, how='any')
pws_full_area_jul_dez_df.head(2)

,29N-59W,29N-57W,29N-55W,29N-53W,29N-51W,29N-49W,29N-47W,29N-45W,29N-43W,29N-41W,29N-39W,29N-37W,29N-35W,29N-33W,29N-31W,29N-29W,29N-27W,29N-25W,29N-23W,29N-21W,29N-19W,29N-17W,29N-15W,29N-13W,27N-59W,27N-57W,27N-55W,27N-53W,27N-51W,27N-49W,27N-47W,27N-45W,27N-43W,27N-41W,27N-39W,27N-37W,27N-35W,27N-33W,27N-31W,27N-29W,27N-27W,27N-25W,27N-23W,27N-21W,27N-19W,27N-17W,27N-15W,25N-59W,25N-57W,25N-55W,25N-53W,25N-51W,25N-49W,25N-47W,25N-45W,25N-43W,25N-41W,25N-39W,25N-37W,25N-35W,25N-33W,25N-31W,25N-29W,25N-27W,25N-25W,25N-23W,25N-21W,25N-19W,25N-17W,25N-15W,23N-59W,23N-57W,23N-55W,23N-53W,23N-51W,23N-49W,23N-47W,23N-45W,23N-43W,23N-41W,23N-39W,23N-37W,23N-35W,23N-33W,23N-31W,23N-29W,23N-27W,23N-25W,23N-23W,23N-21W,23N-19W,23N-17W,21N-59W,21N-57W,21N-55W,21N-53W,21N-51W,21N-49W,21N-47W,21N-45W,21N-43W,21N-41W,21N-39W,21N-37W,21N-35W,21N-33W,21N-31W,21N-29W,21N-27W,21N-25W,21N-23W,21N-21W,21N-19W,21N-17W,19N-59W,19N-57W,19N-55W,19N-53W,19N-51W,19N-49W,19N-47W,19N-45W,19N-43W,19N-41W,19N-39W,19N-37W,19N-35W,19N-33W,19N-31W,19N-29W,19N-27W,19N-25W,19N-23W,19N-21W,19N-19W,19N-17W,17N-59W,17N-57W,17N-55W,17N-53W,17N-51W,17N-49W,17N-47W,17N-45W,17N-43W,17N-41W,17N-39W,17N-37W,17N-35W,17N-33W,17N-31W,17N-29W,17N-27W,17N-25W,17N-23W,17N-21W,17N-19W,17N-17W,15N-59W,15N-57W,15N-55W,15N-53W,15N-51W,15N-49W,15N-47W,15N-45W,15N-43W,15N-41W,15N-39W,15N-37W,15N-35W,15N-33W,15N-31W,15N-29W,15N-27W,15N-25W,15N-23W,15N-21W,15N-19W,15N-17W,13N-59W,13N-57W,13N-55W,13N-53W,13N-51W,13N-49W,13N-47W,13N-45W,13N-43W,13N-41W,13N-39W,13N-37W,13N-35W,13N-33W,13N-31W,13N-29W,13N-27W,13N-25W,13N-23W,13N-21W,13N-19W,13N-17W,11N-59W,11N-57W,11N-55W,11N-53W,11N-51W,11N-49W,11N-47W,11N-45W,11N-43W,11N-41W,11N-39W,11N-37W,11N-35W,11N-33W,11N-31W,11N-29W,11N-27W,11N-25W,11N-23W,11N-21W,11N-19W,11N-17W,11N-15W,9N-59W,9N-57W,9N-55W,9N-53W,9N-51W,9N-49W,9N-47W,9N-45W,9N-43W,9N-41W,9N-39W,9N-37W,9N-35W,9N-33W,9N-31W,9N-29W,9N-27W,9N-25W,9N-23W,9N-21W,9N-19W,9N-17W,9N-15W,9N-13W,7N-57W,...,1S-19W,1S-17W,1S-15W,1S-13W,1S-11W,1S-9W,1S-7W,1S-5W,1S-3W,1S-1W,1S-1E,1S-3E,1S-5E,1S-7E,1S-9E,3S-43W,3S-41W,3S-39W,3S-37W,3S-35W,3S-33W,3S-31W,3S-29W,3S-27W,3S-25W,3S-23W,3S-21W,3S-19W,3S-17W,3S-15W,3S-13W,3S-11W,3S-9W,3S-7W,3S-5W,3S-3W,3S-1W,3S-1E,3S-3E,3S-5E,3S-7E,3S-9E,5S-37W,5S-35W,5S-33W,5S-31W,5S-29W,5S-27W,5S-25W,5S-23W,5S-21W,5S-19W,5S-17W,5S-15W,5S-13W,5S-11W,5S-9W,5S-7W,5S-5W,5S-3W,5S-1W,5S-1E,5S-3E,5S-5E,5S-7E,5S-9E,5S-11E,7S-35W,7S-33W,7S-31W,7S-29W,7S-27W,7S-25W,7S-23W,7S-21W,7S-19W,7S-17W,7S-15W,7S-13W,7S-11W,7S-9W,7S-7W,7S-5W,7S-3W,7S-1W,7S-1E,7S-3E,7S-5E,7S-7E,7S-9E,7S-11E,7S-13E,9S-35W,9S-33W,9S-31W,9S-29W,9S-27W,9S-25W,9S-23W,9S-21W,9S-19W,9S-17W,9S-15W,9S-13W,9S-11W,9S-9W,9S-7W,9S-5W,9S-3W,9S-1W,9S-1E,9S-3E,9S-5E,9S-7E,9S-9E,9S-11E,9S-13E,11S-37W,11S-35W,11S-33W,11S-31W,11S-29W,11S-27W,11S-25W,11S-23W,11S-21W,11S-19W,11S-17W,11S-15W,11S-13W,11S-11W,11S-9W,11S-7W,11S-5W,11S-3W,11S-1W,11S-1E,11S-3E,11S-5E,11S-7E,11S-9E,11S-11E,11S-13E,13S-39W,13S-37W,13S-35W,13S-33W,13S-31W,13S-29W,13S-27W,13S-25W,13S-23W,13S-21W,13S-19W,13S-17W,13S-15W,13S-13W,13S-11W,13S-9W,13S-7W,13S-5W,13S-3W,13S-1W,13S-1E,13S-3E,13S-5E,13S-7E,13S-9E,13S-11E,13S-13E,15S-39W,15S-37W,15S-35W,15S-33W,15S-31W,15S-29W,15S-27W,15S-25W,15S-23W,15S-21W,15S-19W,15S-17W,15S-15W,15S-13W,15S-11W,15S-9W,15S-7W,15S-5W,15S-3W,15S-1W,15S-1E,15S-3E,15S-5E,15S-7E,15S-9E,15S-11E,15S-13E,17S-39W,17S-37W,17S-35W,17S-33W,17S-31W,17S-29W,17S-27W,17S-25W,17S-23W,17S-21W,17S-19W,17S-17W,17S-15W,17S-13W,17S-11W,17S-9W,17S-7W,17S-5W,17S-3W,17S-1W,17S-1E,17S-3E,17S-5E,17S-7E,17S-9E,17S-11E,19S-39W,19S-37W,19S-35W,19S-33W,19S-31W,19S-29W,19S-27W,19S-25W,19S-23W,19S-21W,19S-19W,19S-17W,19S-15W,19S-13W,19S-11W,19S-9W,19S-7W,19S-5W,19S-3W,19S-1W,19S-1E,19S-3E,19S-5E,19S-7E,19S-9E,19S-11E,19S-13E
1964-07-01,18.346934,16.812198,16.357261,15.190787,14.748898,14.500000,12.800391,13.613229,14.334922,17.593465,22.378785,28.579188,35.243297,44.715210,51.315690,58.789965,57.622305,55.833771,55.286979,59.052942,63.558556,64.111933,61.036

In [27]:
## Creating transformed df PWS full area

In [28]:
columns = pws_full_area_jul_dez_df.columns 
months = second_semester_months
pws_merged_colum_names=[]

for col in columns:
    for month in months:
        column_name = 'pws_'+ col + '_' +str(month)
        pws_merged_colum_names.append(column_name)

        #For debug only        
# for column_name in merged_colum_names:
#     print(column_name)
            

In [29]:
pws_transformed_df = pd.DataFrame(index=range(1973,2018,1), columns=pws_merged_colum_names)

for year in range(1973,2018,1):
#     print("Year " , year) #For debug only
    for month in range(7,13,1):
        for pws_column in pws_full_area_jul_dez_df.columns:
            value = pws_full_area_jul_dez_df.loc[str(year)+'-'+str(month)+'-01'][pws_column]
            column_name = 'pws_' +pws_column+'_'+str(month)
            pws_transformed_df.set_value(year, column_name, value)

In [30]:
pws_transformed_df = create_transformed_df(pws_full_area_jul_dez_df, "pws_")

In [31]:
pws_transformed_df.head()

,pws_29N-59W_7,pws_29N-59W_8,pws_29N-59W_9,pws_29N-59W_10,pws_29N-59W_11,pws_29N-59W_12,pws_29N-57W_7,pws_29N-57W_8,pws_29N-57W_9,pws_29N-57W_10,pws_29N-57W_11,pws_29N-57W_12,pws_29N-55W_7,pws_29N-55W_8,pws_29N-55W_9,pws_29N-55W_10,pws_29N-55W_11,pws_29N-55W_12,pws_29N-53W_7,pws_29N-53W_8,pws_29N-53W_9,pws_29N-53W_10,pws_29N-53W_11,pws_29N-53W_12,pws_29N-51W_7,pws_29N-51W_8,pws_29N-51W_9,pws_29N-51W_10,pws_29N-51W_11,pws_29N-51W_12,pws_29N-49W_7,pws_29N-49W_8,pws_29N-49W_9,pws_29N-49W_10,pws_29N-49W_11,pws_29N-49W_12,pws_29N-47W_7,pws_29N-47W_8,pws_29N-47W_9,pws_29N-47W_10,pws_29N-47W_11,pws_29N-47W_12,pws_29N-45W_7,pws_29N-45W_8,pws_29N-45W_9,pws_29N-45W_10,pws_29N-45W_11,pws_29N-45W_12,pws_29N-43W_7,pws_29N-43W_8,pws_29N-43W_9,pws_29N-43W_10,pws_29N-43W_11,pws_29N-43W_12,pws_29N-41W_7,pws_29N-41W_8,pws_29N-41W_9,pws_29N-41W_10,pws_29N-41W_11,pws_29N-41W_12,pws_29N-39W_7,pws_29N-39W_8,pws_29N-39W_9,pws_29N-39W_10,pws_29N-39W_11,pws_29N-39W_12,pws_29N-37W_7,pws_29N-37W_8,pws_29N-37W_9,pws_29N-37W_10,pws_29N-37W_11,pws_29N-37W_12,pws_29N-35W_7,pws_29N-35W_8,pws_29N-35W_9,pws_29N-35W_10,pws_29N-35W_11,pws_29N-35W_12,pws_29N-33W_7,pws_29N-33W_8,pws_29N-33W_9,pws_29N-33W_10,pws_29N-33W_11,pws_29N-33W_12,pws_29N-31W_7,pws_29N-31W_8,pws_29N-31W_9,pws_29N-31W_10,pws_29N-31W_11,pws_29N-31W_12,pws_29N-29W_7,pws_29N-29W_8,pws_29N-29W_9,pws_29N-29W_10,pws_29N-29W_11,pws_29N-29W_12,pws_29N-27W_7,pws_29N-27W_8,pws_29N-27W_9,pws_29N-27W_10,pws_29N-27W_11,pws_29N-27W_12,pws_29N-25W_7,pws_29N-25W_8,pws_29N-25W_9,pws_29N-25W_10,pws_29N-25W_11,pws_29N-25W_12,pws_29N-23W_7,pws_29N-23W_8,pws_29N-23W_9,pws_29N-23W_10,pws_29N-23W_11,pws_29N-23W_12,pws_29N-21W_7,pws_29N-21W_8,pws_29N-21W_9,pws_29N-21W_10,pws_29N-21W_11,pws_29N-21W_12,pws_29N-19W_7,pws_29N-19W_8,pws_29N-19W_9,pws_29N-19W_10,pws_29N-19W_11,pws_29N-19W_12,pws_29N-17W_7,pws_29N-17W_8,pws_29N-17W_9,pws_29N-17W_10,pws_29N-17W_11,pws_29N-17W_12,pws_29N-15W_7,pws_29N-15W_8,pws_29N-15W_9,pws_29N-15W_10,pws_29N-15W_11,pws_29N-15W_12,pws_29N-13W_7,pws_29N-13W_8,pws_29N-13W_9,pws_29N-13W_10,pws_29N-13W_11,pws_29N-13W_12,pws_27N-59W_7,pws_27N-59W_8,pws_27N-59W_9,pws_27N-59W_10,pws_27N-59W_11,pws_27N-59W_12,pws_27N-57W_7,pws_27N-57W_8,pws_27N-57W_9,pws_27N-57W_10,pws_27N-57W_11,pws_27N-57W_12,pws_27N-55W_7,pws_27N-55W_8,pws_27N-55W_9,pws_27N-55W_10,pws_27N-55W_11,pws_27N-55W_12,pws_27N-53W_7,pws_27N-53W_8,pws_27N-53W_9,pws_27N-53W_10,pws_27N-53W_11,pws_27N-53W_12,pws_27N-51W_7,pws_27N-51W_8,pws_27N-51W_9,pws_27N-51W_10,pws_27N-51W_11,pws_27N-51W_12,pws_27N-49W_7,pws_27N-49W_8,pws_27N-49W_9,pws_27N-49W_10,pws_27N-49W_11,pws_27N-49W_12,pws_27N-47W_7,pws_27N-47W_8,pws_27N-47W_9,pws_27N-47W_10,pws_27N-47W_11,pws_27N-47W_12,pws_27N-45W_7,pws_27N-45W_8,pws_27N-45W_9,pws_27N-45W_10,pws_27N-45W_11,pws_27N-45W_12,pws_27N-43W_7,pws_27N-43W_8,pws_27N-43W_9,pws_27N-43W_10,pws_27N-43W_11,pws_27N-43W_12,pws_27N-41W_7,pws_27N-41W_8,pws_27N-41W_9,pws_27N-41W_10,pws_27N-41W_11,pws_27N-41W_12,pws_27N-39W_7,pws_27N-39W_8,pws_27N-39W_9,pws_27N-39W_10,pws_27N-39W_11,pws_27N-39W_12,pws_27N-37W_7,pws_27N-37W_8,pws_27N-37W_9,pws_27N-37W_10,pws_27N-37W_11,pws_27N-37W_12,pws_27N-35W_7,pws_27N-35W_8,pws_27N-35W_9,pws_27N-35W_10,pws_27N-35W_11,pws_27N-35W_12,pws_27N-33W_7,pws_27N-33W_8,pws_27N-33W_9,pws_27N-33W_10,pws_27N-33W_11,pws_27N-33W_12,pws_27N-31W_7,pws_27N-31W_8,pws_27N-31W_9,pws_27N-31W_10,pws_27N-31W_11,pws_27N-31W_12,pws_27N-29W_7,pws_27N-29W_8,pws_27N-29W_9,pws_27N-29W_10,pws_27N-29W_11,pws_27N-29W_12,pws_27N-27W_7,pws_27N-27W_8,pws_27N-27W_9,pws_27N-27W_10,pws_27N-27W_11,pws_27N-27W_12,pws_27N-25W_7,pws_27N-25W_8,pws_27N-25W_9,pws_27N-25W_10,...,pws_17S-17W_9,pws_17S-17W_10,pws_17S-17W_11,pws_17S-17W_12,pws_17S-15W_7,pws_17S-15W_8,pws_17S-15W_9,pws_17S-15W_10,pws_17S-15W_11,pws_17S-15W_12,pws_17S-13W_7,pws_17S-13W_8,pws_17S-13W_9,pws_17S-13W_10,pws_17S-13W_11,pws_17S-13W_12,pws_17S-11W_7,pws_17S-11W_8,pws_17S-11W_9,pws_17S-11W_10,pws_17S-11W_11,pws_17S-11W_12,pws_17S-9W_7,pws_17S-9W_8,pws_17S-9W_9,pws_17S-9W_1

# Merge sst_transformed_df and pws_transformed_df

### Droping first row


In [32]:
rainy_classified_df.drop(1973, inplace=True)

In [33]:
rainy_classified_df

1974    strong
1975    normal
1976      weak
1977      weak
1978      weak
1979      weak
1980      weak
1981      weak
1982      weak
1983      weak
1984    normal
1985    strong
1986    strong
1987      weak
1988    normal
1989    normal
1990      weak
1991      weak
1992      weak
1993      weak
1994    normal
1995    normal
1996    normal
1997      weak
1998      weak
1999      weak
2000    normal
2001      weak
2002      weak
2003    normal
2004      weak
2005      weak
2006      weak
2007      weak
2008    normal
2009    normal
2010      weak
2011      weak
2012      weak
2013      weak
2014      weak
2015      weak
2016      weak
2017      weak
Name: classes, dtype: object

# Decrease index to make easy join by index(year)

In [34]:
rainy_classified_df.index = range(1973,2017)

In [35]:
rainy_classified_df

1973    strong
1974    normal
1975      weak
1976      weak
1977      weak
1978      weak
1979      weak
1980      weak
1981      weak
1982      weak
1983    normal
1984    strong
1985    strong
1986      weak
1987    normal
1988    normal
1989      weak
1990      weak
1991      weak
1992      weak
1993    normal
1994    normal
1995    normal
1996      weak
1997      weak
1998      weak
1999    normal
2000      weak
2001      weak
2002    normal
2003      weak
2004      weak
2005      weak
2006      weak
2007    normal
2008    normal
2009      weak
2010      weak
2011      weak
2012      weak
2013      weak
2014      weak
2015      weak
2016      weak
Name: classes, dtype: object

#Defining classificers functions

In [36]:
def test_knn(X,y):
    
    knn = KNeighborsClassifier(n_neighbors=2)

    knn.fit(X, y)
    new_pred_class = knn.predict(X_test)
    metrics.accuracy_score(test_df['classes'], new_pred_class)
    year = 2001
    for v in new_pred_class:
        print(year, v)
    return new_pred_class

In [37]:
def test_svc(X,y):
    svc = SVC()
    svc.fit(X, y)
    new_pred_class = svc.predict(X_test)
    year = 2001
    for v in new_pred_class:
        print(year, v)
        year += 1
    return new_pred_class

In [38]:
def test_gpc(X,y):
    gpc = GaussianProcessClassifier()
    gpc.fit(X, y)
    new_pred_class = gpc.predict(X_test)
    year = 2001
    for v in new_pred_class:
        print(year, v)
        year += 1
    return new_pred_class

In [39]:
def test_dtc(X,y):

    dtc = DecisionTreeClassifier(random_state=1)
    dtc.fit(X, y)
    new_pred_class = dtc.predict(X_test)
    year = 2001
    for v in new_pred_class:
        print(year, v)
        year += 1
    return new_pred_class

In [40]:
def test_rfc(X,y):

    rfc = RandomForestClassifier(random_state=1)
    rfc.fit(X, y)
    new_pred_class = rfc.predict(X_test)
    year = 2001
    for v in new_pred_class:
        print(year, v)
        year += 1
    return new_pred_class

In [41]:
def test_mlp(X,y):
    mlp = MLPClassifier(random_state=1)
    mlp.fit(X, y)
    new_pred_class = mlp.predict(X_test)
    year = 2001
    for v in new_pred_class:
        print(year, v)
        year += 1
    return new_pred_class

In [42]:
def test_adc(X,y):
    adc = AdaBoostClassifier(random_state=1)
    adc.fit(X, y)
    new_pred_class = adc.predict(X_test)
    year = 2001
    for v in new_pred_class:
        print(year, v)
        year += 1
    return new_pred_class

In [43]:
def test_gnb(X,y):
    gnb = GaussianNB()
    gnb.fit(X, y)
    new_pred_class = gnb.predict(X_test)
    year = 2001
    for v in new_pred_class:
        print(year, v)
        year += 1
    return new_pred_class

### Init columns form regions: coast1, coast2, coast3

In [44]:
colunas_coast_1 = constroi_colunas_latitude_longitude(init_lat=0, end_lat=-5,
                                                    init_long=39, end_long=30)

colunas_coast_2 = constroi_colunas_latitude_longitude(init_lat=0, end_lat=-10,
                                                    init_long=49, end_long=29)

colunas_coast_3 = constroi_colunas_latitude_longitude(init_lat=5, end_lat=-10,
                                                    init_long=50, end_long=19)


In [45]:
colunas_coast_1

['1S-39W',
 '1S-37W',
 '1S-35W',
 '1S-33W',
 '1S-31W',
 '3S-39W',
 '3S-37W',
 '3S-35W',
 '3S-33W',
 '3S-31W',
 '5S-39W',
 '5S-37W',
 '5S-35W',
 '5S-33W',
 '5S-31W']

In [46]:
colunas_coast_2

['1S-49W',
 '1S-47W',
 '1S-45W',
 '1S-43W',
 '1S-41W',
 '1S-39W',
 '1S-37W',
 '1S-35W',
 '1S-33W',
 '1S-31W',
 '1S-29W',
 '3S-49W',
 '3S-47W',
 '3S-45W',
 '3S-43W',
 '3S-41W',
 '3S-39W',
 '3S-37W',
 '3S-35W',
 '3S-33W',
 '3S-31W',
 '3S-29W',
 '5S-49W',
 '5S-47W',
 '5S-45W',
 '5S-43W',
 '5S-41W',
 '5S-39W',
 '5S-37W',
 '5S-35W',
 '5S-33W',
 '5S-31W',
 '5S-29W',
 '7S-49W',
 '7S-47W',
 '7S-45W',
 '7S-43W',
 '7S-41W',
 '7S-39W',
 '7S-37W',
 '7S-35W',
 '7S-33W',
 '7S-31W',
 '7S-29W',
 '9S-49W',
 '9S-47W',
 '9S-45W',
 '9S-43W',
 '9S-41W',
 '9S-39W',
 '9S-37W',
 '9S-35W',
 '9S-33W',
 '9S-31W',
 '9S-29W']

In [47]:
colunas_coast_3

['5N-49W',
 '5N-47W',
 '5N-45W',
 '5N-43W',
 '5N-41W',
 '5N-39W',
 '5N-37W',
 '5N-35W',
 '5N-33W',
 '5N-31W',
 '5N-29W',
 '5N-27W',
 '5N-25W',
 '5N-23W',
 '5N-21W',
 '5N-19W',
 '3N-49W',
 '3N-47W',
 '3N-45W',
 '3N-43W',
 '3N-41W',
 '3N-39W',
 '3N-37W',
 '3N-35W',
 '3N-33W',
 '3N-31W',
 '3N-29W',
 '3N-27W',
 '3N-25W',
 '3N-23W',
 '3N-21W',
 '3N-19W',
 '1N-49W',
 '1N-47W',
 '1N-45W',
 '1N-43W',
 '1N-41W',
 '1N-39W',
 '1N-37W',
 '1N-35W',
 '1N-33W',
 '1N-31W',
 '1N-29W',
 '1N-27W',
 '1N-25W',
 '1N-23W',
 '1N-21W',
 '1N-19W',
 '1S-49W',
 '1S-47W',
 '1S-45W',
 '1S-43W',
 '1S-41W',
 '1S-39W',
 '1S-37W',
 '1S-35W',
 '1S-33W',
 '1S-31W',
 '1S-29W',
 '1S-27W',
 '1S-25W',
 '1S-23W',
 '1S-21W',
 '1S-19W',
 '3S-49W',
 '3S-47W',
 '3S-45W',
 '3S-43W',
 '3S-41W',
 '3S-39W',
 '3S-37W',
 '3S-35W',
 '3S-33W',
 '3S-31W',
 '3S-29W',
 '3S-27W',
 '3S-25W',
 '3S-23W',
 '3S-21W',
 '3S-19W',
 '5S-49W',
 '5S-47W',
 '5S-45W',
 '5S-43W',
 '5S-41W',
 '5S-39W',
 '5S-37W',
 '5S-35W',
 '5S-33W',
 '5S-31W',
 '5S-29W',

### Init dataframes from : coast1, coast2, coast3

In [48]:
coast1_sst_df=pd.DataFrame(sst_full_area_jul_dez_df, index=sst_full_area_jul_dez_df.index, columns=colunas_coast_1)
coast2_sst_df=pd.DataFrame(sst_full_area_jul_dez_df, index=sst_full_area_jul_dez_df.index, columns=colunas_coast_2)
coast3_sst_df=pd.DataFrame(sst_full_area_jul_dez_df, index=sst_full_area_jul_dez_df.index, columns=colunas_coast_3)

coast1_pws_df=pd.DataFrame(pws_full_area_jul_dez_df, index=pws_full_area_jul_dez_df.index, columns=colunas_coast_1)
coast2_pws_df=pd.DataFrame(pws_full_area_jul_dez_df, index=pws_full_area_jul_dez_df.index, columns=colunas_coast_2)
coast3_pws_df=pd.DataFrame(pws_full_area_jul_dez_df, index=pws_full_area_jul_dez_df.index, columns=colunas_coast_3)

In [49]:
coast1_sst_df.head()

,1S-39W,1S-37W,1S-35W,1S-33W,1S-31W,3S-39W,3S-37W,3S-35W,3S-33W,3S-31W,5S-39W,5S-37W,5S-35W,5S-33W,5S-31W
1973-07-01,27.9,27.8,27.2,26.6,26.4,27.9,27.9,27.5,26.9,26.8,NaN,28.0,27.7,27.4,27.2
1973-08-01,27.7,27.6,27.3,27.0,26.9,27.6,27.5,27.3,27.0,26.9,NaN,27.4,27.4,27.1,27.0
1973-09-01,27.7,27.7,27.3,26.9,26.6,27.1,27.1,26.9,26.7,26.8,NaN,26.6,26.7,26.7,26.9
1973-10-01,27.9,27.9,27.6,27.2,27.0,27.8,27.7,27.5,27.1,26.9,NaN,27.5,27.4,27.3,27.1
1973-11-01,27.9,27.9,27.4,26.8,26.3,27.9,27.8,27.5,27.0,26.6,NaN,27.6,27.5,27.3,27.2


In [50]:
coast2_sst_df.head()

,1S-49W,1S-47W,1S-45W,1S-43W,1S-41W,1S-39W,1S-37W,1S-35W,1S-33W,1S-31W,1S-29W,3S-49W,3S-47W,3S-45W,3S-43W,3S-41W,3S-39W,3S-37W,3S-35W,3S-33W,3S-31W,3S-29W,5S-49W,5S-47W,5S-45W,5S-43W,5S-41W,5S-39W,5S-37W,5S-35W,5S-33W,5S-31W,5S-29W,7S-49W,7S-47W,7S-45W,7S-43W,7S-41W,7S-39W,7S-37W,7S-35W,7S-33W,7S-31W,7S-29W,9S-49W,9S-47W,9S-45W,9S-43W,9S-41W,9S-39W,9S-37W,9S-35W,9S-33W,9S-31W,9S-29W
1973-07-01,NaN,28.2,27.9,27.8,27.8,27.9,27.8,27.2,26.6,26.4,26.8,NaN,NaN,NaN,27.7,27.8,27.9,27.9,27.5,26.9,26.8,27.1,NaN,NaN,NaN,NaN,NaN,NaN,28.0,27.7,27.4,27.2,27.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.6,27.5,27.3,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.2,27.3,27.1,26.6
1973-08-01,NaN,27.0,27.5,27.8,27.8,27.7,27.6,27.3,27.0,26.9,27.0,NaN,NaN,NaN,27.9,27.7,27.6,27.5,27.3,27.0,26.9,27.0,NaN,NaN,NaN,NaN,NaN,NaN,27.4,27.4,27.1,27.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.3,27.2,27.0,26.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.2,27.1,26.9,26.5
1973-09-01,NaN,28.4,27.7,27.3,27.4,27.7,27.7,27.3,26.9,26.6,26.8,NaN,NaN,NaN,26.8,26.9,27.1,27.1,26.9,26.7,26.8,27.1,NaN,NaN,NaN,NaN,NaN,NaN,26.6,26.7,26.7,26.9,27.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.7,26.8,26.8,26.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.8,26.7,26.6,26.7
1973-10-01,NaN,27.3,27.7,27.9,28.0,27.9,27.9,27.6,27.2,27.0,27.1,NaN,NaN,NaN,28.0,27.9,27.8,27.7,27.5,27.1,26.9,27.0,NaN,NaN,NaN,NaN,NaN,NaN,27.5,27.4,27.3,27.1,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.4,27.5,27.3,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.3,27.5,27.3,26.8
1973-11-01,NaN,28.0,27.7,27.6,27.7,27.9,27.9,27.4,26.8,26.3,26.3,NaN,NaN,NaN,27.4,27.6,27.9,27.8,27.5,27.0,26.6,26.6,NaN,NaN,NaN,NaN,NaN,NaN,27.6,27.5,27.3,27.2,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.4,27.5,27.4,27.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.5,27.7,27.5,27.1


In [51]:
coast3_sst_df.head()

,5N-49W,5N-47W,5N-45W,5N-43W,5N-41W,5N-39W,5N-37W,5N-35W,5N-33W,5N-31W,5N-29W,5N-27W,5N-25W,5N-23W,5N-21W,5N-19W,3N-49W,3N-47W,3N-45W,3N-43W,3N-41W,3N-39W,3N-37W,3N-35W,3N-33W,3N-31W,3N-29W,3N-27W,3N-25W,3N-23W,3N-21W,3N-19W,1N-49W,1N-47W,1N-45W,1N-43W,1N-41W,1N-39W,1N-37W,1N-35W,1N-33W,1N-31W,1N-29W,1N-27W,1N-25W,1N-23W,1N-21W,1N-19W,1S-49W,1S-47W,1S-45W,1S-43W,1S-41W,1S-39W,1S-37W,1S-35W,1S-33W,1S-31W,1S-29W,1S-27W,1S-25W,1S-23W,1S-21W,1S-19W,3S-49W,3S-47W,3S-45W,3S-43W,3S-41W,3S-39W,3S-37W,3S-35W,3S-33W,3S-31W,3S-29W,3S-27W,3S-25W,3S-23W,3S-21W,3S-19W,5S-49W,5S-47W,5S-45W,5S-43W,5S-41W,5S-39W,5S-37W,5S-35W,5S-33W,5S-31W,5S-29W,5S-27W,5S-25W,5S-23W,5S-21W,5S-19W,7S-49W,7S-47W,7S-45W,7S-43W,7S-41W,7S-39W,7S-37W,7S-35W,7S-33W,7S-31W,7S-29W,7S-27W,7S-25W,7S-23W,7S-21W,7S-19W,9S-49W,9S-47W,9S-45W,9S-43W,9S-41W,9S-39W,9S-37W,9S-35W,9S-33W,9S-31W,9S-29W,9S-27W,9S-25W,9S-23W,9S-21W,9S-19W
1973-07-01,27.7,27.8,28.0,28.2,28.4,28.5,28.6,28.2,27.9,27.8,27.7,27.8,27.9,28.0,27.8,27.4,27.8,27.8,27.9,28.1,28.3,28.4,28.3,27.9,27.5,27.3,27.3,27.5,27.4,27.1,26.9,26.7,28.0,28.0,27.9,27.9,28.0,28.1,28.0,27.5,26.9,26.6,26.9,27.2,26.8,26.2,26.0,25.9,NaN,28.2,27.9,27.8,27.8,27.9,27.8,27.2,26.6,26.4,26.8,27.3,26.7,25.9,25.5,25.4,NaN,NaN,NaN,27.7,27.8,27.9,27.9,27.5,26.9,26.8,27.1,27.3,26.7,26.0,25.4,25.2,NaN,NaN,NaN,NaN,NaN,NaN,28.0,27.7,27.4,27.2,27.2,26.9,26.4,26.0,25.7,25.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.6,27.5,27.3,27.0,26.5,26.1,25.8,25.7,25.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.2,27.3,27.1,26.6,26.0,25.7,25.4,25.1,24.9
1973-08-01,28.1,28.1,28.3,28.6,28.7,28.6,28.5,28.5,28.3,28.0,27.7,27.4,27.4,27.3,26.9,26.5,27.7,27.4,27.7,28.2,28.4,28.3,28.2,28.1,28.0,27.6,27.3,27.2,27.0,26.6,26.1,25.8,27.5,27.1,27.4,27.9,28.1,28.0,27.8,27.6,27.4,27.1,27.1,27.1,26.5,26.0,25.6,25.4,NaN,27.0,27.5,27.8,27.8,27.7,27.6,27.3,27.0,26.9,27.0,27.0,26.4,25.6,25.3,25.2,NaN,NaN,NaN,27.9,27.7,27.6,27.5,27.3,27.0,26.9,27.0,27.0,26.4,25.6,25.1,25.1,NaN,NaN,NaN,NaN,NaN,NaN,27.4,27.4,27.1,27.0,27.0,26.7,26.3,25.7,25.2,24.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.3,27.2,27.0,26.8,26.5,26.2,26.0,25.5,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.2,27.1,26.9,26.5,26.2,26.1,25.9,25.6,25.2
1973-09-01,28.3,28.6,28.8,28.8,28.8,28.9,28.8,28.5,28.1,27.7,27.6,27.6,27.6,27.5,27.3,27.1,28.2,28.4,28.4,28.4,28.5,28.7,28.8,28.3,27.5,27.1,27.1,27.3,27.2,27.0,26.7,26.5,28.5,28.4,28.1,27.9,28.0,28.3,28.4,27.9,27.2,26.6,26.8,27.0,26.6,26.2,26.0,25.8,NaN,28.4,27.7,27.3,27.4,27.7,27.7,27.3,26.9,26.6,26.8,26.9,26.3,25.6,25.3,25.3,NaN,NaN,NaN,26.8,26.9,27.1,27.1,26.9,26.7,26.8,27.1,27.2,26.3,25.4,25.2,25.2,NaN,NaN,NaN,NaN,NaN,NaN,26.6,26.7,26.7,26.9,27.1,27.1,26.5,25.8,25.5,25.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.7,26.8,26.8,26.8,26.9,26.5,26.0,25.7,25.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.8,26.7,26.6,26.7,26.6,26.3,26.0,25.7,25.7
1973-10-01,27.9,28.2,28.3,28.3,28.2,27.9,27.7,27.9,28.2,28.3,28.3,28.3,28.1,27.7,27.4,27.3,27.7,28.0,28.2,28.3,28.3,28.1,28.0,27.9,27.9,27.9,28.0,28.1,28.0,27.6,27.2,27.1,27.5,27.8,28.0,28.0,28.1,28.2,28.0,27.8,27.6,27.4,27.5,27.6,27.5,27.2,27.1,26.9,NaN,27.3,27.7,27.9,28.0,27.9,27.9,27.6,27.2,27.0,27.1,27.2,26.9,26.7,26.7,26.5,NaN,NaN,NaN,28.0,27.9,27.8,27.7,27.5,27.1,26.9,27.0,26.9,26.7,26.5,26.2,26.0,NaN,NaN,NaN,NaN,NaN,NaN,27.5,27.4,27.3,27.1,27.0,26.9,26.7,26.4,26.1,25.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.4,27.5,27.3,27.0,26.7,26.5,26.4,26.2,25.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.3,27.5,27.3,26.8,26.2,26.1,26.1,26.0,25.7
1973-11-01,27.9,27.8,28.1,28.2,28.2,27.9,27.5,27.6,27.6,27.8,27.9,28.0,27.9,27.8,27.7,27.7,27.9,27.8,27.9,28.2,28.3,28.1,27.9,27.5,27.3,27.2,27.3,27.5,27.5,27.5,27.2,26.8,28.1,27.8,27.8,27.8,27.9,28.0,27.9,27.5,26.9,26.6,26.6,26.8,27.0,27.1,26.8,26.3,NaN,28.0,27.7,27.6,27.7,27.9,27.9,27.4,26.8,26.3,26.3,26.5,26.6,26.8,26.7,26.2,NaN,NaN,NaN,27.4,27.6,27.9,27.8,27.5,27.0,26.6,26.6,26.6,26.6,26.4,26.4,26.4,NaN,NaN,NaN,NaN,NaN,NaN,27.6,27.5,27.3,27.2,27.0,26.8,26.4,26.0,26.0,26.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.4,27.5,27.4,27.1,26.6,26.2,25.8,25.7,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.5,27.7,27.

In [52]:
coast1_pws_df.head()

,1S-39W,1S-37W,1S-35W,1S-33W,1S-31W,3S-39W,3S-37W,3S-35W,3S-33W,3S-31W,5S-39W,5S-37W,5S-35W,5S-33W,5S-31W
1964-07-01,40.474189,35.956223,29.775493,24.602845,24.627018,44.297404,38.544001,30.608005,25.830408,26.992036,NaN,42.717678,33.479098,29.353194,29.557571
1964-08-01,73.254966,64.885438,51.335173,35.711343,25.642543,78.274389,68.120188,53.149788,37.694827,29.673052,NaN,67.770938,52.405534,41.160175,39.136939
1964-09-01,83.908760,71.598953,54.557584,37.930067,29.210443,92.031516,77.218910,58.098021,41.968679,34.900573,NaN,72.182269,53.845427,42.180801,40.349597
1964-10-01,55.504414,48.256088,41.056668,35.284699,33.100604,61.015818,50.520194,41.175357,35.916709,37.075733,NaN,46.403879,35.483095,32.792835,38.709172
1964-11-01,60.788486,52.828496,42.757105,33.698961,28.125611,63.466684,51.924753,41.284380,32.514920,29.699832,NaN,46.460413,35.441924,29.219514,32.002500


In [53]:
coast2_pws_df.head()

,1S-49W,1S-47W,1S-45W,1S-43W,1S-41W,1S-39W,1S-37W,1S-35W,1S-33W,1S-31W,1S-29W,3S-49W,3S-47W,3S-45W,3S-43W,3S-41W,3S-39W,3S-37W,3S-35W,3S-33W,3S-31W,3S-29W,5S-49W,5S-47W,5S-45W,5S-43W,5S-41W,5S-39W,5S-37W,5S-35W,5S-33W,5S-31W,5S-29W,7S-49W,7S-47W,7S-45W,7S-43W,7S-41W,7S-39W,7S-37W,7S-35W,7S-33W,7S-31W,7S-29W,9S-49W,9S-47W,9S-45W,9S-43W,9S-41W,9S-39W,9S-37W,9S-35W,9S-33W,9S-31W,9S-29W
1964-07-01,NaN,32.854984,37.300000,39.484934,40.695209,40.474189,35.956223,29.775493,24.602845,24.627018,27.485451,NaN,NaN,NaN,44.040890,43.633130,44.297404,38.544001,30.608005,25.830408,26.992036,31.718922,NaN,NaN,NaN,NaN,NaN,NaN,42.717678,33.479098,29.353194,29.557571,34.042473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.052395,33.114498,32.015621,34.033072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.812058,35.300000,32.867157,31.999062
1964-08-01,NaN,48.499175,55.155235,63.111964,69.926890,73.254966,64.885438,51.335173,35.711343,25.642543,25.134041,NaN,NaN,NaN,71.451802,76.627997,78.274389,68.120188,53.149788,37.694827,29.673052,31.722705,NaN,NaN,NaN,NaN,NaN,NaN,67.770938,52.405534,41.160175,39.136939,44.055647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.846085,45.790064,48.396901,56.635325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.208959,49.401316,56.162977,64.803858
1964-09-01,NaN,52.600856,62.020803,74.168524,82.694921,83.908760,71.598953,54.557584,37.930067,29.210443,31.367659,NaN,NaN,NaN,86.046557,92.787553,92.031516,77.218910,58.098021,41.968679,34.900573,38.997051,NaN,NaN,NaN,NaN,NaN,NaN,72.182269,53.845427,42.180801,40.349597,47.273037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.916739,40.121441,44.533695,53.968231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.518587,39.571833,47.752801,57.810207
1964-10-01,NaN,35.658519,48.110914,57.804239,59.510419,55.504414,48.256088,41.056668,35.284699,33.100604,34.375136,NaN,NaN,NaN,69.947695,68.142278,61.015818,50.520194,41.175357,35.916709,37.075733,42.934834,NaN,NaN,NaN,NaN,NaN,NaN,46.403879,35.483095,32.792835,38.709172,49.234642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.537709,27.463430,37.542110,52.368311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.143485,23.477010,36.356292,52.878729
1964-11-01,NaN,25.927784,40.786885,54.952434,61.241979,60.788486,52.828496,42.757105,33.698961,28.125611,29.375160,NaN,NaN,NaN,67.146556,68.155851,63.466684,51.924753,41.284380,32.514920,29.699832,33.707714,NaN,NaN,NaN,NaN,NaN,NaN,46.460413,35.441924,29.219514,32.002500,40.949847,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.301590,27.312451,35.183235,48.111641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.315395,27.134664,36.300551,50.149875


In [54]:
coast3_pws_df.head()

,5N-49W,5N-47W,5N-45W,5N-43W,5N-41W,5N-39W,5N-37W,5N-35W,5N-33W,5N-31W,5N-29W,5N-27W,5N-25W,5N-23W,5N-21W,5N-19W,3N-49W,3N-47W,3N-45W,3N-43W,3N-41W,3N-39W,3N-37W,3N-35W,3N-33W,3N-31W,3N-29W,3N-27W,3N-25W,3N-23W,3N-21W,3N-19W,1N-49W,1N-47W,1N-45W,1N-43W,1N-41W,1N-39W,1N-37W,1N-35W,1N-33W,1N-31W,1N-29W,1N-27W,1N-25W,1N-23W,1N-21W,1N-19W,1S-49W,1S-47W,1S-45W,1S-43W,1S-41W,1S-39W,1S-37W,1S-35W,1S-33W,1S-31W,1S-29W,1S-27W,1S-25W,1S-23W,1S-21W,1S-19W,3S-49W,3S-47W,3S-45W,3S-43W,3S-41W,3S-39W,3S-37W,3S-35W,3S-33W,3S-31W,3S-29W,3S-27W,3S-25W,3S-23W,3S-21W,3S-19W,5S-49W,5S-47W,5S-45W,5S-43W,5S-41W,5S-39W,5S-37W,5S-35W,5S-33W,5S-31W,5S-29W,5S-27W,5S-25W,5S-23W,5S-21W,5S-19W,7S-49W,7S-47W,7S-45W,7S-43W,7S-41W,7S-39W,7S-37W,7S-35W,7S-33W,7S-31W,7S-29W,7S-27W,7S-25W,7S-23W,7S-21W,7S-19W,9S-49W,9S-47W,9S-45W,9S-43W,9S-41W,9S-39W,9S-37W,9S-35W,9S-33W,9S-31W,9S-29W,9S-27W,9S-25W,9S-23W,9S-21W,9S-19W
1964-07-01,10.692988,11.229426,11.280514,12.702756,14.402778,15.250246,17.755281,20.329781,21.781644,20.930361,20.497805,22.013178,24.960970,32.221887,39.658164,44.851979,15.454773,17.288725,19.159593,22.380572,24.177055,25.437178,26.802985,25.768974,23.726146,20.942302,19.523319,21.760744,25.684431,32.330326,38.438652,41.111799,22.583401,25.501569,29.279686,32.472142,33.952025,34.342394,32.203882,28.241813,23.583469,21.002143,21.653868,24.932910,28.435541,30.768328,33.378137,34.065378,NaN,32.854984,37.300000,39.484934,40.695209,40.474189,35.956223,29.775493,24.602845,24.627018,27.485451,31.030630,34.033219,33.324165,32.128648,31.062840,NaN,NaN,NaN,44.040890,43.633130,44.297404,38.544001,30.608005,25.830408,26.992036,31.718922,37.110915,39.668753,38.683976,36.627039,36.249138,NaN,NaN,NaN,NaN,NaN,NaN,42.717678,33.479098,29.353194,29.557571,34.042473,39.462007,40.376354,39.757515,41.962364,46.779910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.052395,33.114498,32.015621,34.033072,34.220023,32.467984,34.139274,41.011218,50.438577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.812058,35.300000,32.867157,31.999062,26.913380,22.556817,25.235689,34.488984,46.113339
1964-08-01,20.447494,23.405341,25.316003,28.708884,33.852769,39.408755,39.197066,33.057828,27.074896,25.700195,30.978218,40.477648,46.802778,50.800886,51.660527,49.545938,28.675076,32.249031,35.415957,39.940080,46.250189,51.600097,50.464740,41.769846,31.770584,27.070279,29.823648,36.179690,40.240527,42.311464,42.617485,41.047899,36.338547,41.099392,46.099024,52.162151,59.064202,64.015779,59.142709,47.589600,33.902950,25.850725,25.719448,29.565520,33.452205,34.148499,32.440099,29.600676,NaN,48.499175,55.155235,63.111964,69.926890,73.254966,64.885438,51.335173,35.711343,25.642543,25.134041,28.892213,32.244224,31.312138,28.023205,23.140009,NaN,NaN,NaN,71.451802,76.627997,78.274389,68.120188,53.149788,37.694827,29.673052,31.722705,36.623080,38.626416,35.558403,29.860509,23.306008,NaN,NaN,NaN,NaN,NaN,NaN,67.770938,52.405534,41.160175,39.136939,44.055647,50.782871,54.061539,49.729267,39.785927,30.376471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.846085,45.790064,48.396901,56.635325,66.638202,73.968439,70.277166,57.656656,42.441725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.208959,49.401316,56.162977,64.803858,79.069716,90.069362,88.416571,74.868485,57.910707
1964-09-01,21.733154,27.840977,33.032408,35.863770,35.328883,31.921466,30.183770,27.419154,28.733430,29.937602,30.331831,31.614079,31.672385,35.406920,39.256210,41.105231,29.213867,36.292699,43.472865,50.100399,53.501869,52.211589,45.190818,37.161943,32.300000,30.371203,30.594444,30.991934,30.731905,31.246440,32.829864,35.435858,39.141410,45.078154,53.485793,62.583784,69.305267,69.856997,59.634302,46.405280,33.882887,27.821754,28.817529,29.706902,30.134698,27.882970,26.518861,28.129877,NaN,52.600856,62.020803,74.168524,82.694921,83.908760,71.598953,54.557584,37.930067,29.210443,31.367659,34.122280,34.494202,31.541243,30.072745,29.333428,NaN,NaN,NaN,86.046557,92.787553,92.031516,77.218910,58.098021,41.968679,34.900573,38.997051,43.649513,43.931765,41.438267,38.415752,37.833583,NaN,NaN,NaN,NaN,NaN,Na

## Preprossessing data

In [55]:
### Removing NaN

In [56]:
coast1_sst_df.dropna(axis=1, how='any',inplace=True)
coast2_sst_df.dropna(axis=1, how='any', inplace=True)
coast3_sst_df.dropna(axis=1, how='any', inplace=True)

coast1_pws_df.dropna(axis=1, how='any', inplace=True)
coast2_pws_df.dropna(axis=1, how='any', inplace=True)
coast3_pws_df.dropna(axis=1, how='any', inplace=True)

In [57]:
## TODO: feature scaling

In [58]:
scaler = MinMaxScaler()

coast1_sst_scaled_df = pd.DataFrame(scaler.fit_transform(coast1_sst_df), columns=coast1_sst_df.columns)
coast1_sst_scaled_df.index = coast1_sst_df.index

coast2_sst_scaled_df = pd.DataFrame(scaler.fit_transform(coast2_sst_df), columns=coast2_sst_df.columns)
coast2_sst_scaled_df.index = coast2_sst_df.index

coast3_sst_scaled_df = pd.DataFrame(scaler.fit_transform(coast3_sst_df), columns=coast3_sst_df.columns)
coast3_sst_scaled_df.index = coast3_sst_df.index

coast1_pws_scaled_df = pd.DataFrame(scaler.fit_transform(coast1_pws_df), columns=coast1_pws_df.columns)
coast1_pws_scaled_df.index = coast1_pws_df.index

coast2_pws_scaled_df = pd.DataFrame(scaler.fit_transform(coast2_pws_df), columns=coast2_pws_df.columns)
coast2_pws_scaled_df.index = coast2_pws_df.index

coast3_pws_scaled_df = pd.DataFrame(scaler.fit_transform(coast3_pws_df), columns=coast3_pws_df.columns)
coast3_pws_scaled_df.index = coast3_pws_df.index


### Prepare data for coast1

In [59]:
coast1_sst_transformed_df = create_transformed_df(coast1_sst_scaled_df, "sst_")
coast1_sst_transformed_df.head()

,sst_1S-39W_7,sst_1S-39W_8,sst_1S-39W_9,sst_1S-39W_10,sst_1S-39W_11,sst_1S-39W_12,sst_1S-37W_7,sst_1S-37W_8,sst_1S-37W_9,sst_1S-37W_10,sst_1S-37W_11,sst_1S-37W_12,sst_1S-35W_7,sst_1S-35W_8,sst_1S-35W_9,sst_1S-35W_10,sst_1S-35W_11,sst_1S-35W_12,sst_1S-33W_7,sst_1S-33W_8,sst_1S-33W_9,sst_1S-33W_10,sst_1S-33W_11,sst_1S-33W_12,sst_1S-31W_7,sst_1S-31W_8,sst_1S-31W_9,sst_1S-31W_10,sst_1S-31W_11,sst_1S-31W_12,sst_3S-39W_7,sst_3S-39W_8,sst_3S-39W_9,sst_3S-39W_10,sst_3S-39W_11,sst_3S-39W_12,sst_3S-37W_7,sst_3S-37W_8,sst_3S-37W_9,sst_3S-37W_10,sst_3S-37W_11,sst_3S-37W_12,sst_3S-35W_7,sst_3S-35W_8,sst_3S-35W_9,sst_3S-35W_10,sst_3S-35W_11,sst_3S-35W_12,sst_3S-33W_7,sst_3S-33W_8,sst_3S-33W_9,sst_3S-33W_10,sst_3S-33W_11,sst_3S-33W_12,sst_3S-31W_7,sst_3S-31W_8,sst_3S-31W_9,sst_3S-31W_10,sst_3S-31W_11,sst_3S-31W_12,sst_5S-37W_7,sst_5S-37W_8,sst_5S-37W_9,sst_5S-37W_10,sst_5S-37W_11,sst_5S-37W_12,sst_5S-35W_7,sst_5S-35W_8,sst_5S-35W_9,sst_5S-35W_10,sst_5S-35W_11,sst_5S-35W_12,sst_5S-33W_7,sst_5S-33W_8,sst_5S-33W_9,sst_5S-33W_10,sst_5S-33W_11,sst_5S-33W_12,sst_5S-31W_7,sst_5S-31W_8,sst_5S-31W_9,sst_5S-31W_10,sst_5S-31W_11,sst_5S-31W_12
1973,0.992933,0.985866,0.985866,0.992933,0.992933,0.978799,0.985915,0.978873,0.982394,0.989437,0.989437,0.982394,0.967972,0.97153,0.97153,0.982206,0.975089,0.975089,0.953405,0.967742,0.964158,0.97491,0.960573,0.964158,0.94964,0.967626,0.956835,0.971223,0.946043,0.956835,0.996466,0.985866,0.968198,0.992933,0.996466,0.971731,0.996454,0.98227,0.968085,0.989362,0.992908,0.98227,0.989247,0.982079,0.967742,0.989247,0.989247,0.982079,0.967742,0.971326,0.960573,0.97491,0.971326,0.971326,0.967509,0.971119,0.967509,0.971119,0.960289,0.963899,0.992958,0.971831,0.943662,0.975352,0.978873,0.975352,0.985816,0.975177,0.950355,0.975177,0.978723,0.975177,0.985663,0.97491,0.960573,0.982079,0.982079,0.978495,0.978417,0.971223,0.967626,0.97482,0.978417,0.971223
1974,0.982332,0.936396,0.961131,0.975265,0.975265,0.961131,0.975352,0.93662,0.950704,0.961268,0.96831,0.957746,0.97153,0.932384,0.953737,0.953737,0.960854,0.950178,0.97491,0.935484,0.960573,0.946237,0.960573,0.946237,0.971223,0.935252,0.94964,0.938849,0.960432,0.94964,0.982332,0.936396,0.961131,0.975265,0.975265,0.971731,0.978723,0.939716,0.953901,0.964539,0.971631,0.971631,0.985663,0.942652,0.960573,0.964158,0.971326,0.967742,0.982079,0.939068,0.953405,0.953405,0.967742,0.949821,0.981949,0.942238,0.953069,0.949458,0.963899,0.949458,0.964789,0.933099,0.947183,0.961268,0.957746,0.982394,0.975177,0.93617,0.946809,0.957447,0.960993,0.968085,0.989247,0.942652,0.949821,0.956989,0.967742,0.956989,0.989209,0.946043,0.94964,0.953237,0.967626,0.953237
1975,0.978799,0.961131,0.943463,0.964664,0.975265,0.971731,0.978873,0.961268,0.943662,0.964789,0.964789,0.964789,0.964413,0.950178,0.932384,0.957295,0.960854,0.957295,0.946237,0.935484,0.917563,0.949821,0.956989,0.953405,0.935252,0.928058,0.910072,0.938849,0.953237,0.956835,0.978799,0.946996,0.939929,0.961131,0.985866,0.978799,0.98227,0.950355,0.939716,0.964539,0.98227,0.978723,0.978495,0.956989,0.939068,0.964158,0.97491,0.97491,0.964158,0.949821,0.924731,0.949821,0.964158,0.960573,0.953069,0.945848,0.927798,0.945848,0.960289,0.960289,0.964789,0.922535,0.943662,0.964789,0.978873,0.989437,0.968085,0.939716,0.93617,0.953901,0.975177,0.975177,0.971326,0.956989,0.935484,0.949821,0.971326,0.967742,0.964029,0.956835,0.935252,0.946043,0.964029,0.964029
1976,0.936396,0.932862,0.95053,0.95053,0.968198,0.961131,0.926056,0.93662,0.940141,0.943662,0.964789,0.957746,0.914591,0.921708,0.932384,0.935943,0.953737,0.950178,0.903226,0.90681,0.924731,0.935484,0.946237,0.942652,0.899281,0.899281,0.92446,0.938849,0.938849,0.938849,0.943463,0.932862,0.95053,0.954064,0.964664,0.964664,0.939716,0.939716,0.939716,0.943262,0.964539,0.964539,0.9319,0.9319,0.935484,0.942652,0.964158,0.967742,0.917563,0.913978,0.924731,0.935484,0.953405,0.953405,0.913357,0.909747,0.920578,0.935018,0.953069,0.945848,0.943662,0.922535,0.929577,0.940141,0.950704,0.961268,0.93617,0.92198

In [60]:
coast1_pws_transformed_df = create_transformed_df(coast1_pws_scaled_df, "pws_")
coast1_pws_transformed_df.head()

,pws_1S-39W_7,pws_1S-39W_8,pws_1S-39W_9,pws_1S-39W_10,pws_1S-39W_11,pws_1S-39W_12,pws_1S-37W_7,pws_1S-37W_8,pws_1S-37W_9,pws_1S-37W_10,pws_1S-37W_11,pws_1S-37W_12,pws_1S-35W_7,pws_1S-35W_8,pws_1S-35W_9,pws_1S-35W_10,pws_1S-35W_11,pws_1S-35W_12,pws_1S-33W_7,pws_1S-33W_8,pws_1S-33W_9,pws_1S-33W_10,pws_1S-33W_11,pws_1S-33W_12,pws_1S-31W_7,pws_1S-31W_8,pws_1S-31W_9,pws_1S-31W_10,pws_1S-31W_11,pws_1S-31W_12,pws_3S-39W_7,pws_3S-39W_8,pws_3S-39W_9,pws_3S-39W_10,pws_3S-39W_11,pws_3S-39W_12,pws_3S-37W_7,pws_3S-37W_8,pws_3S-37W_9,pws_3S-37W_10,pws_3S-37W_11,pws_3S-37W_12,pws_3S-35W_7,pws_3S-35W_8,pws_3S-35W_9,pws_3S-35W_10,pws_3S-35W_11,pws_3S-35W_12,pws_3S-33W_7,pws_3S-33W_8,pws_3S-33W_9,pws_3S-33W_10,pws_3S-33W_11,pws_3S-33W_12,pws_3S-31W_7,pws_3S-31W_8,pws_3S-31W_9,pws_3S-31W_10,pws_3S-31W_11,pws_3S-31W_12,pws_5S-37W_7,pws_5S-37W_8,pws_5S-37W_9,pws_5S-37W_10,pws_5S-37W_11,pws_5S-37W_12,pws_5S-35W_7,pws_5S-35W_8,pws_5S-35W_9,pws_5S-35W_10,pws_5S-35W_11,pws_5S-35W_12,pws_5S-33W_7,pws_5S-33W_8,pws_5S-33W_9,pws_5S-33W_10,pws_5S-33W_11,pws_5S-33W_12,pws_5S-31W_7,pws_5S-31W_8,pws_5S-31W_9,pws_5S-31W_10,pws_5S-31W_11,pws_5S-31W_12
1973,0.0224398,0.464017,0.255056,0.118829,0.295453,0.0965351,0.0966187,0.422139,0.220526,0.110351,0.274923,0.0744636,0.208042,0.352145,0.211461,0.121744,0.264546,0.0793741,0.317959,0.257524,0.224403,0.168474,0.241503,0.12836,0.38038,0.187353,0.213136,0.216227,0.197015,0.176837,2.45001e-05,0.53868,0.306233,0.17314,0.32105,0.129488,0.134595,0.516275,0.284643,0.174259,0.306605,0.117097,0.242417,0.410996,0.243928,0.157192,0.253881,0.0945066,0.337618,0.281114,0.226255,0.170673,0.200319,0.101465,0.407477,0.203724,0.217454,0.199671,0.174644,0.138296,0.241044,0.547487,0.3215,0.246948,0.323021,0.206449,0.286556,0.440998,0.267654,0.18364,0.232792,0.123928,0.355107,0.296594,0.247162,0.156963,0.173325,0.0802037,0.406535,0.218322,0.243832,0.160022,0.16952,0.0917054
1974,0.214425,0.184744,0.291587,0.252561,0.11887,0.0858352,0.199068,0.186093,0.310892,0.25552,0.153762,0.0711204,0.214127,0.207882,0.354684,0.287728,0.224733,0.0624478,0.239066,0.241878,0.396878,0.315379,0.304717,0.0587978,0.216952,0.27196,0.403843,0.280538,0.344195,0.0488836,0.263231,0.189977,0.285506,0.287788,0.124472,0.123798,0.276331,0.216201,0.303695,0.31565,0.160464,0.132592,0.277522,0.199401,0.308176,0.308502,0.178689,0.10543,0.283439,0.196918,0.330528,0.292771,0.193889,0.0621486,0.271774,0.24176,0.342775,0.249183,0.231077,0.0401731,0.360546,0.275387,0.298307,0.351766,0.210253,0.228933,0.363503,0.223441,0.273955,0.311441,0.139899,0.15262,0.372364,0.209502,0.299015,0.263932,0.106711,0.0857304,0.362713,0.251389,0.334356,0.220657,0.145096,0.0607519
1975,0,0.315576,0.445704,0.314959,0.107894,0.170523,0.0236768,0.301735,0.426968,0.268512,0.110823,0.208626,0.0885069,0.311735,0.399625,0.239913,0.151851,0.250823,0.168719,0.337528,0.349492,0.248892,0.217208,0.285354,0.192117,0.346324,0.280126,0.284806,0.254271,0.301158,0.0295019,0.316459,0.505325,0.343591,0.179168,0.271278,0.0978456,0.337875,0.496491,0.29828,0.186146,0.31682,0.127559,0.323434,0.421245,0.231134,0.180809,0.306316,0.157025,0.323549,0.343582,0.212774,0.204619,0.296006,0.204825,0.33475,0.276962,0.240444,0.246684,0.305832,0.222784,0.377076,0.50648,0.289778,0.265694,0.380135,0.194604,0.353672,0.434569,0.175359,0.197781,0.314676,0.207103,0.347958,0.347742,0.129933,0.186294,0.279408,0.250113,0.37231,0.279734,0.185609,0.22852,0.29145
1976,0.243372,0.242166,0.391078,0.249805,0.143471,0.142855,0.224187,0.270222,0.297306,0.239448,0.132175,0.123116,0.219492,0.325543,0.233157,0.247077,0.151039,0.115626,0.197736,0.393021,0.234527,0.238199,0.182612,0.153666,0.130661,0.435948,0.304954,0.213149,0.198347,0.214769,0.24476,0.189099,0.39693,0.232264,0.139873,0.13573,0.269321,0.240678,0.324548,0.239753,0.150262,0.115668,0.251613,0.251644,0.236308,0.209925,0.145744,0.0769164,0.199342,0.280758,0.196951,0.164783,0.143285,0.0849719,0.148183,0.322959,0.253883,0.134253,0.161238,0.163514,0.318961,0.237145,0.333161,0.236702,0.182671

In [61]:
df_new = pd.concat([coast1_sst_transformed_df, coast1_pws_transformed_df], axis=1)
df_new.head()


,sst_1S-39W_7,sst_1S-39W_8,sst_1S-39W_9,sst_1S-39W_10,sst_1S-39W_11,sst_1S-39W_12,sst_1S-37W_7,sst_1S-37W_8,sst_1S-37W_9,sst_1S-37W_10,sst_1S-37W_11,sst_1S-37W_12,sst_1S-35W_7,sst_1S-35W_8,sst_1S-35W_9,sst_1S-35W_10,sst_1S-35W_11,sst_1S-35W_12,sst_1S-33W_7,sst_1S-33W_8,sst_1S-33W_9,sst_1S-33W_10,sst_1S-33W_11,sst_1S-33W_12,sst_1S-31W_7,sst_1S-31W_8,sst_1S-31W_9,sst_1S-31W_10,sst_1S-31W_11,sst_1S-31W_12,sst_3S-39W_7,sst_3S-39W_8,sst_3S-39W_9,sst_3S-39W_10,sst_3S-39W_11,sst_3S-39W_12,sst_3S-37W_7,sst_3S-37W_8,sst_3S-37W_9,sst_3S-37W_10,sst_3S-37W_11,sst_3S-37W_12,sst_3S-35W_7,sst_3S-35W_8,sst_3S-35W_9,sst_3S-35W_10,sst_3S-35W_11,sst_3S-35W_12,sst_3S-33W_7,sst_3S-33W_8,sst_3S-33W_9,sst_3S-33W_10,sst_3S-33W_11,sst_3S-33W_12,sst_3S-31W_7,sst_3S-31W_8,sst_3S-31W_9,sst_3S-31W_10,sst_3S-31W_11,sst_3S-31W_12,sst_5S-37W_7,sst_5S-37W_8,sst_5S-37W_9,sst_5S-37W_10,sst_5S-37W_11,sst_5S-37W_12,sst_5S-35W_7,sst_5S-35W_8,sst_5S-35W_9,sst_5S-35W_10,sst_5S-35W_11,sst_5S-35W_12,sst_5S-33W_7,sst_5S-33W_8,sst_5S-33W_9,sst_5S-33W_10,sst_5S-33W_11,sst_5S-33W_12,sst_5S-31W_7,sst_5S-31W_8,sst_5S-31W_9,sst_5S-31W_10,sst_5S-31W_11,sst_5S-31W_12,pws_1S-39W_7,pws_1S-39W_8,pws_1S-39W_9,pws_1S-39W_10,pws_1S-39W_11,pws_1S-39W_12,pws_1S-37W_7,pws_1S-37W_8,pws_1S-37W_9,pws_1S-37W_10,pws_1S-37W_11,pws_1S-37W_12,pws_1S-35W_7,pws_1S-35W_8,pws_1S-35W_9,pws_1S-35W_10,pws_1S-35W_11,pws_1S-35W_12,pws_1S-33W_7,pws_1S-33W_8,pws_1S-33W_9,pws_1S-33W_10,pws_1S-33W_11,pws_1S-33W_12,pws_1S-31W_7,pws_1S-31W_8,pws_1S-31W_9,pws_1S-31W_10,pws_1S-31W_11,pws_1S-31W_12,pws_3S-39W_7,pws_3S-39W_8,pws_3S-39W_9,pws_3S-39W_10,pws_3S-39W_11,pws_3S-39W_12,pws_3S-37W_7,pws_3S-37W_8,pws_3S-37W_9,pws_3S-37W_10,pws_3S-37W_11,pws_3S-37W_12,pws_3S-35W_7,pws_3S-35W_8,pws_3S-35W_9,pws_3S-35W_10,pws_3S-35W_11,pws_3S-35W_12,pws_3S-33W_7,pws_3S-33W_8,pws_3S-33W_9,pws_3S-33W_10,pws_3S-33W_11,pws_3S-33W_12,pws_3S-31W_7,pws_3S-31W_8,pws_3S-31W_9,pws_3S-31W_10,pws_3S-31W_11,pws_3S-31W_12,pws_5S-37W_7,pws_5S-37W_8,pws_5S-37W_9,pws_5S-37W_10,pws_5S-37W_11,pws_5S-37W_12,pws_5S-35W_7,pws_5S-35W_8,pws_5S-35W_9,pws_5S-35W_10,pws_5S-35W_11,pws_5S-35W_12,pws_5S-33W_7,pws_5S-33W_8,pws_5S-33W_9,pws_5S-33W_10,pws_5S-33W_11,pws_5S-33W_12,pws_5S-31W_7,pws_5S-31W_8,pws_5S-31W_9,pws_5S-31W_10,pws_5S-31W_11,pws_5S-31W_12
1973,0.992933,0.985866,0.985866,0.992933,0.992933,0.978799,0.985915,0.978873,0.982394,0.989437,0.989437,0.982394,0.967972,0.97153,0.97153,0.982206,0.975089,0.975089,0.953405,0.967742,0.964158,0.97491,0.960573,0.964158,0.94964,0.967626,0.956835,0.971223,0.946043,0.956835,0.996466,0.985866,0.968198,0.992933,0.996466,0.971731,0.996454,0.98227,0.968085,0.989362,0.992908,0.98227,0.989247,0.982079,0.967742,0.989247,0.989247,0.982079,0.967742,0.971326,0.960573,0.97491,0.971326,0.971326,0.967509,0.971119,0.967509,0.971119,0.960289,0.963899,0.992958,0.971831,0.943662,0.975352,0.978873,0.975352,0.985816,0.975177,0.950355,0.975177,0.978723,0.975177,0.985663,0.97491,0.960573,0.982079,0.982079,0.978495,0.978417,0.971223,0.967626,0.97482,0.978417,0.971223,0.0224398,0.464017,0.255056,0.118829,0.295453,0.0965351,0.0966187,0.422139,0.220526,0.110351,0.274923,0.0744636,0.208042,0.352145,0.211461,0.121744,0.264546,0.0793741,0.317959,0.257524,0.224403,0.168474,0.241503,0.12836,0.38038,0.187353,0.213136,0.216227,0.197015,0.176837,2.45001e-05,0.53868,0.306233,0.17314,0.32105,0.129488,0.134595,0.516275,0.284643,0.174259,0.306605,0.117097,0.242417,0.410996,0.243928,0.157192,0.253881,0.0945066,0.337618,0.281114,0.226255,0.170673,0.200319,0.101465,0.407477,0.203724,0.217454,0.199671,0.174644,0.138296,0.241044,0.547487,0.3215,0.246948,0.323021,0.206449,0.286556,0.440998,0.267654,0.18364,0.232792,0.123928,0.355107,0.296594,0.247162,0.156963,0.173325,0.0802037,0.406535,0.218322,0.243832,0.160022,0.16952,0.0917054
1974,0.982332,0.936396,0.961131,0.975265,0.975265,0.961131,0.975352,0.93662,0.950704,0.961268,0.96831,0.957746,0.97153,0.932384,0.953737,0.953737,0.960854,0.950178,0.97491,0.935484,0.960573,0.946237,0.960573,0.946237,0.97

In [62]:
dataset_df = pd.concat([df_new, rainy_classified_df], axis=1)
dataset_df.head()

,sst_1S-39W_7,sst_1S-39W_8,sst_1S-39W_9,sst_1S-39W_10,sst_1S-39W_11,sst_1S-39W_12,sst_1S-37W_7,sst_1S-37W_8,sst_1S-37W_9,sst_1S-37W_10,sst_1S-37W_11,sst_1S-37W_12,sst_1S-35W_7,sst_1S-35W_8,sst_1S-35W_9,sst_1S-35W_10,sst_1S-35W_11,sst_1S-35W_12,sst_1S-33W_7,sst_1S-33W_8,sst_1S-33W_9,sst_1S-33W_10,sst_1S-33W_11,sst_1S-33W_12,sst_1S-31W_7,sst_1S-31W_8,sst_1S-31W_9,sst_1S-31W_10,sst_1S-31W_11,sst_1S-31W_12,sst_3S-39W_7,sst_3S-39W_8,sst_3S-39W_9,sst_3S-39W_10,sst_3S-39W_11,sst_3S-39W_12,sst_3S-37W_7,sst_3S-37W_8,sst_3S-37W_9,sst_3S-37W_10,sst_3S-37W_11,sst_3S-37W_12,sst_3S-35W_7,sst_3S-35W_8,sst_3S-35W_9,sst_3S-35W_10,sst_3S-35W_11,sst_3S-35W_12,sst_3S-33W_7,sst_3S-33W_8,sst_3S-33W_9,sst_3S-33W_10,sst_3S-33W_11,sst_3S-33W_12,sst_3S-31W_7,sst_3S-31W_8,sst_3S-31W_9,sst_3S-31W_10,sst_3S-31W_11,sst_3S-31W_12,sst_5S-37W_7,sst_5S-37W_8,sst_5S-37W_9,sst_5S-37W_10,sst_5S-37W_11,sst_5S-37W_12,sst_5S-35W_7,sst_5S-35W_8,sst_5S-35W_9,sst_5S-35W_10,sst_5S-35W_11,sst_5S-35W_12,sst_5S-33W_7,sst_5S-33W_8,sst_5S-33W_9,sst_5S-33W_10,sst_5S-33W_11,sst_5S-33W_12,sst_5S-31W_7,sst_5S-31W_8,sst_5S-31W_9,sst_5S-31W_10,sst_5S-31W_11,sst_5S-31W_12,pws_1S-39W_7,pws_1S-39W_8,pws_1S-39W_9,pws_1S-39W_10,pws_1S-39W_11,pws_1S-39W_12,pws_1S-37W_7,pws_1S-37W_8,pws_1S-37W_9,pws_1S-37W_10,pws_1S-37W_11,pws_1S-37W_12,pws_1S-35W_7,pws_1S-35W_8,pws_1S-35W_9,pws_1S-35W_10,pws_1S-35W_11,pws_1S-35W_12,pws_1S-33W_7,pws_1S-33W_8,pws_1S-33W_9,pws_1S-33W_10,pws_1S-33W_11,pws_1S-33W_12,pws_1S-31W_7,pws_1S-31W_8,pws_1S-31W_9,pws_1S-31W_10,pws_1S-31W_11,pws_1S-31W_12,pws_3S-39W_7,pws_3S-39W_8,pws_3S-39W_9,pws_3S-39W_10,pws_3S-39W_11,pws_3S-39W_12,pws_3S-37W_7,pws_3S-37W_8,pws_3S-37W_9,pws_3S-37W_10,pws_3S-37W_11,pws_3S-37W_12,pws_3S-35W_7,pws_3S-35W_8,pws_3S-35W_9,pws_3S-35W_10,pws_3S-35W_11,pws_3S-35W_12,pws_3S-33W_7,pws_3S-33W_8,pws_3S-33W_9,pws_3S-33W_10,pws_3S-33W_11,pws_3S-33W_12,pws_3S-31W_7,pws_3S-31W_8,pws_3S-31W_9,pws_3S-31W_10,pws_3S-31W_11,pws_3S-31W_12,pws_5S-37W_7,pws_5S-37W_8,pws_5S-37W_9,pws_5S-37W_10,pws_5S-37W_11,pws_5S-37W_12,pws_5S-35W_7,pws_5S-35W_8,pws_5S-35W_9,pws_5S-35W_10,pws_5S-35W_11,pws_5S-35W_12,pws_5S-33W_7,pws_5S-33W_8,pws_5S-33W_9,pws_5S-33W_10,pws_5S-33W_11,pws_5S-33W_12,pws_5S-31W_7,pws_5S-31W_8,pws_5S-31W_9,pws_5S-31W_10,pws_5S-31W_11,pws_5S-31W_12,classes
1973,0.992933,0.985866,0.985866,0.992933,0.992933,0.978799,0.985915,0.978873,0.982394,0.989437,0.989437,0.982394,0.967972,0.97153,0.97153,0.982206,0.975089,0.975089,0.953405,0.967742,0.964158,0.97491,0.960573,0.964158,0.94964,0.967626,0.956835,0.971223,0.946043,0.956835,0.996466,0.985866,0.968198,0.992933,0.996466,0.971731,0.996454,0.98227,0.968085,0.989362,0.992908,0.98227,0.989247,0.982079,0.967742,0.989247,0.989247,0.982079,0.967742,0.971326,0.960573,0.97491,0.971326,0.971326,0.967509,0.971119,0.967509,0.971119,0.960289,0.963899,0.992958,0.971831,0.943662,0.975352,0.978873,0.975352,0.985816,0.975177,0.950355,0.975177,0.978723,0.975177,0.985663,0.97491,0.960573,0.982079,0.982079,0.978495,0.978417,0.971223,0.967626,0.97482,0.978417,0.971223,0.0224398,0.464017,0.255056,0.118829,0.295453,0.0965351,0.0966187,0.422139,0.220526,0.110351,0.274923,0.0744636,0.208042,0.352145,0.211461,0.121744,0.264546,0.0793741,0.317959,0.257524,0.224403,0.168474,0.241503,0.12836,0.38038,0.187353,0.213136,0.216227,0.197015,0.176837,2.45001e-05,0.53868,0.306233,0.17314,0.32105,0.129488,0.134595,0.516275,0.284643,0.174259,0.306605,0.117097,0.242417,0.410996,0.243928,0.157192,0.253881,0.0945066,0.337618,0.281114,0.226255,0.170673,0.200319,0.101465,0.407477,0.203724,0.217454,0.199671,0.174644,0.138296,0.241044,0.547487,0.3215,0.246948,0.323021,0.206449,0.286556,0.440998,0.267654,0.18364,0.232792,0.123928,0.355107,0.296594,0.247162,0.156963,0.173325,0.0802037,0.406535,0.218322,0.243832,0.160022,0.16952,0.0917054,strong
1974,0.982332,0.936396,0.961131,0.975265,0.975265,0.961131,0.975352,0.93662,0.950704,0.961268,0.96831,0.957746,0.97153,0.932384,0.953737,0.953737,0.960854,0.950178,0.97491,0.935484,0.960573,0.946237,0.96057

In [63]:
#Removing 2017 due we don't have full 2018 available
dataset_df = dataset_df.loc['1973-01-01':'2016-12-01']

In [64]:
#Train
train_df = dataset_df.loc['1973-01-01':'2000-12-01']

test_df = dataset_df.loc['2000-01-01':'2016-12-01']

#Dropt last column to be the X values
feature_cols = dataset_df.columns
feature_cols=feature_cols.drop('classes')

# Input data(X)
X = train_df.loc[:, feature_cols]

# Output data(y)
y = train_df['classes']

X_test = test_df.loc[:, feature_cols]


### Testing classifiers for coast1

In [65]:
new_pred_class = test_knn(X,y)

2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 weak
2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 weak
2001 weak
2001 normal


In [66]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.25

In [67]:
new_pred_class = test_svc(X,y)

2001 weak
2002 weak
2003 weak
2004 weak
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 weak
2011 weak
2012 weak
2013 weak
2014 weak
2015 weak
2016 weak


In [68]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.8125

In [69]:
new_pred_class = test_gpc(X,y)

2001 normal
2002 weak
2003 normal
2004 normal
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 normal
2011 weak
2012 weak
2013 weak
2014 weak
2015 weak
2016 weak


In [70]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.5625

In [71]:
new_pred_class = test_dtc(X,y)

2001 weak
2002 normal
2003 weak
2004 normal
2005 weak
2006 weak
2007 weak
2008 normal
2009 weak
2010 weak
2011 weak
2012 weak
2013 strong
2014 strong
2015 normal
2016 normal


In [72]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.625

In [73]:
new_pred_class = test_rfc(X,y)

2001 normal
2002 normal
2003 normal
2004 normal
2005 normal
2006 weak
2007 normal
2008 normal
2009 weak
2010 normal
2011 normal
2012 normal
2013 normal
2014 normal
2015 normal
2016 normal


In [74]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.3125

In [75]:
new_pred_class = test_mlp(X,y)

2001 weak
2002 weak
2003 weak
2004 weak
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 weak
2011 weak
2012 weak
2013 weak
2014 normal
2015 normal
2016 normal


In [76]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.625

In [77]:
new_pred_class = test_adc(X,y)

2001 weak
2002 weak
2003 normal
2004 normal
2005 normal
2006 weak
2007 weak
2008 normal
2009 normal
2010 normal
2011 normal
2012 normal
2013 weak
2014 weak
2015 normal
2016 normal


In [78]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.3125

In [79]:
new_pred_class = test_gnb(X,y)

2001 normal
2002 normal
2003 normal
2004 normal
2005 normal
2006 normal
2007 normal
2008 normal
2009 normal
2010 normal
2011 normal
2012 normal
2013 normal
2014 weak
2015 weak
2016 weak


In [80]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.375

### Prepare data for coast2

In [81]:
coast2_sst_transformed_df = create_transformed_df(coast2_sst_scaled_df, "sst_")
coast2_sst_transformed_df.head()

,sst_1S-47W_7,sst_1S-47W_8,sst_1S-47W_9,sst_1S-47W_10,sst_1S-47W_11,sst_1S-47W_12,sst_1S-45W_7,sst_1S-45W_8,sst_1S-45W_9,sst_1S-45W_10,sst_1S-45W_11,sst_1S-45W_12,sst_1S-43W_7,sst_1S-43W_8,sst_1S-43W_9,sst_1S-43W_10,sst_1S-43W_11,sst_1S-43W_12,sst_1S-41W_7,sst_1S-41W_8,sst_1S-41W_9,sst_1S-41W_10,sst_1S-41W_11,sst_1S-41W_12,sst_1S-39W_7,sst_1S-39W_8,sst_1S-39W_9,sst_1S-39W_10,sst_1S-39W_11,sst_1S-39W_12,sst_1S-37W_7,sst_1S-37W_8,sst_1S-37W_9,sst_1S-37W_10,sst_1S-37W_11,sst_1S-37W_12,sst_1S-35W_7,sst_1S-35W_8,sst_1S-35W_9,sst_1S-35W_10,sst_1S-35W_11,sst_1S-35W_12,sst_1S-33W_7,sst_1S-33W_8,sst_1S-33W_9,sst_1S-33W_10,sst_1S-33W_11,sst_1S-33W_12,sst_1S-31W_7,sst_1S-31W_8,sst_1S-31W_9,sst_1S-31W_10,sst_1S-31W_11,sst_1S-31W_12,sst_1S-29W_7,sst_1S-29W_8,sst_1S-29W_9,sst_1S-29W_10,sst_1S-29W_11,sst_1S-29W_12,sst_3S-43W_7,sst_3S-43W_8,sst_3S-43W_9,sst_3S-43W_10,sst_3S-43W_11,sst_3S-43W_12,sst_3S-41W_7,sst_3S-41W_8,sst_3S-41W_9,sst_3S-41W_10,sst_3S-41W_11,sst_3S-41W_12,sst_3S-39W_7,sst_3S-39W_8,sst_3S-39W_9,sst_3S-39W_10,sst_3S-39W_11,sst_3S-39W_12,sst_3S-37W_7,sst_3S-37W_8,sst_3S-37W_9,sst_3S-37W_10,sst_3S-37W_11,sst_3S-37W_12,sst_3S-35W_7,sst_3S-35W_8,sst_3S-35W_9,sst_3S-35W_10,sst_3S-35W_11,sst_3S-35W_12,sst_3S-33W_7,sst_3S-33W_8,sst_3S-33W_9,sst_3S-33W_10,sst_3S-33W_11,sst_3S-33W_12,sst_3S-31W_7,sst_3S-31W_8,sst_3S-31W_9,sst_3S-31W_10,sst_3S-31W_11,sst_3S-31W_12,sst_3S-29W_7,sst_3S-29W_8,sst_3S-29W_9,sst_3S-29W_10,sst_3S-29W_11,sst_3S-29W_12,sst_5S-37W_7,sst_5S-37W_8,sst_5S-37W_9,sst_5S-37W_10,sst_5S-37W_11,sst_5S-37W_12,sst_5S-35W_7,sst_5S-35W_8,sst_5S-35W_9,sst_5S-35W_10,sst_5S-35W_11,sst_5S-35W_12,sst_5S-33W_7,sst_5S-33W_8,sst_5S-33W_9,sst_5S-33W_10,sst_5S-33W_11,sst_5S-33W_12,sst_5S-31W_7,sst_5S-31W_8,sst_5S-31W_9,sst_5S-31W_10,sst_5S-31W_11,sst_5S-31W_12,sst_5S-29W_7,sst_5S-29W_8,sst_5S-29W_9,sst_5S-29W_10,sst_5S-29W_11,sst_5S-29W_12,sst_7S-35W_7,sst_7S-35W_8,sst_7S-35W_9,sst_7S-35W_10,sst_7S-35W_11,sst_7S-35W_12,sst_7S-33W_7,sst_7S-33W_8,sst_7S-33W_9,sst_7S-33W_10,sst_7S-33W_11,sst_7S-33W_12,sst_7S-31W_7,sst_7S-31W_8,sst_7S-31W_9,sst_7S-31W_10,sst_7S-31W_11,sst_7S-31W_12,sst_7S-29W_7,sst_7S-29W_8,sst_7S-29W_9,sst_7S-29W_10,sst_7S-29W_11,sst_7S-29W_12,sst_9S-35W_7,sst_9S-35W_8,sst_9S-35W_9,sst_9S-35W_10,sst_9S-35W_11,sst_9S-35W_12,sst_9S-33W_7,sst_9S-33W_8,sst_9S-33W_9,sst_9S-33W_10,sst_9S-33W_11,sst_9S-33W_12,sst_9S-31W_7,sst_9S-31W_8,sst_9S-31W_9,sst_9S-31W_10,sst_9S-31W_11,sst_9S-31W_12,sst_9S-29W_7,sst_9S-29W_8,sst_9S-29W_9,sst_9S-29W_10,sst_9S-29W_11,sst_9S-29W_12
1973,0.989691,0.948454,0.996564,0.958763,0.982818,0.931271,0.986063,0.972125,0.979094,0.979094,0.979094,0.944251,0.989437,0.989437,0.971831,0.992958,0.982394,0.957746,0.989437,0.989437,0.975352,0.996479,0.985915,0.96831,0.992933,0.985866,0.985866,0.992933,0.992933,0.978799,0.985915,0.978873,0.982394,0.989437,0.989437,0.982394,0.967972,0.97153,0.97153,0.982206,0.975089,0.975089,0.953405,0.967742,0.964158,0.97491,0.960573,0.964158,0.94964,0.967626,0.956835,0.971223,0.946043,0.956835,0.960714,0.967857,0.960714,0.971429,0.942857,0.95,0.969072,0.975945,0.938144,0.979381,0.958763,0.927835,0.992958,0.989437,0.961268,0.996479,0.985915,0.957746,0.996466,0.985866,0.968198,0.992933,0.996466,0.971731,0.996454,0.98227,0.968085,0.989362,0.992908,0.98227,0.989247,0.982079,0.967742,0.989247,0.989247,0.982079,0.967742,0.971326,0.960573,0.97491,0.971326,0.971326,0.967509,0.971119,0.967509,0.971119,0.960289,0.963899,0.978495,0.97491,0.978495,0.97491,0.960573,0.964158,0.992958,0.971831,0.943662,0.975352,0.978873,0.975352,0.985816,0.975177,0.950355,0.975177,0.978723,0.975177,0.985663,0.97491,0.960573,0.982079,0.982079,0.978495,0.978417,0.971223,0.967626,0.97482,0.978417,0.971223,0.978495,0.971326,0.97491,0.971326,0.971326,0.967742,0.98227,0.971631,0.950355,0.975177,0.975177,0.98227,0.985663,0.97491,0.960573,0.985663,0.985663,0.982079,0.982014,0.971223,0.964029,0.982014,0.985612,0.978417,0.971429,0.964286,0.964286,0.971429,0.975,0.971429,0.961404,0.961404,0.947368,0.964912,0.97193,0.98

In [82]:
coast2_pws_transformed_df = create_transformed_df(coast2_pws_scaled_df, "pws_")
coast2_pws_transformed_df.head()

,pws_1S-47W_7,pws_1S-47W_8,pws_1S-47W_9,pws_1S-47W_10,pws_1S-47W_11,pws_1S-47W_12,pws_1S-45W_7,pws_1S-45W_8,pws_1S-45W_9,pws_1S-45W_10,pws_1S-45W_11,pws_1S-45W_12,pws_1S-43W_7,pws_1S-43W_8,pws_1S-43W_9,pws_1S-43W_10,pws_1S-43W_11,pws_1S-43W_12,pws_1S-41W_7,pws_1S-41W_8,pws_1S-41W_9,pws_1S-41W_10,pws_1S-41W_11,pws_1S-41W_12,pws_1S-39W_7,pws_1S-39W_8,pws_1S-39W_9,pws_1S-39W_10,pws_1S-39W_11,pws_1S-39W_12,pws_1S-37W_7,pws_1S-37W_8,pws_1S-37W_9,pws_1S-37W_10,pws_1S-37W_11,pws_1S-37W_12,pws_1S-35W_7,pws_1S-35W_8,pws_1S-35W_9,pws_1S-35W_10,pws_1S-35W_11,pws_1S-35W_12,pws_1S-33W_7,pws_1S-33W_8,pws_1S-33W_9,pws_1S-33W_10,pws_1S-33W_11,pws_1S-33W_12,pws_1S-31W_7,pws_1S-31W_8,pws_1S-31W_9,pws_1S-31W_10,pws_1S-31W_11,pws_1S-31W_12,pws_1S-29W_7,pws_1S-29W_8,pws_1S-29W_9,pws_1S-29W_10,pws_1S-29W_11,pws_1S-29W_12,pws_3S-43W_7,pws_3S-43W_8,pws_3S-43W_9,pws_3S-43W_10,pws_3S-43W_11,pws_3S-43W_12,pws_3S-41W_7,pws_3S-41W_8,pws_3S-41W_9,pws_3S-41W_10,pws_3S-41W_11,pws_3S-41W_12,pws_3S-39W_7,pws_3S-39W_8,pws_3S-39W_9,pws_3S-39W_10,pws_3S-39W_11,pws_3S-39W_12,pws_3S-37W_7,pws_3S-37W_8,pws_3S-37W_9,pws_3S-37W_10,pws_3S-37W_11,pws_3S-37W_12,pws_3S-35W_7,pws_3S-35W_8,pws_3S-35W_9,pws_3S-35W_10,pws_3S-35W_11,pws_3S-35W_12,pws_3S-33W_7,pws_3S-33W_8,pws_3S-33W_9,pws_3S-33W_10,pws_3S-33W_11,pws_3S-33W_12,pws_3S-31W_7,pws_3S-31W_8,pws_3S-31W_9,pws_3S-31W_10,pws_3S-31W_11,pws_3S-31W_12,pws_3S-29W_7,pws_3S-29W_8,pws_3S-29W_9,pws_3S-29W_10,pws_3S-29W_11,pws_3S-29W_12,pws_5S-37W_7,pws_5S-37W_8,pws_5S-37W_9,pws_5S-37W_10,pws_5S-37W_11,pws_5S-37W_12,pws_5S-35W_7,pws_5S-35W_8,pws_5S-35W_9,pws_5S-35W_10,pws_5S-35W_11,pws_5S-35W_12,pws_5S-33W_7,pws_5S-33W_8,pws_5S-33W_9,pws_5S-33W_10,pws_5S-33W_11,pws_5S-33W_12,pws_5S-31W_7,pws_5S-31W_8,pws_5S-31W_9,pws_5S-31W_10,pws_5S-31W_11,pws_5S-31W_12,pws_5S-29W_7,pws_5S-29W_8,pws_5S-29W_9,pws_5S-29W_10,pws_5S-29W_11,pws_5S-29W_12,pws_7S-35W_7,pws_7S-35W_8,pws_7S-35W_9,pws_7S-35W_10,pws_7S-35W_11,pws_7S-35W_12,pws_7S-33W_7,pws_7S-33W_8,pws_7S-33W_9,pws_7S-33W_10,pws_7S-33W_11,pws_7S-33W_12,pws_7S-31W_7,pws_7S-31W_8,pws_7S-31W_9,pws_7S-31W_10,pws_7S-31W_11,pws_7S-31W_12,pws_7S-29W_7,pws_7S-29W_8,pws_7S-29W_9,pws_7S-29W_10,pws_7S-29W_11,pws_7S-29W_12,pws_9S-35W_7,pws_9S-35W_8,pws_9S-35W_9,pws_9S-35W_10,pws_9S-35W_11,pws_9S-35W_12,pws_9S-33W_7,pws_9S-33W_8,pws_9S-33W_9,pws_9S-33W_10,pws_9S-33W_11,pws_9S-33W_12,pws_9S-31W_7,pws_9S-31W_8,pws_9S-31W_9,pws_9S-31W_10,pws_9S-31W_11,pws_9S-31W_12,pws_9S-29W_7,pws_9S-29W_8,pws_9S-29W_9,pws_9S-29W_10,pws_9S-29W_11,pws_9S-29W_12
1973,0.056909,0.331639,0.301436,0.256152,0.211644,0.263477,0,0.310556,0.311105,0.224057,0.224736,0.235968,0,0.383755,0.352666,0.222957,0.300526,0.226192,0,0.434736,0.312767,0.167475,0.31461,0.162534,0.0224398,0.464017,0.255056,0.118829,0.295453,0.0965351,0.0966187,0.422139,0.220526,0.110351,0.274923,0.0744636,0.208042,0.352145,0.211461,0.121744,0.264546,0.0793741,0.317959,0.257524,0.224403,0.168474,0.241503,0.12836,0.38038,0.187353,0.213136,0.216227,0.197015,0.176837,0.322986,0.198672,0.167726,0.205673,0.158418,0.158693,0,0.394438,0.380104,0.286742,0.309543,0.2406,0,0.471325,0.355072,0.239573,0.340513,0.203006,2.45001e-05,0.53868,0.306233,0.17314,0.32105,0.129488,0.134595,0.516275,0.284643,0.174259,0.306605,0.117097,0.242417,0.410996,0.243928,0.157192,0.253881,0.0945066,0.337618,0.281114,0.226255,0.170673,0.200319,0.101465,0.407477,0.203724,0.217454,0.199671,0.174644,0.138296,0.348073,0.175584,0.174668,0.168721,0.140841,0.110692,0.241044,0.547487,0.3215,0.246948,0.323021,0.206449,0.286556,0.440998,0.267654,0.18364,0.232792,0.123928,0.355107,0.296594,0.247162,0.156963,0.173325,0.0802037,0.406535,0.218322,0.243832,0.160022,0.16952,0.0917054,0.387341,0.218202,0.232532,0.157402,0.167865,0.0916968,0.265023,0.357409,0.252208,0.150019,0.146748,0.136261,0.349852,0.288504,0.301041,0.148938,0.146268,0.111763,0.382949,0.263518,0.333235,0.156363,0.159762,0.0968973,0.354248,0.277372,0.316564,0.149414,0.173218,0.0801183,0.230437,0.214839,0.230661,0.137501,0.0730146,0.164222,0.2

In [83]:
df_new = pd.concat([coast2_sst_transformed_df, coast2_pws_transformed_df], axis=1)
df_new.head()


,sst_1S-47W_7,sst_1S-47W_8,sst_1S-47W_9,sst_1S-47W_10,sst_1S-47W_11,sst_1S-47W_12,sst_1S-45W_7,sst_1S-45W_8,sst_1S-45W_9,sst_1S-45W_10,sst_1S-45W_11,sst_1S-45W_12,sst_1S-43W_7,sst_1S-43W_8,sst_1S-43W_9,sst_1S-43W_10,sst_1S-43W_11,sst_1S-43W_12,sst_1S-41W_7,sst_1S-41W_8,sst_1S-41W_9,sst_1S-41W_10,sst_1S-41W_11,sst_1S-41W_12,sst_1S-39W_7,sst_1S-39W_8,sst_1S-39W_9,sst_1S-39W_10,sst_1S-39W_11,sst_1S-39W_12,sst_1S-37W_7,sst_1S-37W_8,sst_1S-37W_9,sst_1S-37W_10,sst_1S-37W_11,sst_1S-37W_12,sst_1S-35W_7,sst_1S-35W_8,sst_1S-35W_9,sst_1S-35W_10,sst_1S-35W_11,sst_1S-35W_12,sst_1S-33W_7,sst_1S-33W_8,sst_1S-33W_9,sst_1S-33W_10,sst_1S-33W_11,sst_1S-33W_12,sst_1S-31W_7,sst_1S-31W_8,sst_1S-31W_9,sst_1S-31W_10,sst_1S-31W_11,sst_1S-31W_12,sst_1S-29W_7,sst_1S-29W_8,sst_1S-29W_9,sst_1S-29W_10,sst_1S-29W_11,sst_1S-29W_12,sst_3S-43W_7,sst_3S-43W_8,sst_3S-43W_9,sst_3S-43W_10,sst_3S-43W_11,sst_3S-43W_12,sst_3S-41W_7,sst_3S-41W_8,sst_3S-41W_9,sst_3S-41W_10,sst_3S-41W_11,sst_3S-41W_12,sst_3S-39W_7,sst_3S-39W_8,sst_3S-39W_9,sst_3S-39W_10,sst_3S-39W_11,sst_3S-39W_12,sst_3S-37W_7,sst_3S-37W_8,sst_3S-37W_9,sst_3S-37W_10,sst_3S-37W_11,sst_3S-37W_12,sst_3S-35W_7,sst_3S-35W_8,sst_3S-35W_9,sst_3S-35W_10,sst_3S-35W_11,sst_3S-35W_12,sst_3S-33W_7,sst_3S-33W_8,sst_3S-33W_9,sst_3S-33W_10,sst_3S-33W_11,sst_3S-33W_12,sst_3S-31W_7,sst_3S-31W_8,sst_3S-31W_9,sst_3S-31W_10,sst_3S-31W_11,sst_3S-31W_12,sst_3S-29W_7,sst_3S-29W_8,sst_3S-29W_9,sst_3S-29W_10,sst_3S-29W_11,sst_3S-29W_12,sst_5S-37W_7,sst_5S-37W_8,sst_5S-37W_9,sst_5S-37W_10,sst_5S-37W_11,sst_5S-37W_12,sst_5S-35W_7,sst_5S-35W_8,sst_5S-35W_9,sst_5S-35W_10,sst_5S-35W_11,sst_5S-35W_12,sst_5S-33W_7,sst_5S-33W_8,sst_5S-33W_9,sst_5S-33W_10,sst_5S-33W_11,sst_5S-33W_12,sst_5S-31W_7,sst_5S-31W_8,sst_5S-31W_9,sst_5S-31W_10,sst_5S-31W_11,sst_5S-31W_12,sst_5S-29W_7,sst_5S-29W_8,sst_5S-29W_9,sst_5S-29W_10,sst_5S-29W_11,sst_5S-29W_12,sst_7S-35W_7,sst_7S-35W_8,sst_7S-35W_9,sst_7S-35W_10,sst_7S-35W_11,sst_7S-35W_12,sst_7S-33W_7,sst_7S-33W_8,sst_7S-33W_9,sst_7S-33W_10,sst_7S-33W_11,sst_7S-33W_12,sst_7S-31W_7,sst_7S-31W_8,sst_7S-31W_9,sst_7S-31W_10,sst_7S-31W_11,sst_7S-31W_12,sst_7S-29W_7,sst_7S-29W_8,sst_7S-29W_9,sst_7S-29W_10,sst_7S-29W_11,sst_7S-29W_12,sst_9S-35W_7,sst_9S-35W_8,sst_9S-35W_9,sst_9S-35W_10,sst_9S-35W_11,sst_9S-35W_12,sst_9S-33W_7,sst_9S-33W_8,sst_9S-33W_9,sst_9S-33W_10,sst_9S-33W_11,sst_9S-33W_12,sst_9S-31W_7,sst_9S-31W_8,sst_9S-31W_9,sst_9S-31W_10,sst_9S-31W_11,sst_9S-31W_12,sst_9S-29W_7,sst_9S-29W_8,sst_9S-29W_9,sst_9S-29W_10,sst_9S-29W_11,sst_9S-29W_12,pws_1S-47W_7,pws_1S-47W_8,pws_1S-47W_9,pws_1S-47W_10,pws_1S-47W_11,pws_1S-47W_12,pws_1S-45W_7,pws_1S-45W_8,pws_1S-45W_9,pws_1S-45W_10,pws_1S-45W_11,pws_1S-45W_12,pws_1S-43W_7,pws_1S-43W_8,pws_1S-43W_9,pws_1S-43W_10,pws_1S-43W_11,pws_1S-43W_12,pws_1S-41W_7,pws_1S-41W_8,pws_1S-41W_9,pws_1S-41W_10,pws_1S-41W_11,pws_1S-41W_12,pws_1S-39W_7,pws_1S-39W_8,pws_1S-39W_9,pws_1S-39W_10,pws_1S-39W_11,pws_1S-39W_12,pws_1S-37W_7,pws_1S-37W_8,pws_1S-37W_9,pws_1S-37W_10,pws_1S-37W_11,pws_1S-37W_12,pws_1S-35W_7,pws_1S-35W_8,pws_1S-35W_9,pws_1S-35W_10,pws_1S-35W_11,pws_1S-35W_12,pws_1S-33W_7,pws_1S-33W_8,pws_1S-33W_9,pws_1S-33W_10,pws_1S-33W_11,pws_1S-33W_12,pws_1S-31W_7,pws_1S-31W_8,pws_1S-31W_9,pws_1S-31W_10,pws_1S-31W_11,pws_1S-31W_12,pws_1S-29W_7,pws_1S-29W_8,pws_1S-29W_9,pws_1S-29W_10,pws_1S-29W_11,pws_1S-29W_12,pws_3S-43W_7,pws_3S-43W_8,pws_3S-43W_9,pws_3S-43W_10,pws_3S-43W_11,pws_3S-43W_12,pws_3S-41W_7,pws_3S-41W_8,pws_3S-41W_9,pws_3S-41W_10,pws_3S-41W_11,pws_3S-41W_12,pws_3S-39W_7,pws_3S-39W_8,pws_3S-39W_9,pws_3S-39W_10,pws_3S-39W_11,pws_3S-39W_12,pws_3S-37W_7,pws_3S-37W_8,pws_3S-37W_9,pws_3S-37W_10,pws_3S-37W_11,pws_3S-37W_12,pws_3S-35W_7,pws_3S-35W_8,pws_3S-35W_9,pws_3S-35W_10,pws_3S-35W_11,pws_3S-35W_12,pws_3S-33W_7,pws_3S-33W_8,pws_3S-33W_9,pws_3S-33W_10,pws_3S-33W_11,pws_3S-33W_12,pws_3S-31W_7,pws_3S-31W_8,pws_3S-31W_9,pws_3S-31W_10,pws_3S-31W_11,pws_3S-31W_12,pws_3S-29W_7,pws_3S-29W_8,pws_3S-29W_9,pws_3S-29W_10,pws_3S-29W_11,pws_3S-29W_12,pws_5S-37W_7,pws_5S-37W_8,pws_

In [84]:
dataset_df = pd.concat([df_new, rainy_classified_df], axis=1)
dataset_df.head()

,sst_1S-47W_7,sst_1S-47W_8,sst_1S-47W_9,sst_1S-47W_10,sst_1S-47W_11,sst_1S-47W_12,sst_1S-45W_7,sst_1S-45W_8,sst_1S-45W_9,sst_1S-45W_10,sst_1S-45W_11,sst_1S-45W_12,sst_1S-43W_7,sst_1S-43W_8,sst_1S-43W_9,sst_1S-43W_10,sst_1S-43W_11,sst_1S-43W_12,sst_1S-41W_7,sst_1S-41W_8,sst_1S-41W_9,sst_1S-41W_10,sst_1S-41W_11,sst_1S-41W_12,sst_1S-39W_7,sst_1S-39W_8,sst_1S-39W_9,sst_1S-39W_10,sst_1S-39W_11,sst_1S-39W_12,sst_1S-37W_7,sst_1S-37W_8,sst_1S-37W_9,sst_1S-37W_10,sst_1S-37W_11,sst_1S-37W_12,sst_1S-35W_7,sst_1S-35W_8,sst_1S-35W_9,sst_1S-35W_10,sst_1S-35W_11,sst_1S-35W_12,sst_1S-33W_7,sst_1S-33W_8,sst_1S-33W_9,sst_1S-33W_10,sst_1S-33W_11,sst_1S-33W_12,sst_1S-31W_7,sst_1S-31W_8,sst_1S-31W_9,sst_1S-31W_10,sst_1S-31W_11,sst_1S-31W_12,sst_1S-29W_7,sst_1S-29W_8,sst_1S-29W_9,sst_1S-29W_10,sst_1S-29W_11,sst_1S-29W_12,sst_3S-43W_7,sst_3S-43W_8,sst_3S-43W_9,sst_3S-43W_10,sst_3S-43W_11,sst_3S-43W_12,sst_3S-41W_7,sst_3S-41W_8,sst_3S-41W_9,sst_3S-41W_10,sst_3S-41W_11,sst_3S-41W_12,sst_3S-39W_7,sst_3S-39W_8,sst_3S-39W_9,sst_3S-39W_10,sst_3S-39W_11,sst_3S-39W_12,sst_3S-37W_7,sst_3S-37W_8,sst_3S-37W_9,sst_3S-37W_10,sst_3S-37W_11,sst_3S-37W_12,sst_3S-35W_7,sst_3S-35W_8,sst_3S-35W_9,sst_3S-35W_10,sst_3S-35W_11,sst_3S-35W_12,sst_3S-33W_7,sst_3S-33W_8,sst_3S-33W_9,sst_3S-33W_10,sst_3S-33W_11,sst_3S-33W_12,sst_3S-31W_7,sst_3S-31W_8,sst_3S-31W_9,sst_3S-31W_10,sst_3S-31W_11,sst_3S-31W_12,sst_3S-29W_7,sst_3S-29W_8,sst_3S-29W_9,sst_3S-29W_10,sst_3S-29W_11,sst_3S-29W_12,sst_5S-37W_7,sst_5S-37W_8,sst_5S-37W_9,sst_5S-37W_10,sst_5S-37W_11,sst_5S-37W_12,sst_5S-35W_7,sst_5S-35W_8,sst_5S-35W_9,sst_5S-35W_10,sst_5S-35W_11,sst_5S-35W_12,sst_5S-33W_7,sst_5S-33W_8,sst_5S-33W_9,sst_5S-33W_10,sst_5S-33W_11,sst_5S-33W_12,sst_5S-31W_7,sst_5S-31W_8,sst_5S-31W_9,sst_5S-31W_10,sst_5S-31W_11,sst_5S-31W_12,sst_5S-29W_7,sst_5S-29W_8,sst_5S-29W_9,sst_5S-29W_10,sst_5S-29W_11,sst_5S-29W_12,sst_7S-35W_7,sst_7S-35W_8,sst_7S-35W_9,sst_7S-35W_10,sst_7S-35W_11,sst_7S-35W_12,sst_7S-33W_7,sst_7S-33W_8,sst_7S-33W_9,sst_7S-33W_10,sst_7S-33W_11,sst_7S-33W_12,sst_7S-31W_7,sst_7S-31W_8,sst_7S-31W_9,sst_7S-31W_10,sst_7S-31W_11,sst_7S-31W_12,sst_7S-29W_7,sst_7S-29W_8,sst_7S-29W_9,sst_7S-29W_10,sst_7S-29W_11,sst_7S-29W_12,sst_9S-35W_7,sst_9S-35W_8,sst_9S-35W_9,sst_9S-35W_10,sst_9S-35W_11,sst_9S-35W_12,sst_9S-33W_7,sst_9S-33W_8,sst_9S-33W_9,sst_9S-33W_10,sst_9S-33W_11,sst_9S-33W_12,sst_9S-31W_7,sst_9S-31W_8,sst_9S-31W_9,sst_9S-31W_10,sst_9S-31W_11,sst_9S-31W_12,sst_9S-29W_7,sst_9S-29W_8,sst_9S-29W_9,sst_9S-29W_10,sst_9S-29W_11,sst_9S-29W_12,pws_1S-47W_7,pws_1S-47W_8,pws_1S-47W_9,pws_1S-47W_10,pws_1S-47W_11,pws_1S-47W_12,pws_1S-45W_7,pws_1S-45W_8,pws_1S-45W_9,pws_1S-45W_10,pws_1S-45W_11,pws_1S-45W_12,pws_1S-43W_7,pws_1S-43W_8,pws_1S-43W_9,pws_1S-43W_10,pws_1S-43W_11,pws_1S-43W_12,pws_1S-41W_7,pws_1S-41W_8,pws_1S-41W_9,pws_1S-41W_10,pws_1S-41W_11,pws_1S-41W_12,pws_1S-39W_7,pws_1S-39W_8,pws_1S-39W_9,pws_1S-39W_10,pws_1S-39W_11,pws_1S-39W_12,pws_1S-37W_7,pws_1S-37W_8,pws_1S-37W_9,pws_1S-37W_10,pws_1S-37W_11,pws_1S-37W_12,pws_1S-35W_7,pws_1S-35W_8,pws_1S-35W_9,pws_1S-35W_10,pws_1S-35W_11,pws_1S-35W_12,pws_1S-33W_7,pws_1S-33W_8,pws_1S-33W_9,pws_1S-33W_10,pws_1S-33W_11,pws_1S-33W_12,pws_1S-31W_7,pws_1S-31W_8,pws_1S-31W_9,pws_1S-31W_10,pws_1S-31W_11,pws_1S-31W_12,pws_1S-29W_7,pws_1S-29W_8,pws_1S-29W_9,pws_1S-29W_10,pws_1S-29W_11,pws_1S-29W_12,pws_3S-43W_7,pws_3S-43W_8,pws_3S-43W_9,pws_3S-43W_10,pws_3S-43W_11,pws_3S-43W_12,pws_3S-41W_7,pws_3S-41W_8,pws_3S-41W_9,pws_3S-41W_10,pws_3S-41W_11,pws_3S-41W_12,pws_3S-39W_7,pws_3S-39W_8,pws_3S-39W_9,pws_3S-39W_10,pws_3S-39W_11,pws_3S-39W_12,pws_3S-37W_7,pws_3S-37W_8,pws_3S-37W_9,pws_3S-37W_10,pws_3S-37W_11,pws_3S-37W_12,pws_3S-35W_7,pws_3S-35W_8,pws_3S-35W_9,pws_3S-35W_10,pws_3S-35W_11,pws_3S-35W_12,pws_3S-33W_7,pws_3S-33W_8,pws_3S-33W_9,pws_3S-33W_10,pws_3S-33W_11,pws_3S-33W_12,pws_3S-31W_7,pws_3S-31W_8,pws_3S-31W_9,pws_3S-31W_10,pws_3S-31W_11,pws_3S-31W_12,pws_3S-29W_7,pws_3S-29W_8,pws_3S-29W_9,pws_3S-29W_10,pws_3S-29W_11,pws_3S-29W_12,pws_5S-37W_7,pws_5S-37W_8,pws_

In [85]:
#Removing 2017 due we don't have full 2018 available
dataset_df = dataset_df.loc['1973-01-01':'2016-12-01']

In [86]:
#Train
train_df = dataset_df.loc['1973-01-01':'2000-12-01']

test_df = dataset_df.loc['2000-01-01':'2016-12-01']

#Dropt last column to be the X values
feature_cols = dataset_df.columns
feature_cols=feature_cols.drop('classes')

# Input data(X)
X = train_df.loc[:, feature_cols]

# Output data(y)
y = train_df['classes']

X_test = test_df.loc[:, feature_cols]


### Testing classifiers for coast2

In [87]:
new_pred_class = test_knn(X,y)

2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 weak
2001 weak
2001 normal


In [88]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.3125

In [89]:
new_pred_class = test_svc(X,y)

2001 weak
2002 weak
2003 weak
2004 weak
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 weak
2011 weak
2012 weak
2013 weak
2014 weak
2015 weak
2016 weak


In [90]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.8125

In [91]:
new_pred_class = test_gpc(X,y)

2001 normal
2002 weak
2003 normal
2004 normal
2005 weak
2006 weak
2007 normal
2008 weak
2009 weak
2010 normal
2011 normal
2012 weak
2013 weak
2014 weak
2015 weak
2016 weak


In [92]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.5625

In [93]:
new_pred_class = test_dtc(X,y)

2001 normal
2002 normal
2003 normal
2004 normal
2005 normal
2006 normal
2007 normal
2008 normal
2009 strong
2010 normal
2011 normal
2012 weak
2013 normal
2014 strong
2015 strong
2016 normal


In [94]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.25

In [95]:
new_pred_class = test_rfc(X,y)

2001 weak
2002 normal
2003 normal
2004 normal
2005 normal
2006 normal
2007 weak
2008 normal
2009 normal
2010 normal
2011 normal
2012 weak
2013 normal
2014 normal
2015 normal
2016 normal


In [96]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.25

In [97]:
new_pred_class = test_mlp(X,y)

2001 weak
2002 weak
2003 weak
2004 weak
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 weak
2011 weak
2012 weak
2013 weak
2014 normal
2015 normal
2016 normal


In [98]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.625

In [99]:
new_pred_class = test_adc(X,y)

2001 normal
2002 normal
2003 weak
2004 normal
2005 normal
2006 normal
2007 normal
2008 normal
2009 normal
2010 normal
2011 normal
2012 normal
2013 normal
2014 weak
2015 normal
2016 normal


In [100]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.3125

In [101]:
new_pred_class = test_gnb(X,y)

2001 normal
2002 normal
2003 normal
2004 normal
2005 normal
2006 normal
2007 normal
2008 normal
2009 normal
2010 normal
2011 normal
2012 normal
2013 normal
2014 weak
2015 weak
2016 weak


In [102]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.375

### Prepare data for coast3

In [103]:
coast3_sst_transformed_df = create_transformed_df(coast3_sst_scaled_df, "sst_")
coast3_sst_transformed_df.head()

,sst_5N-49W_7,sst_5N-49W_8,sst_5N-49W_9,sst_5N-49W_10,sst_5N-49W_11,sst_5N-49W_12,sst_5N-47W_7,sst_5N-47W_8,sst_5N-47W_9,sst_5N-47W_10,sst_5N-47W_11,sst_5N-47W_12,sst_5N-45W_7,sst_5N-45W_8,sst_5N-45W_9,sst_5N-45W_10,sst_5N-45W_11,sst_5N-45W_12,sst_5N-43W_7,sst_5N-43W_8,sst_5N-43W_9,sst_5N-43W_10,sst_5N-43W_11,sst_5N-43W_12,sst_5N-41W_7,sst_5N-41W_8,sst_5N-41W_9,sst_5N-41W_10,sst_5N-41W_11,sst_5N-41W_12,sst_5N-39W_7,sst_5N-39W_8,sst_5N-39W_9,sst_5N-39W_10,sst_5N-39W_11,sst_5N-39W_12,sst_5N-37W_7,sst_5N-37W_8,sst_5N-37W_9,sst_5N-37W_10,sst_5N-37W_11,sst_5N-37W_12,sst_5N-35W_7,sst_5N-35W_8,sst_5N-35W_9,sst_5N-35W_10,sst_5N-35W_11,sst_5N-35W_12,sst_5N-33W_7,sst_5N-33W_8,sst_5N-33W_9,sst_5N-33W_10,sst_5N-33W_11,sst_5N-33W_12,sst_5N-31W_7,sst_5N-31W_8,sst_5N-31W_9,sst_5N-31W_10,sst_5N-31W_11,sst_5N-31W_12,sst_5N-29W_7,sst_5N-29W_8,sst_5N-29W_9,sst_5N-29W_10,sst_5N-29W_11,sst_5N-29W_12,sst_5N-27W_7,sst_5N-27W_8,sst_5N-27W_9,sst_5N-27W_10,sst_5N-27W_11,sst_5N-27W_12,sst_5N-25W_7,sst_5N-25W_8,sst_5N-25W_9,sst_5N-25W_10,sst_5N-25W_11,sst_5N-25W_12,sst_5N-23W_7,sst_5N-23W_8,sst_5N-23W_9,sst_5N-23W_10,sst_5N-23W_11,sst_5N-23W_12,sst_5N-21W_7,sst_5N-21W_8,sst_5N-21W_9,sst_5N-21W_10,sst_5N-21W_11,sst_5N-21W_12,sst_5N-19W_7,sst_5N-19W_8,sst_5N-19W_9,sst_5N-19W_10,sst_5N-19W_11,sst_5N-19W_12,sst_3N-49W_7,sst_3N-49W_8,sst_3N-49W_9,sst_3N-49W_10,sst_3N-49W_11,sst_3N-49W_12,sst_3N-47W_7,sst_3N-47W_8,sst_3N-47W_9,sst_3N-47W_10,sst_3N-47W_11,sst_3N-47W_12,sst_3N-45W_7,sst_3N-45W_8,sst_3N-45W_9,sst_3N-45W_10,sst_3N-45W_11,sst_3N-45W_12,sst_3N-43W_7,sst_3N-43W_8,sst_3N-43W_9,sst_3N-43W_10,sst_3N-43W_11,sst_3N-43W_12,sst_3N-41W_7,sst_3N-41W_8,sst_3N-41W_9,sst_3N-41W_10,sst_3N-41W_11,sst_3N-41W_12,sst_3N-39W_7,sst_3N-39W_8,sst_3N-39W_9,sst_3N-39W_10,sst_3N-39W_11,sst_3N-39W_12,sst_3N-37W_7,sst_3N-37W_8,sst_3N-37W_9,sst_3N-37W_10,sst_3N-37W_11,sst_3N-37W_12,sst_3N-35W_7,sst_3N-35W_8,sst_3N-35W_9,sst_3N-35W_10,sst_3N-35W_11,sst_3N-35W_12,sst_3N-33W_7,sst_3N-33W_8,sst_3N-33W_9,sst_3N-33W_10,sst_3N-33W_11,sst_3N-33W_12,sst_3N-31W_7,sst_3N-31W_8,sst_3N-31W_9,sst_3N-31W_10,sst_3N-31W_11,sst_3N-31W_12,sst_3N-29W_7,sst_3N-29W_8,sst_3N-29W_9,sst_3N-29W_10,sst_3N-29W_11,sst_3N-29W_12,sst_3N-27W_7,sst_3N-27W_8,sst_3N-27W_9,sst_3N-27W_10,sst_3N-27W_11,sst_3N-27W_12,sst_3N-25W_7,sst_3N-25W_8,sst_3N-25W_9,sst_3N-25W_10,sst_3N-25W_11,sst_3N-25W_12,sst_3N-23W_7,sst_3N-23W_8,sst_3N-23W_9,sst_3N-23W_10,sst_3N-23W_11,sst_3N-23W_12,sst_3N-21W_7,sst_3N-21W_8,sst_3N-21W_9,sst_3N-21W_10,sst_3N-21W_11,sst_3N-21W_12,sst_3N-19W_7,sst_3N-19W_8,sst_3N-19W_9,sst_3N-19W_10,sst_3N-19W_11,sst_3N-19W_12,sst_1N-49W_7,sst_1N-49W_8,sst_1N-49W_9,sst_1N-49W_10,sst_1N-49W_11,sst_1N-49W_12,sst_1N-47W_7,sst_1N-47W_8,sst_1N-47W_9,sst_1N-47W_10,sst_1N-47W_11,sst_1N-47W_12,sst_1N-45W_7,sst_1N-45W_8,sst_1N-45W_9,sst_1N-45W_10,sst_1N-45W_11,sst_1N-45W_12,sst_1N-43W_7,sst_1N-43W_8,sst_1N-43W_9,sst_1N-43W_10,sst_1N-43W_11,sst_1N-43W_12,sst_1N-41W_7,sst_1N-41W_8,sst_1N-41W_9,sst_1N-41W_10,sst_1N-41W_11,sst_1N-41W_12,sst_1N-39W_7,sst_1N-39W_8,sst_1N-39W_9,sst_1N-39W_10,sst_1N-39W_11,sst_1N-39W_12,sst_1N-37W_7,sst_1N-37W_8,sst_1N-37W_9,sst_1N-37W_10,sst_1N-37W_11,sst_1N-37W_12,sst_1N-35W_7,sst_1N-35W_8,sst_1N-35W_9,sst_1N-35W_10,sst_1N-35W_11,sst_1N-35W_12,sst_1N-33W_7,sst_1N-33W_8,sst_1N-33W_9,sst_1N-33W_10,sst_1N-33W_11,sst_1N-33W_12,sst_1N-31W_7,sst_1N-31W_8,sst_1N-31W_9,sst_1N-31W_10,...,sst_1S-19W_9,sst_1S-19W_10,sst_1S-19W_11,sst_1S-19W_12,sst_3S-43W_7,sst_3S-43W_8,sst_3S-43W_9,sst_3S-43W_10,sst_3S-43W_11,sst_3S-43W_12,sst_3S-41W_7,sst_3S-41W_8,sst_3S-41W_9,sst_3S-41W_10,sst_3S-41W_11,sst_3S-41W_12,sst_3S-39W_7,sst_3S-39W_8,sst_3S-39W_9,sst_3S-39W_10,sst_3S-39W_11,sst_3S-39W_12,sst_3S-37W_7,sst_3S-37W_8,sst_3S-37W_9,sst_3S-37W_10,sst_3S-37W_11,sst_3S-37W_12,sst_3S-35W_7,sst_3S-35W_8,sst_3S-35W_9,sst_3S-35W_10,sst_3S-35W_11,sst_3S-35W_12,sst_3S-33W_7,sst_3S-33W_8,sst_3S-33W_9,sst_3S-33W_10,sst_3S-33W_11,sst_3S-33W_12,sst_3S-31W_7,sst_3S-31W_8,sst_3S-31W_9,sst_3S-31W_10,sst_3S-31W_11,sst_3S-31W_12

In [104]:
coast3_pws_transformed_df = create_transformed_df(coast3_pws_scaled_df, "pws_")
coast3_pws_transformed_df.head()

,pws_5N-49W_7,pws_5N-49W_8,pws_5N-49W_9,pws_5N-49W_10,pws_5N-49W_11,pws_5N-49W_12,pws_5N-47W_7,pws_5N-47W_8,pws_5N-47W_9,pws_5N-47W_10,pws_5N-47W_11,pws_5N-47W_12,pws_5N-45W_7,pws_5N-45W_8,pws_5N-45W_9,pws_5N-45W_10,pws_5N-45W_11,pws_5N-45W_12,pws_5N-43W_7,pws_5N-43W_8,pws_5N-43W_9,pws_5N-43W_10,pws_5N-43W_11,pws_5N-43W_12,pws_5N-41W_7,pws_5N-41W_8,pws_5N-41W_9,pws_5N-41W_10,pws_5N-41W_11,pws_5N-41W_12,pws_5N-39W_7,pws_5N-39W_8,pws_5N-39W_9,pws_5N-39W_10,pws_5N-39W_11,pws_5N-39W_12,pws_5N-37W_7,pws_5N-37W_8,pws_5N-37W_9,pws_5N-37W_10,pws_5N-37W_11,pws_5N-37W_12,pws_5N-35W_7,pws_5N-35W_8,pws_5N-35W_9,pws_5N-35W_10,pws_5N-35W_11,pws_5N-35W_12,pws_5N-33W_7,pws_5N-33W_8,pws_5N-33W_9,pws_5N-33W_10,pws_5N-33W_11,pws_5N-33W_12,pws_5N-31W_7,pws_5N-31W_8,pws_5N-31W_9,pws_5N-31W_10,pws_5N-31W_11,pws_5N-31W_12,pws_5N-29W_7,pws_5N-29W_8,pws_5N-29W_9,pws_5N-29W_10,pws_5N-29W_11,pws_5N-29W_12,pws_5N-27W_7,pws_5N-27W_8,pws_5N-27W_9,pws_5N-27W_10,pws_5N-27W_11,pws_5N-27W_12,pws_5N-25W_7,pws_5N-25W_8,pws_5N-25W_9,pws_5N-25W_10,pws_5N-25W_11,pws_5N-25W_12,pws_5N-23W_7,pws_5N-23W_8,pws_5N-23W_9,pws_5N-23W_10,pws_5N-23W_11,pws_5N-23W_12,pws_5N-21W_7,pws_5N-21W_8,pws_5N-21W_9,pws_5N-21W_10,pws_5N-21W_11,pws_5N-21W_12,pws_5N-19W_7,pws_5N-19W_8,pws_5N-19W_9,pws_5N-19W_10,pws_5N-19W_11,pws_5N-19W_12,pws_3N-49W_7,pws_3N-49W_8,pws_3N-49W_9,pws_3N-49W_10,pws_3N-49W_11,pws_3N-49W_12,pws_3N-47W_7,pws_3N-47W_8,pws_3N-47W_9,pws_3N-47W_10,pws_3N-47W_11,pws_3N-47W_12,pws_3N-45W_7,pws_3N-45W_8,pws_3N-45W_9,pws_3N-45W_10,pws_3N-45W_11,pws_3N-45W_12,pws_3N-43W_7,pws_3N-43W_8,pws_3N-43W_9,pws_3N-43W_10,pws_3N-43W_11,pws_3N-43W_12,pws_3N-41W_7,pws_3N-41W_8,pws_3N-41W_9,pws_3N-41W_10,pws_3N-41W_11,pws_3N-41W_12,pws_3N-39W_7,pws_3N-39W_8,pws_3N-39W_9,pws_3N-39W_10,pws_3N-39W_11,pws_3N-39W_12,pws_3N-37W_7,pws_3N-37W_8,pws_3N-37W_9,pws_3N-37W_10,pws_3N-37W_11,pws_3N-37W_12,pws_3N-35W_7,pws_3N-35W_8,pws_3N-35W_9,pws_3N-35W_10,pws_3N-35W_11,pws_3N-35W_12,pws_3N-33W_7,pws_3N-33W_8,pws_3N-33W_9,pws_3N-33W_10,pws_3N-33W_11,pws_3N-33W_12,pws_3N-31W_7,pws_3N-31W_8,pws_3N-31W_9,pws_3N-31W_10,pws_3N-31W_11,pws_3N-31W_12,pws_3N-29W_7,pws_3N-29W_8,pws_3N-29W_9,pws_3N-29W_10,pws_3N-29W_11,pws_3N-29W_12,pws_3N-27W_7,pws_3N-27W_8,pws_3N-27W_9,pws_3N-27W_10,pws_3N-27W_11,pws_3N-27W_12,pws_3N-25W_7,pws_3N-25W_8,pws_3N-25W_9,pws_3N-25W_10,pws_3N-25W_11,pws_3N-25W_12,pws_3N-23W_7,pws_3N-23W_8,pws_3N-23W_9,pws_3N-23W_10,pws_3N-23W_11,pws_3N-23W_12,pws_3N-21W_7,pws_3N-21W_8,pws_3N-21W_9,pws_3N-21W_10,pws_3N-21W_11,pws_3N-21W_12,pws_3N-19W_7,pws_3N-19W_8,pws_3N-19W_9,pws_3N-19W_10,pws_3N-19W_11,pws_3N-19W_12,pws_1N-49W_7,pws_1N-49W_8,pws_1N-49W_9,pws_1N-49W_10,pws_1N-49W_11,pws_1N-49W_12,pws_1N-47W_7,pws_1N-47W_8,pws_1N-47W_9,pws_1N-47W_10,pws_1N-47W_11,pws_1N-47W_12,pws_1N-45W_7,pws_1N-45W_8,pws_1N-45W_9,pws_1N-45W_10,pws_1N-45W_11,pws_1N-45W_12,pws_1N-43W_7,pws_1N-43W_8,pws_1N-43W_9,pws_1N-43W_10,pws_1N-43W_11,pws_1N-43W_12,pws_1N-41W_7,pws_1N-41W_8,pws_1N-41W_9,pws_1N-41W_10,pws_1N-41W_11,pws_1N-41W_12,pws_1N-39W_7,pws_1N-39W_8,pws_1N-39W_9,pws_1N-39W_10,pws_1N-39W_11,pws_1N-39W_12,pws_1N-37W_7,pws_1N-37W_8,pws_1N-37W_9,pws_1N-37W_10,pws_1N-37W_11,pws_1N-37W_12,pws_1N-35W_7,pws_1N-35W_8,pws_1N-35W_9,pws_1N-35W_10,pws_1N-35W_11,pws_1N-35W_12,pws_1N-33W_7,pws_1N-33W_8,pws_1N-33W_9,pws_1N-33W_10,pws_1N-33W_11,pws_1N-33W_12,pws_1N-31W_7,pws_1N-31W_8,pws_1N-31W_9,pws_1N-31W_10,...,pws_1S-19W_9,pws_1S-19W_10,pws_1S-19W_11,pws_1S-19W_12,pws_3S-43W_7,pws_3S-43W_8,pws_3S-43W_9,pws_3S-43W_10,pws_3S-43W_11,pws_3S-43W_12,pws_3S-41W_7,pws_3S-41W_8,pws_3S-41W_9,pws_3S-41W_10,pws_3S-41W_11,pws_3S-41W_12,pws_3S-39W_7,pws_3S-39W_8,pws_3S-39W_9,pws_3S-39W_10,pws_3S-39W_11,pws_3S-39W_12,pws_3S-37W_7,pws_3S-37W_8,pws_3S-37W_9,pws_3S-37W_10,pws_3S-37W_11,pws_3S-37W_12,pws_3S-35W_7,pws_3S-35W_8,pws_3S-35W_9,pws_3S-35W_10,pws_3S-35W_11,pws_3S-35W_12,pws_3S-33W_7,pws_3S-33W_8,pws_3S-33W_9,pws_3S-33W_10,pws_3S-33W_11,pws_3S-33W_12,pws_3S-31W_7,pws_3S-31W_8,pws_3S-31W_9,pws_3S-31W_10,pws_3S-31W_11,pws_3S-31W_12

In [105]:
df_new = pd.concat([coast3_sst_transformed_df, coast3_pws_transformed_df], axis=1)
df_new.head()


,sst_5N-49W_7,sst_5N-49W_8,sst_5N-49W_9,sst_5N-49W_10,sst_5N-49W_11,sst_5N-49W_12,sst_5N-47W_7,sst_5N-47W_8,sst_5N-47W_9,sst_5N-47W_10,sst_5N-47W_11,sst_5N-47W_12,sst_5N-45W_7,sst_5N-45W_8,sst_5N-45W_9,sst_5N-45W_10,sst_5N-45W_11,sst_5N-45W_12,sst_5N-43W_7,sst_5N-43W_8,sst_5N-43W_9,sst_5N-43W_10,sst_5N-43W_11,sst_5N-43W_12,sst_5N-41W_7,sst_5N-41W_8,sst_5N-41W_9,sst_5N-41W_10,sst_5N-41W_11,sst_5N-41W_12,sst_5N-39W_7,sst_5N-39W_8,sst_5N-39W_9,sst_5N-39W_10,sst_5N-39W_11,sst_5N-39W_12,sst_5N-37W_7,sst_5N-37W_8,sst_5N-37W_9,sst_5N-37W_10,sst_5N-37W_11,sst_5N-37W_12,sst_5N-35W_7,sst_5N-35W_8,sst_5N-35W_9,sst_5N-35W_10,sst_5N-35W_11,sst_5N-35W_12,sst_5N-33W_7,sst_5N-33W_8,sst_5N-33W_9,sst_5N-33W_10,sst_5N-33W_11,sst_5N-33W_12,sst_5N-31W_7,sst_5N-31W_8,sst_5N-31W_9,sst_5N-31W_10,sst_5N-31W_11,sst_5N-31W_12,sst_5N-29W_7,sst_5N-29W_8,sst_5N-29W_9,sst_5N-29W_10,sst_5N-29W_11,sst_5N-29W_12,sst_5N-27W_7,sst_5N-27W_8,sst_5N-27W_9,sst_5N-27W_10,sst_5N-27W_11,sst_5N-27W_12,sst_5N-25W_7,sst_5N-25W_8,sst_5N-25W_9,sst_5N-25W_10,sst_5N-25W_11,sst_5N-25W_12,sst_5N-23W_7,sst_5N-23W_8,sst_5N-23W_9,sst_5N-23W_10,sst_5N-23W_11,sst_5N-23W_12,sst_5N-21W_7,sst_5N-21W_8,sst_5N-21W_9,sst_5N-21W_10,sst_5N-21W_11,sst_5N-21W_12,sst_5N-19W_7,sst_5N-19W_8,sst_5N-19W_9,sst_5N-19W_10,sst_5N-19W_11,sst_5N-19W_12,sst_3N-49W_7,sst_3N-49W_8,sst_3N-49W_9,sst_3N-49W_10,sst_3N-49W_11,sst_3N-49W_12,sst_3N-47W_7,sst_3N-47W_8,sst_3N-47W_9,sst_3N-47W_10,sst_3N-47W_11,sst_3N-47W_12,sst_3N-45W_7,sst_3N-45W_8,sst_3N-45W_9,sst_3N-45W_10,sst_3N-45W_11,sst_3N-45W_12,sst_3N-43W_7,sst_3N-43W_8,sst_3N-43W_9,sst_3N-43W_10,sst_3N-43W_11,sst_3N-43W_12,sst_3N-41W_7,sst_3N-41W_8,sst_3N-41W_9,sst_3N-41W_10,sst_3N-41W_11,sst_3N-41W_12,sst_3N-39W_7,sst_3N-39W_8,sst_3N-39W_9,sst_3N-39W_10,sst_3N-39W_11,sst_3N-39W_12,sst_3N-37W_7,sst_3N-37W_8,sst_3N-37W_9,sst_3N-37W_10,sst_3N-37W_11,sst_3N-37W_12,sst_3N-35W_7,sst_3N-35W_8,sst_3N-35W_9,sst_3N-35W_10,sst_3N-35W_11,sst_3N-35W_12,sst_3N-33W_7,sst_3N-33W_8,sst_3N-33W_9,sst_3N-33W_10,sst_3N-33W_11,sst_3N-33W_12,sst_3N-31W_7,sst_3N-31W_8,sst_3N-31W_9,sst_3N-31W_10,sst_3N-31W_11,sst_3N-31W_12,sst_3N-29W_7,sst_3N-29W_8,sst_3N-29W_9,sst_3N-29W_10,sst_3N-29W_11,sst_3N-29W_12,sst_3N-27W_7,sst_3N-27W_8,sst_3N-27W_9,sst_3N-27W_10,sst_3N-27W_11,sst_3N-27W_12,sst_3N-25W_7,sst_3N-25W_8,sst_3N-25W_9,sst_3N-25W_10,sst_3N-25W_11,sst_3N-25W_12,sst_3N-23W_7,sst_3N-23W_8,sst_3N-23W_9,sst_3N-23W_10,sst_3N-23W_11,sst_3N-23W_12,sst_3N-21W_7,sst_3N-21W_8,sst_3N-21W_9,sst_3N-21W_10,sst_3N-21W_11,sst_3N-21W_12,sst_3N-19W_7,sst_3N-19W_8,sst_3N-19W_9,sst_3N-19W_10,sst_3N-19W_11,sst_3N-19W_12,sst_1N-49W_7,sst_1N-49W_8,sst_1N-49W_9,sst_1N-49W_10,sst_1N-49W_11,sst_1N-49W_12,sst_1N-47W_7,sst_1N-47W_8,sst_1N-47W_9,sst_1N-47W_10,sst_1N-47W_11,sst_1N-47W_12,sst_1N-45W_7,sst_1N-45W_8,sst_1N-45W_9,sst_1N-45W_10,sst_1N-45W_11,sst_1N-45W_12,sst_1N-43W_7,sst_1N-43W_8,sst_1N-43W_9,sst_1N-43W_10,sst_1N-43W_11,sst_1N-43W_12,sst_1N-41W_7,sst_1N-41W_8,sst_1N-41W_9,sst_1N-41W_10,sst_1N-41W_11,sst_1N-41W_12,sst_1N-39W_7,sst_1N-39W_8,sst_1N-39W_9,sst_1N-39W_10,sst_1N-39W_11,sst_1N-39W_12,sst_1N-37W_7,sst_1N-37W_8,sst_1N-37W_9,sst_1N-37W_10,sst_1N-37W_11,sst_1N-37W_12,sst_1N-35W_7,sst_1N-35W_8,sst_1N-35W_9,sst_1N-35W_10,sst_1N-35W_11,sst_1N-35W_12,sst_1N-33W_7,sst_1N-33W_8,sst_1N-33W_9,sst_1N-33W_10,sst_1N-33W_11,sst_1N-33W_12,sst_1N-31W_7,sst_1N-31W_8,sst_1N-31W_9,sst_1N-31W_10,...,pws_1S-19W_9,pws_1S-19W_10,pws_1S-19W_11,pws_1S-19W_12,pws_3S-43W_7,pws_3S-43W_8,pws_3S-43W_9,pws_3S-43W_10,pws_3S-43W_11,pws_3S-43W_12,pws_3S-41W_7,pws_3S-41W_8,pws_3S-41W_9,pws_3S-41W_10,pws_3S-41W_11,pws_3S-41W_12,pws_3S-39W_7,pws_3S-39W_8,pws_3S-39W_9,pws_3S-39W_10,pws_3S-39W_11,pws_3S-39W_12,pws_3S-37W_7,pws_3S-37W_8,pws_3S-37W_9,pws_3S-37W_10,pws_3S-37W_11,pws_3S-37W_12,pws_3S-35W_7,pws_3S-35W_8,pws_3S-35W_9,pws_3S-35W_10,pws_3S-35W_11,pws_3S-35W_12,pws_3S-33W_7,pws_3S-33W_8,pws_3S-33W_9,pws_3S-33W_10,pws_3S-33W_11,pws_3S-33W_12,pws_3S-31W_7,pws_3S-31W_8,pws_3S-31W_9,pws_3S-31W_10,pws_3S-31W_11,pws_3S-31W_12

In [106]:
dataset_df = pd.concat([df_new, rainy_classified_df], axis=1)
dataset_df.head()

,sst_5N-49W_7,sst_5N-49W_8,sst_5N-49W_9,sst_5N-49W_10,sst_5N-49W_11,sst_5N-49W_12,sst_5N-47W_7,sst_5N-47W_8,sst_5N-47W_9,sst_5N-47W_10,sst_5N-47W_11,sst_5N-47W_12,sst_5N-45W_7,sst_5N-45W_8,sst_5N-45W_9,sst_5N-45W_10,sst_5N-45W_11,sst_5N-45W_12,sst_5N-43W_7,sst_5N-43W_8,sst_5N-43W_9,sst_5N-43W_10,sst_5N-43W_11,sst_5N-43W_12,sst_5N-41W_7,sst_5N-41W_8,sst_5N-41W_9,sst_5N-41W_10,sst_5N-41W_11,sst_5N-41W_12,sst_5N-39W_7,sst_5N-39W_8,sst_5N-39W_9,sst_5N-39W_10,sst_5N-39W_11,sst_5N-39W_12,sst_5N-37W_7,sst_5N-37W_8,sst_5N-37W_9,sst_5N-37W_10,sst_5N-37W_11,sst_5N-37W_12,sst_5N-35W_7,sst_5N-35W_8,sst_5N-35W_9,sst_5N-35W_10,sst_5N-35W_11,sst_5N-35W_12,sst_5N-33W_7,sst_5N-33W_8,sst_5N-33W_9,sst_5N-33W_10,sst_5N-33W_11,sst_5N-33W_12,sst_5N-31W_7,sst_5N-31W_8,sst_5N-31W_9,sst_5N-31W_10,sst_5N-31W_11,sst_5N-31W_12,sst_5N-29W_7,sst_5N-29W_8,sst_5N-29W_9,sst_5N-29W_10,sst_5N-29W_11,sst_5N-29W_12,sst_5N-27W_7,sst_5N-27W_8,sst_5N-27W_9,sst_5N-27W_10,sst_5N-27W_11,sst_5N-27W_12,sst_5N-25W_7,sst_5N-25W_8,sst_5N-25W_9,sst_5N-25W_10,sst_5N-25W_11,sst_5N-25W_12,sst_5N-23W_7,sst_5N-23W_8,sst_5N-23W_9,sst_5N-23W_10,sst_5N-23W_11,sst_5N-23W_12,sst_5N-21W_7,sst_5N-21W_8,sst_5N-21W_9,sst_5N-21W_10,sst_5N-21W_11,sst_5N-21W_12,sst_5N-19W_7,sst_5N-19W_8,sst_5N-19W_9,sst_5N-19W_10,sst_5N-19W_11,sst_5N-19W_12,sst_3N-49W_7,sst_3N-49W_8,sst_3N-49W_9,sst_3N-49W_10,sst_3N-49W_11,sst_3N-49W_12,sst_3N-47W_7,sst_3N-47W_8,sst_3N-47W_9,sst_3N-47W_10,sst_3N-47W_11,sst_3N-47W_12,sst_3N-45W_7,sst_3N-45W_8,sst_3N-45W_9,sst_3N-45W_10,sst_3N-45W_11,sst_3N-45W_12,sst_3N-43W_7,sst_3N-43W_8,sst_3N-43W_9,sst_3N-43W_10,sst_3N-43W_11,sst_3N-43W_12,sst_3N-41W_7,sst_3N-41W_8,sst_3N-41W_9,sst_3N-41W_10,sst_3N-41W_11,sst_3N-41W_12,sst_3N-39W_7,sst_3N-39W_8,sst_3N-39W_9,sst_3N-39W_10,sst_3N-39W_11,sst_3N-39W_12,sst_3N-37W_7,sst_3N-37W_8,sst_3N-37W_9,sst_3N-37W_10,sst_3N-37W_11,sst_3N-37W_12,sst_3N-35W_7,sst_3N-35W_8,sst_3N-35W_9,sst_3N-35W_10,sst_3N-35W_11,sst_3N-35W_12,sst_3N-33W_7,sst_3N-33W_8,sst_3N-33W_9,sst_3N-33W_10,sst_3N-33W_11,sst_3N-33W_12,sst_3N-31W_7,sst_3N-31W_8,sst_3N-31W_9,sst_3N-31W_10,sst_3N-31W_11,sst_3N-31W_12,sst_3N-29W_7,sst_3N-29W_8,sst_3N-29W_9,sst_3N-29W_10,sst_3N-29W_11,sst_3N-29W_12,sst_3N-27W_7,sst_3N-27W_8,sst_3N-27W_9,sst_3N-27W_10,sst_3N-27W_11,sst_3N-27W_12,sst_3N-25W_7,sst_3N-25W_8,sst_3N-25W_9,sst_3N-25W_10,sst_3N-25W_11,sst_3N-25W_12,sst_3N-23W_7,sst_3N-23W_8,sst_3N-23W_9,sst_3N-23W_10,sst_3N-23W_11,sst_3N-23W_12,sst_3N-21W_7,sst_3N-21W_8,sst_3N-21W_9,sst_3N-21W_10,sst_3N-21W_11,sst_3N-21W_12,sst_3N-19W_7,sst_3N-19W_8,sst_3N-19W_9,sst_3N-19W_10,sst_3N-19W_11,sst_3N-19W_12,sst_1N-49W_7,sst_1N-49W_8,sst_1N-49W_9,sst_1N-49W_10,sst_1N-49W_11,sst_1N-49W_12,sst_1N-47W_7,sst_1N-47W_8,sst_1N-47W_9,sst_1N-47W_10,sst_1N-47W_11,sst_1N-47W_12,sst_1N-45W_7,sst_1N-45W_8,sst_1N-45W_9,sst_1N-45W_10,sst_1N-45W_11,sst_1N-45W_12,sst_1N-43W_7,sst_1N-43W_8,sst_1N-43W_9,sst_1N-43W_10,sst_1N-43W_11,sst_1N-43W_12,sst_1N-41W_7,sst_1N-41W_8,sst_1N-41W_9,sst_1N-41W_10,sst_1N-41W_11,sst_1N-41W_12,sst_1N-39W_7,sst_1N-39W_8,sst_1N-39W_9,sst_1N-39W_10,sst_1N-39W_11,sst_1N-39W_12,sst_1N-37W_7,sst_1N-37W_8,sst_1N-37W_9,sst_1N-37W_10,sst_1N-37W_11,sst_1N-37W_12,sst_1N-35W_7,sst_1N-35W_8,sst_1N-35W_9,sst_1N-35W_10,sst_1N-35W_11,sst_1N-35W_12,sst_1N-33W_7,sst_1N-33W_8,sst_1N-33W_9,sst_1N-33W_10,sst_1N-33W_11,sst_1N-33W_12,sst_1N-31W_7,sst_1N-31W_8,sst_1N-31W_9,sst_1N-31W_10,...,pws_1S-19W_10,pws_1S-19W_11,pws_1S-19W_12,pws_3S-43W_7,pws_3S-43W_8,pws_3S-43W_9,pws_3S-43W_10,pws_3S-43W_11,pws_3S-43W_12,pws_3S-41W_7,pws_3S-41W_8,pws_3S-41W_9,pws_3S-41W_10,pws_3S-41W_11,pws_3S-41W_12,pws_3S-39W_7,pws_3S-39W_8,pws_3S-39W_9,pws_3S-39W_10,pws_3S-39W_11,pws_3S-39W_12,pws_3S-37W_7,pws_3S-37W_8,pws_3S-37W_9,pws_3S-37W_10,pws_3S-37W_11,pws_3S-37W_12,pws_3S-35W_7,pws_3S-35W_8,pws_3S-35W_9,pws_3S-35W_10,pws_3S-35W_11,pws_3S-35W_12,pws_3S-33W_7,pws_3S-33W_8,pws_3S-33W_9,pws_3S-33W_10,pws_3S-33W_11,pws_3S-33W_12,pws_3S-31W_7,pws_3S-31W_8,pws_3S-31W_9,pws_3S-31W_10,pws_3S-31W_11,pws_3S-31W_12,pws_3S-29W_7

In [107]:
#Removing 2017 due we don't have full 2018 available
dataset_df = dataset_df.loc['1973-01-01':'2016-12-01']

In [108]:
#Train
train_df = dataset_df.loc['1973-01-01':'2000-12-01']

test_df = dataset_df.loc['2000-01-01':'2016-12-01']

#Dropt last column to be the X values
feature_cols = dataset_df.columns
feature_cols=feature_cols.drop('classes')

# Input data(X)
X = train_df.loc[:, feature_cols]

# Output data(y)
y = train_df['classes']

X_test = test_df.loc[:, feature_cols]


### Testing classifiers for coast3

In [109]:
new_pred_class = test_knn(X,y)

2001 normal
2001 normal
2001 normal
2001 weak
2001 normal
2001 weak
2001 weak
2001 weak
2001 normal
2001 normal
2001 normal
2001 weak
2001 weak
2001 weak
2001 weak
2001 weak


In [110]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.5

In [111]:
new_pred_class = test_svc(X,y)

2001 weak
2002 weak
2003 weak
2004 weak
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 weak
2011 weak
2012 weak
2013 weak
2014 weak
2015 weak
2016 weak


In [112]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.8125

In [113]:
new_pred_class = test_gpc(X,y)

2001 normal
2002 weak
2003 normal
2004 weak
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 normal
2011 weak
2012 weak
2013 weak
2014 weak
2015 weak
2016 weak


In [114]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.625

In [115]:
new_pred_class = test_dtc(X,y)

2001 weak
2002 normal
2003 normal
2004 strong
2005 strong
2006 strong
2007 weak
2008 normal
2009 weak
2010 strong
2011 strong
2012 weak
2013 normal
2014 strong
2015 strong
2016 strong


In [116]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.3125

In [117]:
new_pred_class = test_rfc(X,y)

2001 weak
2002 normal
2003 normal
2004 normal
2005 normal
2006 normal
2007 normal
2008 normal
2009 normal
2010 normal
2011 normal
2012 normal
2013 normal
2014 weak
2015 normal
2016 normal


In [118]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.3125

In [119]:
new_pred_class = test_mlp(X,y)

2001 weak
2002 weak
2003 weak
2004 weak
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 weak
2011 weak
2012 weak
2013 weak
2014 normal
2015 normal
2016 normal


In [120]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.625

In [121]:
new_pred_class = test_adc(X,y)

2001 weak
2002 weak
2003 weak
2004 weak
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 weak
2011 weak
2012 normal
2013 weak
2014 normal
2015 normal
2016 weak


In [122]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.625

In [123]:
new_pred_class = test_gnb(X,y)

2001 normal
2002 normal
2003 normal
2004 normal
2005 normal
2006 normal
2007 normal
2008 weak
2009 normal
2010 normal
2011 normal
2012 weak
2013 weak
2014 weak
2015 weak
2016 weak


In [124]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.4375